In [1]:
# This produces the dataframe for RB

In [2]:
## Notes on the NFL Library ##
# the NFL python library seem to not work on Tuesday probably due to updates (not confirmed)
# unbalanced dataframe - pfr stats start at 2018; all other stats go back to 2017

In [3]:
## Notes: on the fantasypros data
# - I was unable to validate hash rows for the redzone data  

In [4]:
## Required installations
# 

In [5]:
## REQUIRED ACTIONS - Include in a README doc ## 
# modify the season start date in the 'get_current_week' function
# modify the number of weeks if the NFL adds regular season games to the schedule

In [6]:
# import the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import glob
from IPython.display import display, HTML
from datetime import datetime
import nfl_data_py as nfl
import os
import re
import time
from random import sample, uniform, seed
import io
from rapidfuzz import fuzz, process
import numpy as np
import hashlib
import shutil

In [7]:
# Set Pandas options to display all columns in a single row without wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [8]:
# Function to calculate the current week of the NFL season
def get_current_week():
    current_date = datetime.now()
    season_start_date = datetime(2024, 9, 4)  # Update for the season start
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

# Define the current NFL year, week, and season type
current_year = datetime.now().year
current_week = get_current_week()
seasontype = 2 if current_week <= 18 else 3  # Regular season or playoffs

In [9]:
### Begin: Python NFL Library Dataframe ###

In [10]:
# define the years to pull
# nfl.import_weekly_data(years, columns, downcast)
def get_year_range(current_year, current_week, start_year=2017):
    if current_week <= 18:  # Regular season
        return list(range(start_year, current_year + 1))
    else:  # Playoffs
        return list(range(start_year, current_year))

# Use the function
years = get_year_range(current_year, current_week)

In [11]:
# define the base columns. 
base_columns = [
    'season', 'season_type', 'week', 'player_id', 'player_name', 
    'position', 'position_group', 'recent_team'
]

In [12]:
# Import the player IDs from nfl.import_ids() - without parameters
ids_data = nfl.import_ids()

# Drop the unnecessary columns
columns_to_drop = [
    'position', 'team', 'birthdate', 'age', 'draft_year', 
    'draft_round', 'draft_pick', 'draft_ovr', 'twitter_username', 
    'height', 'weight', 'college', 'db_season'
]
ids_data = ids_data.drop(columns=columns_to_drop, errors='ignore')

# Display the resulting dataframe for review
# print(f"Columns after dropping unnecessary ones: {ids_data.columns.tolist()}")
# display(ids_data)

In [13]:
# import the weekly data from nfl.import_weekly_data(years, columns, downcast)
weekly_data = nfl.import_weekly_data(
    years=years,
    columns=base_columns
)

# display(weekly_data)

Downcasting floats.


In [14]:
## Output: a dataframe of ALL NFL athletes info and ids since 2017

# Merge the two dataframes on 'player_id' and 'gsis_id'
# Align column names for merging
ids_data = ids_data.rename(columns={'gsis_id': 'player_id'})  
id_dataframe = pd.merge(weekly_data, ids_data, on='player_id', how='inner')

# Assign the resulting dataframe to a variable
all_players_id_data = id_dataframe

# Display the resulting ID dataframe
# display(all_players_id_data)

In [15]:
## Output: a dataframe of NFL RB info and ids since 2017
# extract RB from the dataframe
# Create a new dataframe with only running backs
running_back_ids = all_players_id_data[all_players_id_data['position'] == 'RB']

# Display the resulting dataframe for review
print(f"Shape of merged dataframe: {running_back_ids.shape}")

# Display the resulting dataframe for review
# display(running_back_ids)

Shape of merged dataframe: (11082, 29)


In [16]:
## Output: a dataframe of NFL RB info, ids, and stats since 2017
# RB-specific columns (receiving-related)
# RB-specific columns (rushing- and receiving-related)
rb_columns = [
    'carries', 'attempts', 'targets', 'target_share', 'receptions', 'rushing_yards', 'receiving_yards', 
    'receiving_yards_after_catch', 'rushing_first_downs', 'receiving_first_downs', 'rushing_tds', 
    'receiving_tds', 'rushing_epa', 'receiving_epa', 'rushing_2pt_conversions', 'receiving_2pt_conversions'
]

# Pull RB-specific columns from weekly data
rb_stats = nfl.import_weekly_data(
    years=years,
    columns=['player_id', 'season', 'week'] + rb_columns  # Include keys for merging
)

# Merge RB-specific stats with running_back_ids
rb_ids_weekly_stats = pd.merge(
    running_back_ids,
    rb_stats,
    on=['player_id', 'season', 'week'],  # Ensure correct alignment
    how='inner'
)

# Display the resulting dataframe for review
print(f"Shape of merged dataframe: {rb_ids_weekly_stats.shape}")

# Row integrity check
print(
    f"Row count matches: {rb_ids_weekly_stats.shape[0] == running_back_ids.shape[0]}"
)

# Display the dataframe
display(rb_ids_weekly_stats)

# csv file
# rb_ids_weekly_stats.to_csv('rb_ids_weekly_stats.csv', index=False)

Downcasting floats.
Shape of merged dataframe: (11082, 45)
Row count matches: True


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,carries,attempts,targets,target_share,receptions,rushing_yards,receiving_yards,receiving_yards_after_catch,rushing_first_downs,receiving_first_downs,rushing_tds,receiving_tds,rushing_epa,receiving_epa,rushing_2pt_conversions,receiving_2pt_conversions
0,2017,REG,1,00-0023500,F.Gore,RB,RB,IND,7877,6a2b129d-a9e5-4131-b491-82269b323f77,9218.0,2282.0,232.0,frankgore/2506404,8479.0,7241.0,2848.0,411568.0,GoreFr00,frank-gore-1,4400.0,3205.0,NaN,7241.0,157341.0,5820.0,NaN,Frank Gore,frank gore,10,0,1,0.047619,1,42.0,10.0,14.0,1.0,0.0,0,0,-0.449218,0.374096,0,0
1,2017,REG,2,00-0023500,F.Gore,RB,RB,IND,7877,6a2b129d-a9e5-4131-b491-82269b323f77,9218.0,2282.0,232.0,frankgore/2506404,8479.0,7241.0,2848.0,411568.0,GoreFr00,frank-gore-1,4400.0,3205.0,NaN,7241.0,157341.0,5820.0,NaN,Frank Gore,frank gore,14,0,2,0.055556,0,46.0,0.0,0.0,2.0,0.0,1,0,-0.984420,-1.496430,0,0
2,2017,REG,3,00-0023500,F.Gore,RB,RB,IND,7877,6a2b129d-a9e5-4131-b491-82269b323f77,9218.0,2282.0,232.0,frankgore/2506404,8479.0,7241.0,2848.0,411568.0,GoreFr00,frank-gore-1,4400.0,3205.0,NaN,7241.0,157341.0,5820.0,NaN,Frank Gore,frank gore,25,0,1,0.041667,1,57.0,0.0,5.0,2.0,0.0,1,0,-5.832108,-0.788598,0,0
3,2017,REG,4,00-0023500,F.Gore,RB,RB,IND,7877,6a2b129d-a9e5-4131-b491-82269b323f77,9218.0,2282.0,232.0,frankgore/2506404,8479.0,7241.0,2848.0,411568.0,GoreFr00,frank-gore-1,4400.0,3205.0,NaN,7241.0,157341.0,5820.0,NaN,Frank Gore,frank gore,12,0,3,0.107143,3,46.0,34.0,43.0,1.0,2.0,0,0,-1.547445,1.718400,0,0
4,2017,REG,5,00-0023500,F.Gore,RB,RB,IND,7877,6a2b129d-a9e5-4131-b491-82269b323f77,9218.0,2282.0,232.0,frankgore/2506404,8479.0,7241.0,2848.0,411568.0,GoreFr00,frank-gore-1,4400.0,3205.0,NaN,7241.0,157341.0,5820.0,NaN,Frank Gore,frank gore,14,0,4,0.121212,3,48.0,38.0,40.0,2.0,1.0,0,0,-4.457448,1.223682,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11077,2024,REG,10,00-0039921,T.Benson,RB,RB,ARI,16594,0c1253f2-1822-445d-8590-fea6fd32703f,25388.0,NaN,11589.0,NaN,4429275.0,40958.0,NaN,3176915.0,BensTr01,trey-benson-2,17767.0,NaN,1565.0,40958.0,0.0,NaN,1248031.0,Trey Benson,trey benson,10,0,2,0.083333,2,62.0,25.0,33.0,3.0,2.0,0,0,1.512646,1.987017,0,0
11078,2024,REG,12,00-0039921,T.Benson,RB,RB,ARI,16594,0c1253f2-1822-445d-8590-fea6fd32703f,25388.0,NaN,11589.0,NaN,4429275.0,40958.0,NaN,3176915.0,BensTr01,trey-benson-2,17767.0,NaN,1565.0,40958.0,0.0,NaN,1248031.0,Trey Benson,trey benson,4,0,0,NaN,0,18.0,0.0,0.0,0.0,0.0,0,0,-0.326119,NaN,0,0
11079,2024,REG,13,00-0039921,T.Benson,RB,RB,ARI,16594,0c1253f2-1822-445d-8590-fea6fd32703f,25388.0,NaN,11589.0,NaN,4429275.0,40958.0,NaN,3176915.0,BensTr01,trey-benson-2,17767.0,NaN,1565.0,40958.0,0.0,NaN,1248031.0,Trey Benson,trey benson,3,0,0,NaN,0,20.0,0.0,0.0,1.0,0.0,0,0,0.515926,NaN,0,0
11080,2024,REG,14,00-0039921,T.Benson,RB,RB,ARI,16594,0c1253f2-1822-445d-8590-fea6fd32703f,25388.0,NaN,11589.0,NaN,4429275.0,40958.0,NaN,3176915.0,BensTr01,trey-benson-2,17767.0,NaN,1565.0,40958.0,0.0,NaN,1248031.0,Trey Benson,trey benson,2,0,1,0.026316,1,15.0,4.0,9.0,1.0,0.0,0,0,0.053110,-0.144496,0,0


In [17]:
# Output: imports the NFL next-generation stats from the nfl python library

# import the next generation stats (NGS) from nfl.import_ngs_data()
# note: ngs starts at week 0 (previous season totals) - not needed so drop those rows

# Pull NGS rushing data for the specified years
ngs_rb_df = nfl.import_ngs_data('rushing', years)

# Exclude rows where 'week' == 0 and filter for 'RB' position in one step
ngs_rb_df = ngs_rb_df[(ngs_rb_df['week'] != 0) & (ngs_rb_df['player_position'] == 'RB')]

# Drop unnecessary columns (already in the nfl python baseline dataframe)
ngs_rb_df = ngs_rb_df.drop(columns=['season_type', 'player_position', 'receptions', 'targets','player_jersey_number'], errors='ignore')

# Display the resulting dataframe
print(f"Shape of NGS RB DataFrame after dropping columns: {ngs_rb_df.shape}")
display(ngs_rb_df)

# ***csv file***
# ngs_rb_df.to_csv('ngs_rb_df.csv', index=False)

Shape of NGS RB DataFrame after dropping columns: (4382, 19)


,season,week,player_display_name,team_abbr,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,avg_rush_yards,rush_touchdowns,player_gsis_id,player_first_name,player_last_name,player_short_name,expected_rush_yards,rush_yards_over_expected,rush_yards_over_expected_per_att,rush_pct_over_expected
629,2017,1,Devonta Freeman,ATL,6.343514,8.333333,3.019091,12,37,3.083333,1,00-0031285,Devonta,Freeman,D.Freeman,NaN,NaN,NaN,NaN
630,2017,1,Todd Gurley,LAR,8.196500,10.526316,3.127438,19,40,2.105263,1,00-0032241,Todd,Gurley,T.Gurley,NaN,NaN,NaN,NaN
631,2017,1,Jamaal Charles,DEN,5.660000,20.000000,3.204667,10,40,4.000000,0,00-0026213,Jamaal,Charles,J.Charles,NaN,NaN,NaN,NaN
632,2017,1,Lamar Miller,HOU,3.960000,11.764706,2.483200,17,65,3.823529,0,00-0029615,Lamar,Miller,L.Miller,NaN,NaN,NaN,NaN
633,2017,1,Le'Veon Bell,PIT,4.232187,0.000000,3.067900,10,32,3.200000,0,00-0030496,Le'Veon,Bell,L.Bell,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5406,2024,21,Brian Robinson,WAS,4.083333,0.000000,2.563818,11,36,3.272727,0,00-0037746,Brian,Robinson,B.Robinson,53.323323,-17.323323,-1.574848,0.272727
5407,2024,21,Kareem Hunt,KC,3.638594,11.764706,2.834563,17,64,3.764706,1,00-0033923,Kareem,Hunt,K.Hunt,72.910294,-8.910294,-0.524135,0.411765
5408,2024,21,James Cook,BUF,2.805882,46.153846,2.643615,13,85,6.538462,2,00-0037248,James,Cook,J.Cook,44.600986,36.399014,3.033251,0.583333
5409,2024,21,Saquon Barkley,PHI,2.827627,33.333333,3.067846,15,118,7.866667,3,00-0034844,Saquon,Barkley,S.Barkley,45.716925,72.283075,4.818872,0.600000


In [18]:
# Output: a dataframe of NFL RB info, ids, weekly stats, and next-gen stats since 2017

# Joins rb_ids_weekly_stats dataframe with ngs_rb_df using the keys 'player_id' and 'player_gsis_id'
# Merge rb_ids_weekly_stats with ngs_rb_df using a left join
rb_ids_ngs_weekly_stats = pd.merge(
    rb_ids_weekly_stats,
    ngs_rb_df,
    left_on=['player_id', 'season', 'week'],  # Keys from rb_ids_weekly_stats
    right_on=['player_gsis_id', 'season', 'week'],  # Keys from ngs_rb_df
    how='left'  # Retain all rows from rb_ids_weekly_stats
)

# Display the shape of the resulting dataframe
print(f"Shape of merged dataframe: {rb_ids_ngs_weekly_stats.shape}")
print(f"Row count matches: {rb_ids_weekly_stats.shape[0] == rb_ids_ngs_weekly_stats.shape[0]}")

# Display a sample of the merged dataframe
display(rb_ids_ngs_weekly_stats)

# ***csv file***
# rb_ids_ngs_weekly_stats.to_csv('rb_ids_ngs_weekly_stats.csv', index=False)

Shape of merged dataframe: (11082, 62)
Row count matches: True


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,carries,attempts,targets,target_share,receptions,rushing_yards,receiving_yards,receiving_yards_after_catch,rushing_first_downs,receiving_first_downs,rushing_tds,receiving_tds,rushing_epa,receiving_epa,rushing_2pt_conversions,receiving_2pt_conversions,player_display_name,team_abbr,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,avg_rush_yards,rush_touchdowns,player_gsis_id,player_first_name,player_last_name,player_short_name,expected_rush_yards,rush_yards_over_expected,rush_yards_over_expected_per_att,rush_pct_over_expected
0,2017,REG,1,00-0023500,F.Gore,RB,RB,IND,7877,6a2b129d-a9e5-4131-b491-82269b323f77,9218.0,2282.0,232.0,frankgore/2506404,8479.0,7241.0,2848.0,411568.0,GoreFr00,frank-gore-1,4400.0,3205.0,NaN,7241.0,157341.0,5820.0,NaN,Frank Gore,frank gore,10,0,1,0.047619,1,42.0,10.0,14.0,1.0,0.0,0,0,-0.449218,0.374096,0,0,Frank Gore,IND,3.610952,10.000000,2.435667,10.0,42.0,4.200000,0.0,00-0023500,Frank,Gore,F.Gore,NaN,NaN,NaN,NaN
1,2017,REG,2,00-0023500,F.Gore,RB,RB,IND,7877,6a2b129d-a9e5-4131-b491-82269b323f77,9218.0,2282.0,232.0,frankgore/2506404,8479.0,7241.0,2848.0,411568.0,GoreFr00,frank-gore-1,4400.0,3205.0,NaN,7241.0,157341.0,5820.0,NaN,Frank Gore,frank gore,14,0,2,0.055556,0,46.0,0.0,0.0,2.0,0.0,1,0,-0.984420,-1.496430,0,0,Frank Gore,IND,4.080435,50.000000,2.736917,14.0,46.0,3.285714,1.0,00-0023500,Frank,Gore,F.Gore,NaN,NaN,NaN,NaN
2,2017,REG,3,00-0023500,F.Gore,RB,RB,IND,7877,6a2b129d-a9e5-4131-b491-82269b323f77,9218.0,2282.0,232.0,frankgore/2506404,8479.0,7241.0,2848.0,411568.0,GoreFr00,frank-gore-1,4400.0,3205.0,NaN,7241.0,157341.0,5820.0,NaN,Frank Gore,frank gore,25,0,1,0.041667,1,57.0,0.0,5.0,2.0,0.0,1,0,-5.832108,-0.788598,0,0,Frank Gore,IND,5.425789,24.000000,2.572100,25.0,57.0,2.280000,1.0,00-0023500,Frank,Gore,F.Gore,NaN,NaN,NaN,NaN
3,2017,REG,4,00-0023500,F.Gore,RB,RB,IND,7877,6a2b129d-a9e5-4131-b491-82269b323f77,9218.0,2282.0,232.0,frankgore/2506404,8479.0,7241.0,2848.0,411568.0,GoreFr00,frank-gore-1,4400.0,3205.0,NaN,7241.0,157341.0,5820.0,NaN,Frank Gore,frank gore,12,0,3,0.107143,3,46.0,34.0,43.0,1.0,2.0,0,0,-1.547445,1.718400,0,0,Frank Gore,IND,3.221957,41.666667,2.545750,12.0,46.0,3.833333,0.0,00-0023500,Frank,Gore,F.Gore,NaN,NaN,NaN,NaN
4,2017,REG,5,00-0023500,F.Gore,RB,RB,IND,7877,6a2b129d-a9e5-4131-b491-82269b323f77,9218.0,2282.0,232.0,frankgore/2506404,8479.0,7241.0,2848.0,411568.0,GoreFr00,frank-gore-1,4400.0,3205.0,NaN,7241.0,157341.0,5820.0,NaN,Frank Gore,frank gore,14,0,4,0.121212,3,48.0,38.0,40.0,2.0,1.0,0,0,-4.457448,1.223682,0,0,Frank Gore,IND,4.459583,28.571429,2.452167,14.0,48.0,3.428571,0.0,00-0023500,Frank,Gore,F.Gore,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11077,2024,REG,10,00-0039921,T.Benson,RB,RB,ARI,16594,0c1253f2-1822-445d-8590-fea6fd32703f,25388.0,NaN,11589.0,NaN,4429275.0,40958.0,NaN,3176915.0,BensTr01,trey-benson-2,17767.0,NaN,1565.0,40958.0,0.0,NaN,1248031.0,Trey Benson,trey benson,10,0,2,0.083333,2,62.0,25.0,33.0,3.0,2.0,0,0,1.512646,1.987017,0,0,Trey Benson,ARI,3.232581,10.000000,2.690778,10.0,62.0,6.200000,0.0,00-0039921,Trey,Benson,T.Benson,43.82976,18.17024,1.817024,0.6
11078,2024,REG,12,00-0039921,T.Benson,RB,RB,ARI,16594,0c1253f2-1822-445d-8590-fea6fd32703f,25388.0,NaN,11589.0,NaN,4429275.0,40958.0,NaN,3176915.0,BensTr01,trey-benson-2,17767.0,NaN,1565.0,40958.0,0.0,NaN,1248031.0,Trey Benson,trey benson,4,0,0,NaN,0,18.0,0.0,0.0,0.0,0.0,0,0,-0.326119,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [19]:
# Output: pro-football reference dataframe for rushing data from the python nfl library
# note: PFR data not available before 2018

# Define the range of years for PFR data (2018 to the current year)
pfr_years = list(range(2018, current_year))

# import pro-football reference data
pfr_rush_df = nfl.import_weekly_pfr('rush',pfr_years)

# Drop unnecessary columns
pfr_rush_df = pfr_rush_df.drop(
    columns=['game_id','pfr_game_id','receiving_int','rushing_broken_tackles', 
             'passing_drops', 'passing_drop_pct'], errors='ignore')

# display dataframe
print(f"Shape of PFR dataframe: {pfr_rush_df.shape}")
display(pfr_rush_df)

# ***csv file***
# pfr_rush_df.to_csv('pfr_rush_df.csv', index=False)

Shape of PFR dataframe: (16106, 13)


,season,week,game_type,team,opponent,pfr_player_name,pfr_player_id,carries,rushing_yards_before_contact,rushing_yards_before_contact_avg,rushing_yards_after_contact,rushing_yards_after_contact_avg,receiving_broken_tackles
0,2018,1,REG,PHI,ATL,Jay Ajayi,AjayJa00,15.0,19.0,1.3,43.0,2.9,NaN
1,2018,1,REG,PHI,ATL,Corey Clement,ClemCo00,5.0,18.0,3.6,8.0,1.6,NaN
2,2018,1,REG,PHI,ATL,Darren Sproles,SproDa00,5.0,2.0,0.4,8.0,1.6,NaN
3,2018,1,REG,PHI,ATL,Nelson Agholor,AghoNe00,1.0,1.0,1.0,15.0,15.0,NaN
4,2018,1,REG,PHI,ATL,Nick Foles,FoleNi00,1.0,-1.0,-1.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2354,2024,22,SB,PHI,KC,Kenny Pickett,PickKe00,3.0,-4.0,-1.3,0.0,0.0,NaN
2355,2024,22,SB,KC,PHI,Patrick Mahomes,MahoPa00,4.0,9.0,2.3,16.0,4.0,NaN
2356,2024,22,SB,KC,PHI,Kareem Hunt,HuntKa00,3.0,4.0,1.3,5.0,1.7,NaN
2357,2024,22,SB,KC,PHI,Isiah Pacheco,PachIs00,3.0,3.0,1.0,4.0,1.3,NaN


In [20]:
# Output: a dataframe of NFL RB info, ids, weekly stats, next-gen stats, and pro-football reference data
# NOTE: unbalanced dataframe - pfr stats start at 2018

# merge the pfr_rush_df with the rb_ids_ngs_weekly_stats dataframe
# match with ids then filter out the unmatched rows as they are likely (TE)
# Step 1: Merge the dataframes with a LEFT JOIN
rb_ids_ngs_pfr_stats = pd.merge(
    rb_ids_ngs_weekly_stats,
    pfr_rush_df,  # Use the full PFR dataframe as position data is unavailable
    left_on=['pfr_id', 'season', 'week'],  # Keys from rb_ids_ngs_weekly_stats
    right_on=['pfr_player_id', 'season', 'week'],  # Keys from pfr_rush_df
    how='left'  # Retain all rows from rb_ids_ngs_weekly_stats
)

# Display the shape of the resulting dataframe
print(f"Shape of merged dataframe: {rb_ids_ngs_pfr_stats.shape}")

# Row integrity check
print(
    f"Row count matches: {rb_ids_weekly_stats.shape[0] == rb_ids_ngs_weekly_stats.shape[0] == rb_ids_ngs_pfr_stats.shape[0]}"
)

# Display the first few rows of the merged dataframe for review
display(rb_ids_ngs_pfr_stats)

# ***csv file***
# rb_ids_ngs_pfr_stats.to_csv('rb_ids_ngs_pfr_stats.csv', index=False)

Shape of merged dataframe: (11082, 73)
Row count matches: True


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,carries_x,attempts,targets,target_share,receptions,rushing_yards,receiving_yards,receiving_yards_after_catch,rushing_first_downs,receiving_first_downs,rushing_tds,receiving_tds,rushing_epa,receiving_epa,rushing_2pt_conversions,receiving_2pt_conversions,player_display_name,team_abbr,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,avg_rush_yards,rush_touchdowns,player_gsis_id,player_first_name,player_last_name,player_short_name,expected_rush_yards,rush_yards_over_expected,rush_yards_over_expected_per_att,rush_pct_over_expected,game_type,team,opponent,pfr_player_name,pfr_player_id,carries_y,rushing_yards_before_contact,rushing_yards_before_contact_avg,rushing_yards_after_contact,rushing_yards_after_contact_avg,receiving_broken_tackles
0,2017,REG,1,00-0023500,F.Gore,RB,RB,IND,7877,6a2b129d-a9e5-4131-b491-82269b323f77,9218.0,2282.0,232.0,frankgore/2506404,8479.0,7241.0,2848.0,411568.0,GoreFr00,frank-gore-1,4400.0,3205.0,NaN,7241.0,157341.0,5820.0,NaN,Frank Gore,frank gore,10,0,1,0.047619,1,42.0,10.0,14.0,1.0,0.0,0,0,-0.449218,0.374096,0,0,Frank Gore,IND,3.610952,10.000000,2.435667,10.0,42.0,4.200000,0.0,00-0023500,Frank,Gore,F.Gore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,REG,2,00-0023500,F.Gore,RB,RB,IND,7877,6a2b129d-a9e5-4131-b491-82269b323f77,9218.0,2282.0,232.0,frankgore/2506404,8479.0,7241.0,2848.0,411568.0,GoreFr00,frank-gore-1,4400.0,3205.0,NaN,7241.0,157341.0,5820.0,NaN,Frank Gore,frank gore,14,0,2,0.055556,0,46.0,0.0,0.0,2.0,0.0,1,0,-0.984420,-1.496430,0,0,Frank Gore,IND,4.080435,50.000000,2.736917,14.0,46.0,3.285714,1.0,00-0023500,Frank,Gore,F.Gore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,REG,3,00-0023500,F.Gore,RB,RB,IND,7877,6a2b129d-a9e5-4131-b491-82269b323f77,9218.0,2282.0,232.0,frankgore/2506404,8479.0,7241.0,2848.0,411568.0,GoreFr00,frank-gore-1,4400.0,3205.0,NaN,7241.0,157341.0,5820.0,NaN,Frank Gore,frank gore,25,0,1,0.041667,1,57.0,0.0,5.0,2.0,0.0,1,0,-5.832108,-0.788598,0,0,Frank Gore,IND,5.425789,24.000000,2.572100,25.0,57.0,2.280000,1.0,00-0023500,Frank,Gore,F.Gore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,REG,4,00-0023500,F.Gore,RB,RB,IND,7877,6a2b129d-a9e5-4131-b491-82269b323f77,9218.0,2282.0,232.0,frankgore/2506404,8479.0,7241.0,2848.0,411568.0,GoreFr00,frank-gore-1,4400.0,3205.0,NaN,7241.0,157341.0,5820.0,NaN,Frank Gore,frank gore,12,0,3,0.107143,3,46.0,34.0,43.0,1.0,2.0,0,0,-1.547445,1.718400,0,0,Frank Gore,IND,3.221957,41.666667,2.545750,12.0,46.0,3.833333,0.0,00-0023500,Frank,Gore,F.Gore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,REG,5,00-0023500,F.Gore,RB,RB,IND,7877,6a2b129d-a9e5-4131-b491-82269b323f77,9218.0,2282.0,232.0,frankgore/2506404,8479.0,7241.0,2848.0,411568.0,GoreFr00,frank-gore-1,4400.0,3205.0,NaN,7241.0,157341.0,5820.0,NaN,Frank Gore,frank gore,14,0,4,0.121212,3,48.0,38.0,40.0,2.0,1.0,0,0,-4.457448,1.223682,0,0,Frank Gore,IND,4.459583,28.571429,2.452167,14.0,48.0,3.428571,0.0,00-0023500,Frank,Gore,F.Gore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11077,2024,REG,10,00-0039921,T.Benson,RB,RB,ARI,16594,0c1253f2-1822-445d-8590-fea6fd32703f,25388.0,NaN,11589.0,NaN,4429275.0,40958.0,NaN,3176915.0,BensTr01,trey-benson-2,17767.0,NaN,1565.0,40958.0,0.0,NaN,1248031.0,Trey Benson,trey benson,10,0,2,0.083333,2,62.0,25.0,33.0,3.0,2.0,0,0,1.512646,1.987017,0,0,Trey

In [21]:
# *** Baseline Dataframe ***

# Output: an ordered dataframe of NFL RB info, ids, weekly stats, next-gen stats, and pro-footeball reference data
# Output: Ordered the df by year, week, and rushing yards
# NOTE: unbalanced dataframe - pfr stats start at 2018

# Order the dataframe by season (year), week, and rushing_yards
rb_ids_ngs_pfr_stats_sorted = rb_ids_ngs_pfr_stats.sort_values(
    by=['season', 'week', 'rushing_yards'], 
    ascending=[True, True, False]  # Ascending for season and week, descending for rushing_yards
)

# Display the shape of the resulting dataframe
print(f"Shape of merged dataframe: {rb_ids_ngs_pfr_stats_sorted.shape}")

# Row integrity check
print(
    f"Row count matches: {rb_ids_weekly_stats.shape[0] == rb_ids_ngs_weekly_stats.shape[0] == rb_ids_ngs_pfr_stats.shape[0] == rb_ids_ngs_pfr_stats_sorted.shape[0]}"
)

# Display the sorted dataframe
print("Dataframe sorted by season, week, and rushing_yards:")
display(rb_ids_ngs_pfr_stats_sorted)


# *** csv file ***
# rb_ids_ngs_pfr_stats_sorted.to_csv('rb_ids_ngs_pfr_stats_sorted.csv', index=False)

Shape of merged dataframe: (11082, 73)
Row count matches: True
Dataframe sorted by season, week, and rushing_yards:


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,carries_x,attempts,targets,target_share,receptions,rushing_yards,receiving_yards,receiving_yards_after_catch,rushing_first_downs,receiving_first_downs,rushing_tds,receiving_tds,rushing_epa,receiving_epa,rushing_2pt_conversions,receiving_2pt_conversions,player_display_name,team_abbr,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,avg_rush_yards,rush_touchdowns,player_gsis_id,player_first_name,player_last_name,player_short_name,expected_rush_yards,rush_yards_over_expected,rush_yards_over_expected_per_att,rush_pct_over_expected,game_type,team,opponent,pfr_player_name,pfr_player_id,carries_y,rushing_yards_before_contact,rushing_yards_before_contact_avg,rushing_yards_after_contact,rushing_yards_after_contact_avg,receiving_broken_tackles
1295,2017,REG,1,00-0033923,K.Hunt,RB,RB,KC,13138,0ef0d0ca-2d2d-455b-ab63-a20c01303e37,16425.0,11841.0,4098.0,NaN,3059915.0,30199.0,NaN,2079567.0,HuntKa00,kareem-hunt-1,11739.0,12295.0,214.0,30199.0,746613.0,18944.0,746613.0,Kareem Hunt,kareem hunt,17,0,5,0.147059,5,148.0,98.0,76.0,6.0,3.0,1,2,3.455881,9.309378,0,0,Kareem Hunt,KC,2.821014,35.294118,2.920235,17.0,148.0,8.705882,1.0,00-0033923,Kareem,Hunt,K.Hunt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1259,2017,REG,1,00-0033893,D.Cook,RB,RB,MIN,13128,8960d61e-433b-41ea-a7ad-4e76be87b582,16374.0,11796.0,4029.0,NaN,3116593.0,30154.0,NaN,2130893.0,CookDa01,dalvin-cook-1,11700.0,12138.0,NaN,30154.0,824080.0,18872.0,824080.0,Dalvin Cook,dalvin cook,22,0,5,0.156250,3,127.0,10.0,12.0,4.0,0.0,0,0,-0.011037,-3.058858,0,0,Dalvin Cook,MIN,3.939449,40.909091,2.865950,22.0,127.0,5.772727,0.0,00-0033893,Dalvin,Cook,D.Cook,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,2017,REG,1,00-0027029,L.McCoy,RB,RB,BUF,9448,166292fc-629e-4c7b-b7bf-f572ca9eeb43,9398.0,4976.0,676.0,leseanmccoy/79607,12514.0,9317.0,NaN,1243187.0,McCoLe01,lesean-mccoy-1,5970.0,5168.0,NaN,9317.0,397945.0,11932.0,NaN,LeSean McCoy,lesean mccoy,22,0,6,0.222222,5,110.0,49.0,39.0,4.0,2.0,0,0,-0.711850,1.988743,0,0,LeSean McCoy,BUF,3.686273,4.545455,2.676778,22.0,110.0,5.000000,0.0,00-0027029,LeSean,McCoy,L.McCoy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1063,2017,REG,1,00-0033045,E.Elliott,RB,RB,DAL,12625,bef8b2b4-78bd-4a4d-bb5d-6b55ada9ef6a,15498.0,10638.0,3164.0,NaN,3051392.0,29238.0,NaN,2060769.0,ElliEz00,ezekiel-elliott-1,10736.0,11265.0,NaN,29238.0,728338.0,17923.0,728338.0,Ezekiel Elliott,ezekiel elliott,24,0,5,0.131579,5,104.0,36.0,42.0,6.0,1.0,0,0,0.570525,-0.114128,0,0,Ezekiel Elliott,DAL,4.143077,20.833333,2.896261,24.0,104.0,4.333333,0.0,00-0033045,Ezekiel,Elliott,E.Elliott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1243,2017,REG,1,00-0033856,L.Fournette,RB,RB,JAX,13129,7f46a7be-286e-4bfe-8778-d03dbe600ce9,16378.0,11759.0,3969.0,NaN,3115364.0,30117.0,NaN,2131693.0,FourLe00,leonard-fournette-1,11687.0,12132.0,NaN,30117.0,822013.0,18803.0,822013.0,Leonard Fournette,leonard fournette,26,0,3,0.142857,3,100.0,24.0,25.0,5.0,1.0,1,0,3.412744,1.308894,0,0,Leonard Fournette,JAX,3.781000,57.692308,2.325435,26.0,100.0,3.846154,1.0,00-0033856,Leonard,Fournette,L.Fournette,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,2024,POST,22,00-0034844,S.Barkley,RB,RB,PHI,13604,9811b753-347c-467a-b3cb-85937e71e2b9,17240.0,45791.0,4866.0,NaN,3929630.0,30972.0,NaN,2185957.0,BarkSa00,saquon-barkley-1,12507.0,

In [22]:
### End: Python NFL Library Dataframe ###

In [23]:
### Begin: Python NFL Library Data Integrity Checks ###

In [24]:
## Data Integrity Checks ##
# load the datframe from the variables in memory or csv files in the current working directory
def load_dataframe(var_name, file_name):
    try:
        return globals()[var_name]  # Try to get the variable from the global namespace
    except KeyError:
        print(f"⚠️ {var_name} not found in memory. Reading from {file_name}")
        return pd.read_csv(file_name)

In [25]:
## Data Integrity Checks ##
# Output: A Stratified Random Sample of years and players from the python nfl library
# test to ensure merged df values match the ids and weekly stats from the unmerged df
def test_rb_weekly_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1):
    """
    Perform integrity checks on RB Weekly Stats data by comparing merged and unmerged dataframes.
    This function checks consistency across selected numerical columns.

    Parameters:
    years (range): Range of years to test.
    num_samples (int): Number of random samples per year.
    tolerance (float): Allowed numerical difference for matching values.

    Returns:
    None
    """
    print("\n🔹 Running Weekly Stats Integrity Check...")

    # Load DataFrames
    merged_df = load_dataframe("rb_ids_ngs_pfr_stats_sorted", "rb_ids_ngs_pfr_stats_sorted.csv")
    weekly_df = load_dataframe("rb_ids_weekly_stats", "rb_ids_weekly_stats.csv")

    print(f"✅ Using {'in-memory variable' if 'rb_ids_ngs_pfr_stats_sorted' in globals() else 'CSV file'}: rb_ids_ngs_pfr_stats_sorted")
    print(f"✅ Using {'in-memory variable' if 'rb_ids_weekly_stats' in globals() else 'CSV file'}: rb_ids_weekly_stats")

    mismatches = []

    for year in years:
        print(f"\nTesting Year: {year}")

        # Filter data for the specified year
        merged_year_df = merged_df[merged_df["season"] == year]
        weekly_year_df = weekly_df[weekly_df["season"] == year]

        # Ensure numeric columns
        numeric_cols = ["receptions", "targets", "rushing_yards", "receiving_yards_after_catch", "rushing_first_downs"]
        merged_year_df.loc[:, numeric_cols] = merged_year_df[numeric_cols].apply(pd.to_numeric, errors="coerce")
        weekly_year_df.loc[:, numeric_cols] = weekly_year_df[numeric_cols].apply(pd.to_numeric, errors="coerce")

        # Select random sample of players for this year
        sampled_players = sample(list(merged_year_df["player_id"].dropna()), min(num_samples, len(merged_year_df)))

        for player_id in sampled_players:
            # Filter player data from each DataFrame
            merged_player = merged_year_df[merged_year_df["player_id"] == player_id]
            weekly_player = weekly_year_df[weekly_year_df["player_id"] == player_id]

            # Loop through each week and compare values
            for week in merged_player["week"].unique():
                merged_row = merged_player[merged_player["week"] == week]
                weekly_row = weekly_player[weekly_player["week"] == week]

                if merged_row.empty or weekly_row.empty:
                    continue  # Skip if there's no data for this player/week

                # Compare numerical columns
                for col in numeric_cols:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    weekly_val = weekly_row[col].values[0] if col in weekly_row else None

                    if merged_val is not None and weekly_val is not None:
                        if abs(float(merged_val) - float(weekly_val)) > tolerance:
                            mismatches.append((year, player_id, week, col, merged_val, weekly_val))

        print(f"Tested {len(sampled_players)} players in Year {year}.")

    # Summary of mismatches
    print(f"\nTotal mismatches found in Weekly Stats: {len(mismatches)}")
    if mismatches:
        print("\nMismatch Details (Year, Player ID, Week, Column, Merged Value, Weekly Value):")
        for m in mismatches[:10]:  # Print only first 10 mismatches
            print(m)
    else:
        print("✅ Weekly Stats Integrity Check Passed!")

# Run the test
test_rb_weekly_stats()


🔹 Running Weekly Stats Integrity Check...
✅ Using in-memory variable: rb_ids_ngs_pfr_stats_sorted
✅ Using in-memory variable: rb_ids_weekly_stats

Testing Year: 2017
Tested 25 players in Year 2017.

Testing Year: 2018
Tested 25 players in Year 2018.

Testing Year: 2019
Tested 25 players in Year 2019.

Testing Year: 2020
Tested 25 players in Year 2020.

Testing Year: 2021
Tested 25 players in Year 2021.

Testing Year: 2022
Tested 25 players in Year 2022.

Testing Year: 2023
Tested 25 players in Year 2023.

Total mismatches found in Weekly Stats: 0
✅ Weekly Stats Integrity Check Passed!


In [26]:
## Data Integrity Checks ##
# Output: A Stratified Random Sample of years and players from the python nfl library
# test to ensure merged df values match the next-gen stats within the unmerged df
def test_rb_ngs_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1):
    """
    Perform integrity checks on RB Next-Gen Stats (NGS) data.
    Compares key stats across merged and unmerged dataframes.
    """
    print("\n🔹 Running Next-Gen Stats (NGS) Integrity Check...")

    # Load dataframes
    merged_df = load_dataframe("rb_ids_ngs_pfr_stats_sorted", "rb_ids_ngs_pfr_stats_sorted.csv")
    ngs_df = load_dataframe("ngs_rb_df", "ngs_rb_df.csv")

    # Validation message
    print("✅ Using in-memory variable: rb_ids_ngs_pfr_stats_sorted" if "rb_ids_ngs_pfr_stats_sorted" in globals() else "✅ Loaded rb_ids_ngs_pfr_stats_sorted from CSV")
    print("✅ Using in-memory variable: ngs_rb_df" if "ngs_rb_df" in globals() else "✅ Loaded ngs_rb_df from CSV")

    mismatches = []

    for year in years:
        print(f"\nTesting Season: {year}")

        # Filter data for the given season
        merged_year_df = merged_df[merged_df["season"] == year]
        ngs_year_df = ngs_df[ngs_df["season"] == year]

        # Select random sample of players for this season
        sampled_players = sample(list(merged_year_df["player_id"].dropna()), min(num_samples, len(merged_year_df)))

        for player_id in sampled_players:
            # Filter player data from each DataFrame
            merged_player = merged_year_df[merged_year_df["player_id"] == player_id]
            ngs_player = ngs_year_df[ngs_year_df["player_gsis_id"] == player_id]

            for week in merged_player["week"].unique():
                merged_row = merged_player[merged_player["week"] == week]
                ngs_row = ngs_player[ngs_player["week"] == week]

                if merged_row.empty or ngs_row.empty:
                    continue  # Skip if there's no data for this player/week
                
                # Compare Next-Gen Stats columns
                for col in [
                    "efficiency",
                    "percent_attempts_gte_eight_defenders",
                    "avg_time_to_los",
                    "rush_attempts",
                    "rush_yards",
                    "avg_rush_yards",
                    "rush_touchdowns",
                    "expected_rush_yards",
                    "rush_yards_over_expected",
                    "rush_yards_over_expected_per_att",
                    "rush_pct_over_expected"
                ]:

                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    ngs_val = ngs_row[col].values[0] if col in ngs_row else None

                    if merged_val is not None and ngs_val is not None:
                        merged_val = float(merged_val)  # Ensure numeric conversion
                        ngs_val = float(ngs_val)  # Ensure numeric conversion

                        if abs(merged_val - ngs_val) > tolerance:
                            mismatches.append((year, player_id, week, col, merged_val, ngs_val))

        print(f"Tested {len(sampled_players)} players in Season {year}.")

    print(f"\nTotal mismatches found in Next-Gen Stats: {len(mismatches)}")

    if mismatches:
        print("\nMismatch Details (Season, Player ID, Week, Column, Merged Value, NGS Value):")
        for m in mismatches[:10]:  # Print only the first 10 mismatches
            print(m)
    else:
        print("✅ Next-Gen Stats Integrity Check Passed!")

# Run the test
test_rb_ngs_stats()


🔹 Running Next-Gen Stats (NGS) Integrity Check...
✅ Using in-memory variable: rb_ids_ngs_pfr_stats_sorted
✅ Using in-memory variable: ngs_rb_df

Testing Season: 2017
Tested 25 players in Season 2017.

Testing Season: 2018
Tested 25 players in Season 2018.

Testing Season: 2019
Tested 25 players in Season 2019.

Testing Season: 2020
Tested 25 players in Season 2020.

Testing Season: 2021
Tested 25 players in Season 2021.

Testing Season: 2022
Tested 25 players in Season 2022.

Testing Season: 2023
Tested 25 players in Season 2023.

Total mismatches found in Next-Gen Stats: 0
✅ Next-Gen Stats Integrity Check Passed!


In [27]:
## Data Integrity Checks ##
# Output: A Stratified Random Sample of years and players from the python nfl library
# test to ensure merged df values match the pro-football reference stats within the unmerged df
def test_rb_pfr_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1):
    """
    Perform integrity checks on RB data from Pro-Football Reference (PFR).
    Compares key stats across merged and unmerged dataframes.
    """
    print("\n🔹 Running Pro-Football Reference (PFR) Integrity Check...")

    # Load dataframes
    merged_df = load_dataframe("rb_ids_ngs_pfr_stats_sorted", "rb_ids_ngs_pfr_stats_sorted.csv")
    pfr_df = load_dataframe("pfr_rush_df", "pfr_rush_df.csv")

    # Validation message
    print("✅ Using in-memory variable: rb_ids_ngs_pfr_stats_sorted" if "rb_ids_ngs_pfr_stats_sorted" in globals() else "✅ Loaded rb_ids_ngs_pfr_stats_sorted from CSV")
    print("✅ Using in-memory variable: pfr_rush_df" if "pfr_rush_df" in globals() else "✅ Loaded pfr_rush_df from CSV")

    mismatches = []

    for year in years:
        print(f"\nTesting Season: {year}")

        # Filter data for the given season
        merged_year_df = merged_df[merged_df["season"] == year].copy()
        pfr_year_df = pfr_df[pfr_df["season"] == year].copy()

        # Ensure 'pfr_id' and 'pfr_player_id' are treated as strings
        merged_year_df["pfr_id"] = merged_year_df["pfr_id"].astype(str)
        pfr_year_df["pfr_player_id"] = pfr_year_df["pfr_player_id"].astype(str)

        # Select random sample of players for this season
        sampled_players = sample(list(merged_year_df["pfr_id"].dropna()), min(num_samples, len(merged_year_df)))

        for player_id in sampled_players:
            # Filter player data from each DataFrame
            merged_player = merged_year_df[merged_year_df["pfr_id"] == player_id]
            pfr_player = pfr_year_df[pfr_year_df["pfr_player_id"] == player_id]

            for week in merged_player["week"].unique():
                merged_row = merged_player[merged_player["week"] == week]
                pfr_row = pfr_player[pfr_player["week"] == week]

                if merged_row.empty or pfr_row.empty:
                    continue  # Skip if there's no data for this player/week

                # Compare PFR Stats columns
                for col in [
                    "carries",
                    "rushing_yards_before_contact",
                    "rushing_yards_before_contact_avg",
                    "rushing_yards_after_contact",
                    "rushing_yards_after_contact_avg"
                ]:
                    
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    pfr_val = pfr_row[col].values[0] if col in pfr_row else None

                    if merged_val is not None and pfr_val is not None:
                        merged_val = float(merged_val)  # Ensure numeric conversion
                        pfr_val = float(pfr_val)  # Ensure numeric conversion

                        if abs(merged_val - pfr_val) > tolerance:
                            mismatches.append((year, player_id, week, col, merged_val, pfr_val))

        print(f"Tested {len(sampled_players)} players in Season {year}.")

    print(f"\nTotal mismatches found in PFR Stats: {len(mismatches)}")

    if mismatches:
        print("\nMismatch Details (Season, Player ID, Week, Column, Merged Value, PFR Value):")
        for m in mismatches[:10]:  # Print only the first 10 mismatches
            print(m)
    else:
        print("✅ Pro-Football Reference (PFR) Integrity Check Passed!")

# Run the test
test_rb_pfr_stats()


🔹 Running Pro-Football Reference (PFR) Integrity Check...
✅ Using in-memory variable: rb_ids_ngs_pfr_stats_sorted
✅ Using in-memory variable: pfr_rush_df

Testing Season: 2017
Tested 25 players in Season 2017.

Testing Season: 2018
Tested 25 players in Season 2018.

Testing Season: 2019
Tested 25 players in Season 2019.

Testing Season: 2020
Tested 25 players in Season 2020.

Testing Season: 2021
Tested 25 players in Season 2021.

Testing Season: 2022
Tested 25 players in Season 2022.

Testing Season: 2023
Tested 25 players in Season 2023.

Total mismatches found in PFR Stats: 0
✅ Pro-Football Reference (PFR) Integrity Check Passed!


In [28]:
### End: Python NFL Library Data Integrity Checks ###

In [29]:
### Begin:fantasypros webscraping ###

In [30]:
# Generates a list of (year, week) combinations for web scraping.
# - 2017-2020: Weeks 1-17
# - 2021 and beyond: Weeks 1-18
def generate_year_week_combinations(start_year, end_year):
    year_week_combinations = []
    for year in range(start_year, end_year + 1):
        max_week = 17 if year <= 2020 else 18
        year_week_combinations.extend([(year, week) for week in range(1, max_week + 1)])
    return year_week_combinations

In [31]:
# output: a dataframe of weekly RB fantasypros advanced stats week-by-week for defined years
# Define the function to scrape weekly RB advanced stats from Fantasy Pros

def scrape_fantasypros_all_weeks_years(start_year, end_year):
    """
    Scrapes FantasyPros RB advanced stats for all weeks and years with correct ID extraction.
    """
    year_week_combinations = generate_year_week_combinations(start_year, end_year)
    all_data = []

    for year, week in year_week_combinations:
        try:
            # Construct the URL
            url = f"https://www.fantasypros.com/nfl/advanced-stats-rb.php?year={year}&week={week}&range=week&view=pergame"
            response = requests.get(url)
            response.raise_for_status()

            # Parse HTML content
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract table headers
            table_headers = [header.text.strip() for header in soup.find('thead').find_all('th')]

            # Find table rows
            table_rows = soup.find('tbody').find_all('tr')

            for row in table_rows:
                # Extract FantasyPros Player ID correctly
                fantasypros_id = "Unknown"
                player_name = "Unknown"
                
                player_link = row.find("a", class_="fp-player-link")
                if player_link:
                    class_list = player_link.get("class", [])
                    for class_name in class_list:
                        if class_name.startswith("fp-id-"):
                            fantasypros_id = class_name.replace("fp-id-", "")  # Extract numeric ID
                            break
                    player_name = player_link.text.strip()  # Extract player name explicitly from <a> tag

                # Extract the rest of the row data
                row_data = [cell.text.strip() for cell in row.find_all('td')]

                # Ensure data matches headers before adding
                if len(row_data) == len(table_headers):  
                    all_data.append([year, week, fantasypros_id, player_name] + row_data)

            # Random delay to avoid server overload
            time.sleep(uniform(0.3, 0.9))

        except Exception as e:
            print(f"⚠️ Error occurred while scraping Year: {year}, Week: {week}: {e}")

    # Convert data to DataFrame
    column_names = ['Year', 'Week', 'FantasyPros_ID', 'Player'] + table_headers  # No duplicate FantasyPros_ID
    rb_fp_advanced_stats_df = pd.DataFrame(all_data, columns=column_names)

    # Drop the redundant 'Rank' column if it exists
    rb_fp_advanced_stats_df.drop(columns=['Rank'], inplace=True, errors='ignore')

    return rb_fp_advanced_stats_df

# ✅ Run the full scrape
rb_fp_advanced_stats_df = scrape_fantasypros_all_weeks_years(2017, years[-1])

# ✅ Display dataset shape for verification
print(f"\n📊 **Shape of FantasyPros advanced DataFrame:** {rb_fp_advanced_stats_df.shape}")

# ✅ Display first few rows to confirm FantasyPros IDs are correctly extracted
display(rb_fp_advanced_stats_df.head(10))

# *** csv file ***
# rb_fp_advanced_stats_df.to_csv("fantasypros_rb_advanced_stats.csv", index=False)



📊 **Shape of FantasyPros advanced DataFrame:** (1120, 22)


,Year,Week,FantasyPros_ID,Player,Player,G,ATT,YDS,YBCON,YACON,BRKTKL,TK LOSS,TK LOSS YDS,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,REC,TGT,RZ TGT,YACON
0,2017,1,16425,Kareem Hunt,Kareem Hunt (KC),1,17,148,104,44,6,0,0,4,1,1,1,1,5,5,0,36
1,2017,1,11657,Mike Gillislee,Mike Gillislee (FA),1,15,45,29,16,0,1,-4,1,0,0,0,0,0,0,0,0
2,2017,1,16378,Leonard Fournette,Leonard Fournette (FA),1,26,100,52,48,2,3,-3,2,0,0,0,0,3,3,0,5
3,2017,1,16450,Tarik Cohen,Tarik Cohen (FA),1,5,66,6,60,2,1,-3,2,1,1,1,0,8,12,1,20
4,2017,1,9398,LeSean McCoy,LeSean McCoy (FA),1,22,110,86,24,1,4,-6,4,3,0,0,0,5,6,0,2
5,2017,1,13912,Todd Gurley II,Todd Gurley II (FA),1,19,40,7,33,2,3,-13,1,0,0,0,0,5,6,0,41
6,2017,1,13977,Ty Montgomery II,Ty Montgomery II (FA),1,19,54,15,39,0,1,-1,0,0,0,0,0,4,4,0,25
7,2017,1,9317,Jonathan Stewart,Jonathan Stewart (FA),1,18,65,18,47,1,0,0,1,0,0,0,0,2,2,1,11
8,2017,2,11896,C.J. Anderson,C.J. Anderson (FA),1,25,118,43,75,1,2,-4,2,2,0,0,0,3,3,1,10
9,2017,2,13912,Todd Gurley II,Todd Gurley II (FA),1,16,88,45,43,0,0,0,2,0,0,0,0,3,4,1,14


In [32]:
# output: a dataframe of RB fantasy points and % rostered stats week-by-week for defined years

# Scrape the weekly RB fantasy points column and the % rostered from the FantasyPros website
def rb_scrape_fantasypros_fpts_rost(start_year, end_year):
    """
    Scrapes FantasyPros RB FPTS and % Rostered data for all weeks and years, including FantasyPros IDs.
    """
    # Generate year-week combinations
    week_combinations = generate_year_week_combinations(start_year, end_year)

    # Initialize an empty list to store data
    all_data = []

    for year, week in week_combinations:
        try:
            # Build the URL
            url = f"https://www.fantasypros.com/nfl/stats/rb.php?year={year}&week={week}&range=week"
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, "html.parser")

            # Locate the table
            table = soup.find("table", {"id": "data"})
            if table:
                table_str = str(table)

                # Read the table while skipping multi-level headers
                df = pd.read_html(io.StringIO(table_str), header=1)[0]

                # Extract FantasyPros Player IDs correctly
                table_rows = table.find("tbody").find_all("tr")

                fantasypros_ids = []
                player_names = []

                for row in table_rows:
                    # Extract FantasyPros Player ID
                    fantasypros_id = "Unknown"
                    player_name = "Unknown"
                    
                    player_link = row.find("a", class_="fp-player-link")
                    if player_link:
                        class_list = player_link.get("class", [])
                        for class_name in class_list:
                            if class_name.startswith("fp-id-"):
                                fantasypros_id = class_name.replace("fp-id-", "")  # Extract numeric ID
                                break
                        player_name = player_link.text.strip()  # Extract player name explicitly from <a> tag

                    fantasypros_ids.append(fantasypros_id)
                    player_names.append(player_name)

                # Add extracted FantasyPros IDs and correct player names
                df.insert(1, "FantasyPros_ID", fantasypros_ids)
                df["Player"] = player_names  # Replace with properly scraped names

                # Add Year and Week columns
                df["Year"] = year
                df["Week"] = week

                # Append dataframe to list
                all_data.append(df)

            # Delay to avoid bombarding the server
            time.sleep(uniform(0.3, 0.9))  # Randomized delay

        except Exception as e:
            print(f"⚠️ Error occurred while scraping Year: {year}, Week: {week}: {e}")

    # Combine all data into a single dataframe
    rb_fpts_perct_rost_df = pd.concat(all_data, ignore_index=True)

    # Drop unnecessary columns
    columns_to_drop = [
        "Rank", "REC", "TGT", "YDS", "Y/R", "LG", "20+", "TD", 
        "ATT", "YDS.1", "TD.1", "FL", "G", "FPTS/G"
    ]
    rb_fpts_perct_rost_df.drop(columns=columns_to_drop, errors="ignore", inplace=True)

    # *** csv file *** 
    # rb_fpts_perct_rost_df.to_csv("fantasypros_rb_fpts_perct_rost.csv", index=False)

    # Display shape of the dataframe
    print(f"\n📊 **Shape of RB FPTS and % Rostered dataframe after column removal:** {rb_fpts_perct_rost_df.shape}")

    # Display first few rows for verification
    display(rb_fpts_perct_rost_df.head(10))

    return rb_fpts_perct_rost_df

# ✅ Run the scraping function for all years
rb_fpts_perct_rost_df = rb_scrape_fantasypros_fpts_rost(2017, years[-1])



📊 **Shape of RB FPTS and % Rostered dataframe after column removal:** (17337, 7)


,FantasyPros_ID,Player,Y/A,FPTS,ROST,Year,Week
0,16425,Kareem Hunt,8.7,40.6,37.3%,2017,1
1,11657,Mike Gillislee,3.0,22.5,0.0%,2017,1
2,16378,Leonard Fournette,3.8,18.4,0.0%,2017,1
3,16450,Tarik Cohen,13.2,17.3,0.0%,2017,1
4,9398,LeSean McCoy,5.0,15.9,2.0%,2017,1
5,13912,Todd Gurley II,2.1,15.6,0.6%,2017,1
6,13977,Ty Montgomery II,2.8,15.3,0.0%,2017,1
7,9317,Jonathan Stewart,3.6,14.2,0.0%,2017,1
8,15498,Ezekiel Elliott,4.3,14.0,3.7%,2017,1
9,12194,Terrance West,4.2,14.0,0.0%,2017,1


In [33]:
# output: a dataframe of weekly RB redzone stats week-by-week for defined years
# scrape the weekly RB redzone stats from the FantasyPros
def rb_scrape_fantasypros_redzone_stats(start_year, end_year):
    """
    Scrapes FantasyPros RB Red Zone Stats for all weeks and years, including FantasyPros IDs.
    """
    # Generate year-week combinations
    week_combinations = generate_year_week_combinations(start_year, end_year)

    # Initialize an empty list to store data
    all_data = []

    for year, week in week_combinations:
        try:
            # Build the URL
            url = f"https://www.fantasypros.com/nfl/red-zone-stats/rb.php?year={year}&range=week&week={week}"
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, "html.parser")

            # Locate the table
            table = soup.find("table", {"id": "data"})
            if table:
                table_str = str(table)

                # Read the table while skipping multi-level headers
                df = pd.read_html(io.StringIO(table_str))[0]

                # Flatten the multi-level column headers
                df.columns = df.columns.droplevel(0)

                # Drop the extra header rows (if any)
                df = df[df['Player'] != 'Player']  # Filter out duplicate header rows

                # Extract FantasyPros Player IDs correctly
                table_rows = table.find("tbody").find_all("tr")

                fantasypros_ids = []
                player_names = []

                for row in table_rows:
                    # Extract FantasyPros Player ID
                    fantasypros_id = "Unknown"
                    player_name = "Unknown"

                    player_link = row.find("a", class_="fp-player-link")
                    if player_link:
                        class_list = player_link.get("class", [])
                        for class_name in class_list:
                            if class_name.startswith("fp-id-"):
                                fantasypros_id = class_name.replace("fp-id-", "")  # Extract numeric ID
                                break
                        player_name = player_link.text.strip()  # Extract player name explicitly from <a> tag

                    fantasypros_ids.append(fantasypros_id)
                    player_names.append(player_name)

                # Add extracted FantasyPros IDs and correct player names
                df.insert(1, "FantasyPros_ID", fantasypros_ids)
                df["Player"] = player_names  # Replace with properly scraped names

                # Add Year and Week columns
                df["Year"] = year
                df["Week"] = week

                # Append dataframe to list
                all_data.append(df)

            # Delay to avoid bombarding the server
            time.sleep(uniform(0.3, 0.9))  # Randomized delay

        except Exception as e:
            print(f"⚠️ Error occurred while scraping Year: {year}, Week: {week}: {e}")

    # Combine all data into a single dataframe
    rb_redzone_stats_df = pd.concat(all_data, ignore_index=True)

    # Drop unnecessary columns
    columns_to_drop = ['Rank', 'ATT', 'YDS', 'TD', 'PCT', 'FL', 'G', 'FPTS', 'FPTS/G', 'ROST %']
    rb_redzone_stats_df.drop(columns=columns_to_drop, errors='ignore', inplace=True)

    # Rename columns to add 'rz' prefix, except for 'Player', 'FantasyPros_ID', 'Year', 'Week'
    rb_redzone_stats_df.rename(
        columns={
            col: f"{col}_rz" for col in rb_redzone_stats_df.columns
            if col not in ['Player', 'FantasyPros_ID', 'Year', 'Week']
        },
        inplace=True
    )

    # *** csv file ***
    # rb_redzone_stats_df.to_csv("fantasypros_rb_redzone_stats.csv", index=False)

    # Display shape of the dataframe
    print(f"\n📊 **Shape of RB Red Zone Stats dataframe after processing:** {rb_redzone_stats_df.shape}")

    # Display first few rows for verification
    display(rb_redzone_stats_df.head(10))

    return rb_redzone_stats_df

# ✅ Run the scraping function for all years
rb_redzone_stats_df = rb_scrape_fantasypros_redzone_stats(2017, years[-1])



📊 **Shape of RB Red Zone Stats dataframe after processing:** (7018, 10)


,FantasyPros_ID,Player,Y/A_rz,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz,Year,Week
0,11657,Mike Gillislee,2.5,0,0,0%,0.0,0.0%,2017,1
1,16425,Kareem Hunt,3.0,1,1,100.0%,3.0,100.0%,2017,1
2,16450,Tarik Cohen,5.0,1,1,100.0%,19.0,50.0%,2017,1
3,13903,Melvin Gordon III,2.0,1,1,100.0%,11.0,50.0%,2017,1
4,9317,Jonathan Stewart,1.5,1,1,100.0%,9.0,100.0%,2017,1
5,16378,Leonard Fournette,1.7,0,0,0%,0.0,0.0%,2017,1
6,12108,Devonta Freeman,5.5,0,0,0%,0.0,0%,2017,1
7,15585,Jordan Howard,5.0,0,1,0.0%,0.0,50.0%,2017,1
8,9717,LeGarrette Blount,4.0,1,1,100.0%,1.0,100.0%,2017,1
9,13953,Malcolm Brown,2.3,0,0,0%,0.0,0%,2017,1


In [34]:
# *** FantasyPros Dataframes from each section can be used as an alternative for merging ***

# output: merged fantasypros dataframe 
# note: NOT finalized yet - needs data normalization 

# ✅ Display the shape of each dataframe before merging
print(f"\n📊 **Shape of RB Advanced Stats DataFrame:** {rb_fp_advanced_stats_df.shape}")
print(f"📊 **Shape of RB FPTS and % Rostered DataFrame:** {rb_fpts_perct_rost_df.shape}")
print(f"📊 **Shape of RB Red Zone Stats DataFrame:** {rb_redzone_stats_df.shape}")

# ✅ Drop 'Player' from fpts and redzone dataframes before merge
rb_fpts_perct_rost_df = rb_fpts_perct_rost_df.drop(columns=["Player"], errors="ignore")
rb_redzone_stats_df = rb_redzone_stats_df.drop(columns=["Player"], errors="ignore")

# ✅ Merge RB Advanced Stats with Fantasy Points and % Rostered
rb_adv_fpts_rost_merged_df = pd.merge(
    rb_fp_advanced_stats_df,  # Baseline DF (keeps 'Player')
    rb_fpts_perct_rost_df,
    on=['FantasyPros_ID', 'Year', 'Week'],
    how='left'  # Retains all rows from the baseline DF
)

# ✅ Merge with Red Zone Stats
rb_adv_fpts_rost_rz_merged_df = pd.merge(
    rb_adv_fpts_rost_merged_df,
    rb_redzone_stats_df,
    on=['FantasyPros_ID', 'Year', 'Week'],
    how='left'
)

# ✅ Sort for easier verification
rb_adv_fpts_rost_rz_merged_df_sorted = rb_adv_fpts_rost_rz_merged_df.sort_values(
    by=['Year', 'Week', 'YDS'], 
    ascending=[True, True, False]
)

# ✅ Display the updated shape after dropping duplicate columns
print(f"\n📊 **fantasypros Shape After Merge:** {rb_adv_fpts_rost_rz_merged_df_sorted.shape}")

# *** csv file ***
# rb_adv_fpts_rost_rz_merged_df_sorted.to_csv('fantasypros_rb_adv_fpts_rost_rz_df.csv', index=False)
display(rb_adv_fpts_rost_rz_merged_df_sorted)


📊 **Shape of RB Advanced Stats DataFrame:** (1120, 22)
📊 **Shape of RB FPTS and % Rostered DataFrame:** (17337, 7)
📊 **Shape of RB Red Zone Stats DataFrame:** (7018, 10)

📊 **fantasypros Shape After Merge:** (1120, 31)


,Year,Week,FantasyPros_ID,Player,Player,G,ATT,YDS,YBCON,YACON,BRKTKL,TK LOSS,TK LOSS YDS,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,REC,TGT,RZ TGT,YACON,Y/A,FPTS,ROST,Y/A_rz,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz
3,2017,1,16450,Tarik Cohen,Tarik Cohen (FA),1,5,66,6,60,2,1,-3,2,1,1,1,0,8,12,1,20,13.2,17.3,0.0%,5.0,1.0,1.0,100.0%,19.0,50.0%
7,2017,1,9317,Jonathan Stewart,Jonathan Stewart (FA),1,18,65,18,47,1,0,0,1,0,0,0,0,2,2,1,11,3.6,14.2,0.0%,1.5,1.0,1.0,100.0%,9.0,100.0%
6,2017,1,13977,Ty Montgomery II,Ty Montgomery II (FA),1,19,54,15,39,0,1,-1,0,0,0,0,0,4,4,0,25,2.8,15.3,0.0%,6.0,0.0,0.0,0%,0.0,0%
1,2017,1,11657,Mike Gillislee,Mike Gillislee (FA),1,15,45,29,16,0,1,-4,1,0,0,0,0,0,0,0,0,3.0,22.5,0.0%,2.5,0.0,0.0,0%,0.0,0.0%
5,2017,1,13912,Todd Gurley II,Todd Gurley II (FA),1,19,40,7,33,2,3,-13,1,0,0,0,0,5,6,0,41,2.1,15.6,0.6%,3.0,0.0,0.0,0%,0.0,0%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,2024,18,22947,Dameon Pierce,Dameon Pierce (HOU),1,19,176,94,258,4,2,-2,8,2,2,2,2,0,2,1,0,9.3,23.6,14.2%,0.0,0.0,1.0,0.0%,0.0,100.0%
1113,2024,18,23133,Bijan Robinson,Bijan Robinson (ATL),1,28,170,84,86,4,1,-2,6,1,0,0,0,2,5,1,3,6.1,29.3,98.7%,3.3,1.0,1.0,100.0%,5.0,100.0%
1112,2024,18,22968,Jahmyr Gibbs,Jahmyr Gibbs (DET),1,23,139,152,126,2,4,-20,8,4,2,2,0,5,5,1,8,6.0,41.0,99.0%,5.4,1.0,1.0,100.0%,10.0,100.0%
1114,2024,18,15514,Derrick Henry,Derrick Henry (BAL),1,20,138,71,67,0,3,-8,4,3,1,1,0,2,3,1,8,6.9,28.1,98.7%,0.7,0.0,1.0,0.0%,0.0,50.0%


In [35]:
### End:fantasypros webscraping ###

In [36]:
### Begin: FantasyPros Data Integrity Checks Section ###

In [37]:
## Data Integrity Checks ##
# Extract unique years and weeks from each dataframe
years_merged = sorted(rb_adv_fpts_rost_rz_merged_df_sorted['Year'].unique())
weeks_merged = sorted(rb_adv_fpts_rost_rz_merged_df_sorted['Week'].unique())

years_redzone = sorted(rb_redzone_stats_df['Year'].unique())
weeks_redzone = sorted(rb_redzone_stats_df['Week'].unique())

years_fpts = sorted(rb_fpts_perct_rost_df['Year'].unique())
weeks_fpts = sorted(rb_fpts_perct_rost_df['Week'].unique())

years_adv_stats = sorted(rb_fp_advanced_stats_df['Year'].unique())
weeks_adv_stats = sorted(rb_fp_advanced_stats_df['Week'].unique())

# Print the results for comparison
print(f"Years in Merged DF: {years_merged}")
print(f"Years in RedZone DF: {years_redzone}")
print(f"Years in FPTS DF: {years_fpts}")
print(f"Years in Advanced Stats DF: {years_adv_stats}\n")

print(f"Weeks in Merged DF: {weeks_merged}")
print(f"Weeks in RedZone DF: {weeks_redzone}")
print(f"Weeks in FPTS DF: {weeks_fpts}")
print(f"Weeks in Advanced Stats DF: {weeks_adv_stats}")

Years in Merged DF: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Years in RedZone DF: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Years in FPTS DF: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Years in Advanced Stats DF: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

Weeks in Merged DF: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Weeks in RedZone DF: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Weeks in FPTS DF: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Weeks in Advanced Stats DF: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [38]:
## Data Integrity Checks ##
# Output: A Stratified Random Sample of years and players from the scraped FantasyPros data
# test to ensure merged df values match the individual unmerged df of advanced stats
# ✅ Integrity Check for RB Advanced Stats (using FantasyPros_ID)
def test_rb_advanced_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1, random_seed=42):
    """
    Integrity check for RB advanced stats.

    - Uses Stratified Random Sampling by year.
    - Tests key stats (REC, YDS, YBC, AIR, YAC, TGT, CATCHABLE).
    - Reports % of matches and mismatches found.
    - Logs whether data is loaded from variables or CSV files.
    """
    seed(random_seed)  # Set seed for reproducibility
    mismatches = []

    print("🔹 Running Advanced Stats Integrity Check...")
    print(f"📌 Test Details: Stratified Random Sample by Year")
    print(f"📌 Columns Tested: ['REC', 'YDS', 'YBC', 'AIR', 'YAC', 'TGT', 'CATCHABLE']\n")

    # Load data from memory or CSV
    merged_df = load_dataframe("rb_adv_fpts_rost_rz_merged_df_sorted", "fantasypros_rb_adv_fpts_rost_rz_df.csv")
    adv_stats_df = load_dataframe("rb_fp_advanced_stats_df", "fantasypros_rb_advanced_stats.csv")

    # Log the source of the data
    if isinstance(merged_df, pd.DataFrame) and isinstance(adv_stats_df, pd.DataFrame):
        print("📥 Data loaded from **variables** in memory.\n")
    else:
        print("📥 Data loaded from **CSV files**.\n")

    for year in years:
        print(f"\nTesting Year: {year}")

        # Filter data by year
        merged_year_df = merged_df[merged_df['Year'] == year]
        adv_year_df = adv_stats_df[adv_stats_df['Year'] == year]

        # Skip if no data
        if merged_year_df.empty or adv_year_df.empty:
            print(f"⚠️ Skipping Year {year} (No data available)")
            continue

        # Select random IDs instead of player names
        sampled_ids = sample(list(merged_year_df['FantasyPros_ID'].dropna()), min(num_samples, len(merged_year_df)))

        total_comparisons = 0
        match_count = 0

        for fp_id in sampled_ids:
            for week in merged_year_df[merged_year_df['FantasyPros_ID'] == fp_id]['Week'].unique():

                merged_row = merged_year_df[(merged_year_df['FantasyPros_ID'] == fp_id) & (merged_year_df['Week'] == week)]
                adv_row = adv_year_df[(adv_year_df['FantasyPros_ID'] == fp_id) & (adv_year_df['Week'] == week)]

                if merged_row.empty or adv_row.empty:
                    continue  # Skip if missing

                for col in ['REC', 'YDS', 'YBC', 'AIR', 'YAC', 'TGT', 'CATCHABLE']:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    adv_val = adv_row[col].values[0] if col in adv_row else None

                    if merged_val is not None and adv_val is not None:
                        total_comparisons += 1
                        if merged_val == adv_val:
                            match_count += 1
                        else:
                            mismatches.append((year, fp_id, week, col, merged_val, adv_val))

        # Yearly summary
        match_percentage = (match_count / total_comparisons * 100) if total_comparisons else 0
        print(f"✅ Tested {len(sampled_ids)} IDs in Year {year} ({total_comparisons} values compared, {match_percentage:.2f}% matched)")

    # summary
    print("\n🔍 Integrity Check Summary:")
    print(f"✔ Total Comparisons: {sum(len(m) for m in mismatches) + match_count}")
    print(f"✔ Overall Match Rate: {(match_count / (match_count + len(mismatches)) * 100) if (match_count + len(mismatches)) > 0 else 0:.2f}%")

    if mismatches:
        print("\n❌ Mismatch Details (Showing up to 10 cases):")
        for m in mismatches[:10]:
            print(m)
    else:
        print("✅ All values matched successfully!")

# ✅ Run the updated test
test_rb_advanced_stats()


🔹 Running Advanced Stats Integrity Check...
📌 Test Details: Stratified Random Sample by Year
📌 Columns Tested: ['REC', 'YDS', 'YBC', 'AIR', 'YAC', 'TGT', 'CATCHABLE']

📥 Data loaded from **variables** in memory.


Testing Year: 2017
✅ Tested 25 IDs in Year 2017 (285 values compared, 100.00% matched)

Testing Year: 2018
✅ Tested 25 IDs in Year 2018 (369 values compared, 100.00% matched)

Testing Year: 2019
✅ Tested 25 IDs in Year 2019 (339 values compared, 100.00% matched)

Testing Year: 2020
✅ Tested 25 IDs in Year 2020 (282 values compared, 100.00% matched)

Testing Year: 2021
✅ Tested 25 IDs in Year 2021 (279 values compared, 100.00% matched)

Testing Year: 2022
✅ Tested 25 IDs in Year 2022 (288 values compared, 100.00% matched)

Testing Year: 2023
✅ Tested 25 IDs in Year 2023 (324 values compared, 100.00% matched)

🔍 Integrity Check Summary:
✔ Total Comparisons: 324
✔ Overall Match Rate: 100.00%
✅ All values matched successfully!


In [39]:
## Data Integrity Checks ##
# Output: A Stratified Random Sample of years and players from the scraped FantasyPros data
# test to ensure merged df values match the individual unmerged df of %rostered and fantasypoints stats
# ✅ Integrity Check for RB Red Zone Stats (using FantasyPros_ID)

# ✅ Integrity Check for RB Fantasy Points & % Rostered (using FantasyPros_ID)
def test_rb_fpts_rost(years=range(2017, 2024), num_samples=25, tolerance=0.1, random_seed=42):
    """
    Integrity check for RB Fantasy Points and % Rostered.

    - Uses Stratified Random Sampling by year.
    - Tests key stats (FPTS, ROST).
    - Reports % of matches and mismatches found.
    - Logs whether data is loaded from variables or CSV files.
    """
    seed(random_seed)  # Set seed for reproducibility
    mismatches = []

    print("\n🔹 Running Fantasy Points & % Rostered Integrity Check...")
    print(f"📌 Test Details: Stratified Random Sample by Year")
    print(f"📌 Columns Tested: ['FPTS', 'ROST']\n")

    # Load data from memory or CSV
    merged_df = load_dataframe("rb_adv_fpts_rost_rz_merged_df_sorted", "fantasypros_rb_adv_fpts_rost_rz_df.csv")
    fpts_df = load_dataframe("rb_fpts_perct_rost_df", "fantasypros_rb_fpts_perct_rost.csv")

    # Log the source of the data
    if isinstance(merged_df, pd.DataFrame) and isinstance(fpts_df, pd.DataFrame):
        print("📥 Data loaded from **variables** in memory.\n")
    else:
        print("📥 Data loaded from **CSV files**.\n")

    for year in years:
        print(f"\nTesting Year: {year}")

        # Filter data by year
        merged_year_df = merged_df[merged_df['Year'] == year]
        fpts_year_df = fpts_df[fpts_df['Year'] == year]

        # Skip if no data
        if merged_year_df.empty or fpts_year_df.empty:
            print(f"⚠️ Skipping Year {year} (No data available)")
            continue

        # Select random FantasyPros_IDs instead of player names
        sampled_ids = sample(list(merged_year_df['FantasyPros_ID'].dropna()), min(num_samples, len(merged_year_df)))

        total_comparisons = 0
        match_count = 0

        for fp_id in sampled_ids:
            for week in merged_year_df[merged_year_df['FantasyPros_ID'] == fp_id]['Week'].unique():

                merged_row = merged_year_df[(merged_year_df['FantasyPros_ID'] == fp_id) & (merged_year_df['Week'] == week)]
                fpts_row = fpts_year_df[(fpts_year_df['FantasyPros_ID'] == fp_id) & (fpts_year_df['Week'] == week)]

                if merged_row.empty or fpts_row.empty:
                    continue  # Skip if missing

                for col in ['FPTS', 'ROST']:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    fpts_val = fpts_row[col].values[0] if col in fpts_row else None

                    if merged_val is not None and fpts_val is not None:
                        total_comparisons += 1
                        if merged_val == fpts_val:
                            match_count += 1
                        else:
                            mismatches.append((year, fp_id, week, col, merged_val, fpts_val))

        # Yearly summary
        match_percentage = (match_count / total_comparisons * 100) if total_comparisons else 0
        print(f"✅ Tested {len(sampled_ids)} IDs in Year {year} ({total_comparisons} values compared, {match_percentage:.2f}% matched)")

    # summary
    print("\n🔍 Integrity Check Summary:")
    print(f"✔ Total Comparisons: {sum(len(m) for m in mismatches) + match_count}")
    print(f"✔ Overall Match Rate: {(match_count / (match_count + len(mismatches)) * 100) if (match_count + len(mismatches)) > 0 else 0:.2f}%")

    if mismatches:
        print("\n❌ Mismatch Details (Showing up to 10 cases):")
        for m in mismatches[:10]:
            print(m)
    else:
        print("✅ All values matched successfully!")

# ✅ Run the updated test
test_rb_fpts_rost()



🔹 Running Fantasy Points & % Rostered Integrity Check...
📌 Test Details: Stratified Random Sample by Year
📌 Columns Tested: ['FPTS', 'ROST']

📥 Data loaded from **variables** in memory.


Testing Year: 2017
✅ Tested 25 IDs in Year 2017 (190 values compared, 100.00% matched)

Testing Year: 2018
✅ Tested 25 IDs in Year 2018 (246 values compared, 100.00% matched)

Testing Year: 2019
✅ Tested 25 IDs in Year 2019 (226 values compared, 100.00% matched)

Testing Year: 2020
✅ Tested 25 IDs in Year 2020 (188 values compared, 100.00% matched)

Testing Year: 2021
✅ Tested 25 IDs in Year 2021 (186 values compared, 100.00% matched)

Testing Year: 2022
✅ Tested 25 IDs in Year 2022 (192 values compared, 100.00% matched)

Testing Year: 2023
✅ Tested 25 IDs in Year 2023 (216 values compared, 100.00% matched)

🔍 Integrity Check Summary:
✔ Total Comparisons: 216
✔ Overall Match Rate: 100.00%
✅ All values matched successfully!


In [40]:
## Data Integrity Checks ##
# Output: A Stratified Random Sample of years and players from the scraped FantasyPros data
# test to ensure merged df values match the individual unmerged df of redzone stats
# ✅ Integrity Check for RB Red Zone Stats (using FantasyPros_ID)
def test_rb_redzone_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1, random_seed=42):
    """
    Integrity check for RB Red Zone stats.

    - Uses Stratified Random Sampling by year.
    - Tests key stats (REC_rz, TGT_rz, REC PCT_rz, Y/R_rz, TGT PCT_rz).
    - Reports % of matches and mismatches found.
    - Logs whether data is loaded from variables or CSV files.
    """
    seed(random_seed)  # Set seed for reproducibility
    mismatches = []

    print("\n🔹 Running Red Zone Stats Integrity Check...")
    print(f"📌 Test Details: Stratified Random Sample by Year")
    print(f"📌 Columns Tested: ['REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz']\n")

    # Load data from memory or CSV
    merged_df = load_dataframe("rb_adv_fpts_rost_rz_merged_df_sorted", "fantasypros_rb_adv_fpts_rost_rz_df.csv")
    redzone_df = load_dataframe("rb_redzone_stats_df", "fantasypros_rb_redzone_stats.csv")

    # Log the source of the data
    if isinstance(merged_df, pd.DataFrame) and isinstance(redzone_df, pd.DataFrame):
        print("📥 Data loaded from **variables** in memory.\n")
    else:
        print("📥 Data loaded from **CSV files**.\n")

    for year in years:
        print(f"\nTesting Year: {year}")

        # Filter data by year
        merged_year_df = merged_df[merged_df['Year'] == year]
        redzone_year_df = redzone_df[redzone_df['Year'] == year]

        # Skip if no data
        if merged_year_df.empty or redzone_year_df.empty:
            print(f"⚠️ Skipping Year {year} (No data available)")
            continue

        # Select random IDs instead of player names
        sampled_ids = sample(list(merged_year_df['FantasyPros_ID'].dropna()), min(num_samples, len(merged_year_df)))

        total_comparisons = 0
        match_count = 0

        for fp_id in sampled_ids:
            for week in merged_year_df[merged_year_df['FantasyPros_ID'] == fp_id]['Week'].unique():

                merged_row = merged_year_df[(merged_year_df['FantasyPros_ID'] == fp_id) & (merged_year_df['Week'] == week)]
                redzone_row = redzone_year_df[(redzone_year_df['FantasyPros_ID'] == fp_id) & (redzone_year_df['Week'] == week)]

                if merged_row.empty or redzone_row.empty:
                    continue  # Skip if missing

                for col in ['REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz']:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    redzone_val = redzone_row[col].values[0] if col in redzone_row else None

                    if merged_val is not None and redzone_val is not None:
                        total_comparisons += 1
                        if merged_val == redzone_val:
                            match_count += 1
                        else:
                            mismatches.append((year, fp_id, week, col, merged_val, redzone_val))

        # Yearly summary
        match_percentage = (match_count / total_comparisons * 100) if total_comparisons else 0
        print(f"✅ Tested {len(sampled_ids)} IDs in Year {year} ({total_comparisons} values compared, {match_percentage:.2f}% matched)")

    # summary
    print("\n🔍 Integrity Check Summary:")
    print(f"✔ Total Comparisons: {sum(len(m) for m in mismatches) + match_count}")
    print(f"✔ Overall Match Rate: {(match_count / (match_count + len(mismatches)) * 100) if (match_count + len(mismatches)) > 0 else 0:.2f}%")

    if mismatches:
        print("\n❌ Mismatch Details (Showing up to 10 cases):")
        for m in mismatches[:10]:
            print(m)
    else:
        print("✅ All values matched successfully!")

# ✅ Run the updated test
test_rb_redzone_stats()



🔹 Running Red Zone Stats Integrity Check...
📌 Test Details: Stratified Random Sample by Year
📌 Columns Tested: ['REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz']

📥 Data loaded from **variables** in memory.


Testing Year: 2017
✅ Tested 25 IDs in Year 2017 (460 values compared, 100.00% matched)

Testing Year: 2018
✅ Tested 25 IDs in Year 2018 (610 values compared, 100.00% matched)

Testing Year: 2019
✅ Tested 25 IDs in Year 2019 (560 values compared, 100.00% matched)

Testing Year: 2020
✅ Tested 25 IDs in Year 2020 (435 values compared, 100.00% matched)

Testing Year: 2021
✅ Tested 25 IDs in Year 2021 (440 values compared, 100.00% matched)

Testing Year: 2022
✅ Tested 25 IDs in Year 2022 (480 values compared, 100.00% matched)

Testing Year: 2023
✅ Tested 25 IDs in Year 2023 (535 values compared, 100.00% matched)

🔍 Integrity Check Summary:
✔ Total Comparisons: 535
✔ Overall Match Rate: 100.00%
✅ All values matched successfully!


In [41]:
### End: FantasyPros Data Integrity Checks Section ###

In [42]:
## Begin: data normalization for fantasypros dataframes ##

In [43]:
# Pre-Normalization
# output: display the columns to confirm all dataframes are present 

# Establish the initial dataframes without any normalization
dataframes = {
    "Merged DataFrame": rb_adv_fpts_rost_rz_merged_df_sorted,
    "Advanced Stats DataFrame": rb_fp_advanced_stats_df,
    "FPTS & ROST DataFrame": rb_fpts_perct_rost_df,
    "Redzone Stats DataFrame": rb_redzone_stats_df
}

# Extract and compare column names
column_details = {name: df.columns.tolist() for name, df in dataframes.items()}

# Convert to DataFrame for easier comparison
columns_df = pd.DataFrame.from_dict(column_details, orient="index").transpose()

# Display column comparison
print("🔍 Initial Column Comparison Across DataFrames:\n")
print(columns_df)


🔍 Initial Column Comparison Across DataFrames:

   Merged DataFrame Advanced Stats DataFrame FPTS & ROST DataFrame Redzone Stats DataFrame
0              Year                     Year        FantasyPros_ID          FantasyPros_ID
1              Week                     Week                   Y/A                  Y/A_rz
2    FantasyPros_ID           FantasyPros_ID                  FPTS                  REC_rz
3            Player                   Player                  ROST                  TGT_rz
4            Player                   Player                  Year              REC PCT_rz
5                 G                        G                  Week                  Y/R_rz
6               ATT                      ATT                  None              TGT PCT_rz
7               YDS                      YDS                  None                    Year
8             YBCON                    YBCON                  None                    Week
9             YACON                    YAC

In [44]:
# Data Normalization - Rename Columns for Consistency
# output: display columns

# Initialize dataframes and rename 'Year' to 'Season' where needed
dataframes = {
    "Merged DataFrame": rb_adv_fpts_rost_rz_merged_df_sorted.copy(),
    "Advanced Stats DataFrame": rb_fp_advanced_stats_df.copy(),
    "FPTS & ROST DataFrame": rb_fpts_perct_rost_df.copy(),
    "Redzone Stats DataFrame": rb_redzone_stats_df.copy()
}

# Rename 'Year' to 'Season' for consistency
for name, df in dataframes.items():
    if 'Year' in df.columns:
        df.rename(columns={'Year': 'Season'}, inplace=True)

# Check renamed columns
for name, df in dataframes.items():
    print(f"\n📊 {name} Columns After Renaming:")
    print(df.columns.tolist())


📊 Merged DataFrame Columns After Renaming:
['Season', 'Week', 'FantasyPros_ID', 'Player', 'Player', 'G', 'ATT', 'YDS', 'YBCON', 'YACON', 'BRKTKL', 'TK LOSS', 'TK LOSS YDS', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'REC', 'TGT', 'RZ TGT', 'YACON', 'Y/A', 'FPTS', 'ROST', 'Y/A_rz', 'REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz']

📊 Advanced Stats DataFrame Columns After Renaming:
['Season', 'Week', 'FantasyPros_ID', 'Player', 'Player', 'G', 'ATT', 'YDS', 'YBCON', 'YACON', 'BRKTKL', 'TK LOSS', 'TK LOSS YDS', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'REC', 'TGT', 'RZ TGT', 'YACON']

📊 FPTS & ROST DataFrame Columns After Renaming:
['FantasyPros_ID', 'Y/A', 'FPTS', 'ROST', 'Season', 'Week']

📊 Redzone Stats DataFrame Columns After Renaming:
['FantasyPros_ID', 'Y/A_rz', 'REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz', 'Season', 'Week']


In [45]:
# Data normalization - handle duplicate 'Player' columns in the merged and advanced stats dataframes
# output: display columns in the merged and advanced stats dataframes

# Rename duplicate 'Player' columns for the relevant dataframes
def rename_player_columns(dataframes):
    # Rename 'Player' columns in the Merged DataFrame
    merged_df = dataframes["Merged DataFrame"]
    merged_df.columns.values[4] = "PlayerTM"  # Only renaming index 4

    # Rename 'Player' columns in the Advanced Stats DataFrame
    adv_stats_df = dataframes["Advanced Stats DataFrame"]
    adv_stats_df.columns.values[4] = "PlayerTM"  # Only renaming index 4


    # Confirm renaming
    print("\n📊 Merged DataFrame Columns After Renaming:")
    print(merged_df.columns.tolist())

    print("\n📊 Advanced Stats DataFrame Columns After Renaming:")
    print(adv_stats_df.columns.tolist())

# Apply the renaming function
rename_player_columns(dataframes)


📊 Merged DataFrame Columns After Renaming:
['Season', 'Week', 'FantasyPros_ID', 'Player', 'PlayerTM', 'G', 'ATT', 'YDS', 'YBCON', 'YACON', 'BRKTKL', 'TK LOSS', 'TK LOSS YDS', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'REC', 'TGT', 'RZ TGT', 'YACON', 'Y/A', 'FPTS', 'ROST', 'Y/A_rz', 'REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz']

📊 Advanced Stats DataFrame Columns After Renaming:
['Season', 'Week', 'FantasyPros_ID', 'Player', 'PlayerTM', 'G', 'ATT', 'YDS', 'YBCON', 'YACON', 'BRKTKL', 'TK LOSS', 'TK LOSS YDS', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'REC', 'TGT', 'RZ TGT', 'YACON']


In [46]:
# Data normalization - confirm renamed 'Player' column in the merged and advanced stats dataframes
# output: display first five rows in the merged and advanced stats dataframes

# Display the first 5 rows of the Merged DataFrame
print("\n🔍 First 5 Rows of Merged DataFrame:")
display(dataframes["Merged DataFrame"].head())

# Display the first 5 rows of the Advanced Stats DataFrame
print("\n🔍 First 5 Rows of Advanced Stats DataFrame:")
display(dataframes["Advanced Stats DataFrame"].head())


🔍 First 5 Rows of Merged DataFrame:


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,ATT,YDS,YBCON,YACON,BRKTKL,TK LOSS,TK LOSS YDS,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,REC,TGT,RZ TGT,YACON,Y/A,FPTS,ROST,Y/A_rz,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz
3,2017,1,16450,Tarik Cohen,Tarik Cohen (FA),1,5,66,6,60,2,1,-3,2,1,1,1,0,8,12,1,20,13.2,17.3,0.0%,5.0,1.0,1.0,100.0%,19.0,50.0%
7,2017,1,9317,Jonathan Stewart,Jonathan Stewart (FA),1,18,65,18,47,1,0,0,1,0,0,0,0,2,2,1,11,3.6,14.2,0.0%,1.5,1.0,1.0,100.0%,9.0,100.0%
6,2017,1,13977,Ty Montgomery II,Ty Montgomery II (FA),1,19,54,15,39,0,1,-1,0,0,0,0,0,4,4,0,25,2.8,15.3,0.0%,6.0,0.0,0.0,0%,0.0,0%
1,2017,1,11657,Mike Gillislee,Mike Gillislee (FA),1,15,45,29,16,0,1,-4,1,0,0,0,0,0,0,0,0,3.0,22.5,0.0%,2.5,0.0,0.0,0%,0.0,0.0%
5,2017,1,13912,Todd Gurley II,Todd Gurley II (FA),1,19,40,7,33,2,3,-13,1,0,0,0,0,5,6,0,41,2.1,15.6,0.6%,3.0,0.0,0.0,0%,0.0,0%



🔍 First 5 Rows of Advanced Stats DataFrame:


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,ATT,YDS,YBCON,YACON,BRKTKL,TK LOSS,TK LOSS YDS,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,REC,TGT,RZ TGT,YACON
0,2017,1,16425,Kareem Hunt,Kareem Hunt (KC),1,17,148,104,44,6,0,0,4,1,1,1,1,5,5,0,36
1,2017,1,11657,Mike Gillislee,Mike Gillislee (FA),1,15,45,29,16,0,1,-4,1,0,0,0,0,0,0,0,0
2,2017,1,16378,Leonard Fournette,Leonard Fournette (FA),1,26,100,52,48,2,3,-3,2,0,0,0,0,3,3,0,5
3,2017,1,16450,Tarik Cohen,Tarik Cohen (FA),1,5,66,6,60,2,1,-3,2,1,1,1,0,8,12,1,20
4,2017,1,9398,LeSean McCoy,LeSean McCoy (FA),1,22,110,86,24,1,4,-6,4,3,0,0,0,5,6,0,2


In [47]:
# data normalization - reorder columns
# output: first three columns for all dataframes: 'Season', 'Week', 'FantasyPros_ID'

# Define a helper function to reorder columns in-place
def reorder_columns_inplace(df, first_cols=['Season', 'Week', 'FantasyPros_ID']):
    """Reorders the DataFrame columns to place first_cols at the beginning in-place."""
    current_cols = list(df.columns)
    remaining_cols = [col for col in current_cols if col not in first_cols]
    reordered_df = df[first_cols + remaining_cols]  # Use list indexing instead of reindex
    return reordered_df

# Apply the function to each DataFrame separately
dataframes["Merged DataFrame"] = reorder_columns_inplace(dataframes["Merged DataFrame"])
dataframes["Advanced Stats DataFrame"] = reorder_columns_inplace(dataframes["Advanced Stats DataFrame"])
dataframes["FPTS & ROST DataFrame"] = reorder_columns_inplace(dataframes["FPTS & ROST DataFrame"])
dataframes["Redzone Stats DataFrame"] = reorder_columns_inplace(dataframes["Redzone Stats DataFrame"])

# Confirm order
for name, df in dataframes.items():
    print(f"\n🔍 {name} First Three Columns: {df.columns[:3].tolist()}")


🔍 Merged DataFrame First Three Columns: ['Season', 'Week', 'FantasyPros_ID']

🔍 Advanced Stats DataFrame First Three Columns: ['Season', 'Week', 'FantasyPros_ID']

🔍 FPTS & ROST DataFrame First Three Columns: ['Season', 'Week', 'FantasyPros_ID']

🔍 Redzone Stats DataFrame First Three Columns: ['Season', 'Week', 'FantasyPros_ID']


In [48]:
# Pre-Normalization - Initial Data Type Inspection
# Output: columns and data types

# Inspect data types of each dataframe before applying normalization
for name, df in dataframes.items():
    print(f"\n📊 Data Types Summary for {name}:")
    print(df.dtypes)


📊 Data Types Summary for Merged DataFrame:
Season              int64
Week                int64
FantasyPros_ID     object
Player             object
PlayerTM           object
G                  object
ATT                object
YDS                object
YBCON              object
YACON              object
YACON              object
BRKTKL             object
TK LOSS            object
TK LOSS YDS        object
10+ YDS            object
20+ YDS            object
30+ YDS            object
40+ YDS            object
50+ YDS            object
REC                object
TGT                object
RZ TGT             object
YACON              object
YACON              object
Y/A               float64
FPTS              float64
ROST               object
Y/A_rz            float64
REC_rz            float64
TGT_rz            float64
REC PCT_rz         object
Y/R_rz            float64
TGT PCT_rz         object
dtype: object

📊 Data Types Summary for Advanced Stats DataFrame:
Season             int64
Week   

In [49]:
# data normalization - convert all columns to object (str) types
# no output
# Helper Function: Convert to Object & Strip Whitespaces
def convert_to_object_and_strip(df):
    """
    Converts all columns in a DataFrame to object type and strips leading/trailing whitespaces.
    """
    df_copy = df.copy()  # Create a copy to avoid modifying the original dataframe

    for col in df_copy.columns:
        # Convert to object type
        df_copy[col] = df_copy[col].astype(str).apply(lambda x: x.strip() if isinstance(x, str) else x)
    
    return df_copy

In [50]:
# Apply the function to all dataframes
dataframes = {name: convert_to_object_and_strip(df) for name, df in dataframes.items()}

# Confirm data type conversion
for name, df in dataframes.items():
    print(f"\n📊 Data Types Summary for {name}:")
    print(df.dtypes)


📊 Data Types Summary for Merged DataFrame:
Season            object
Week              object
FantasyPros_ID    object
Player            object
PlayerTM          object
G                 object
ATT               object
YDS               object
YBCON             object
YACON             object
YACON             object
BRKTKL            object
TK LOSS           object
TK LOSS YDS       object
10+ YDS           object
20+ YDS           object
30+ YDS           object
40+ YDS           object
50+ YDS           object
REC               object
TGT               object
RZ TGT            object
YACON             object
YACON             object
Y/A               object
FPTS              object
ROST              object
Y/A_rz            object
REC_rz            object
TGT_rz            object
REC PCT_rz        object
Y/R_rz            object
TGT PCT_rz        object
dtype: object

📊 Data Types Summary for Advanced Stats DataFrame:
Season            object
Week              object
FantasyPros_ID 

In [51]:
# data normalization - verification of data types
# output: a display of all columns in all dataframes as 'object' datatypes 

def compare_column_dtypes(dataframes, merged_df_name="Merged DataFrame"):
    """
    Compare column data types between the merged dataframe and each unmerged dataframe.

    Parameters:
    - dataframes: dict of dataframes to compare.
    - merged_df_name: Name of the merged dataframe in the dictionary.

    Returns:
    - Comparison results printed to the console.
    """
    merged_df = dataframes[merged_df_name]
    merged_dtypes = merged_df.dtypes.to_dict()

    comparison_results = {}

    for name, df in dataframes.items():
        if name == merged_df_name:
            continue  # Skip comparison with itself

        df_dtypes = df.dtypes.to_dict()
        mismatch_report = []

        for col, dtype in df_dtypes.items():
            if col in merged_dtypes:
                if dtype != merged_dtypes[col]:
                    mismatch_report.append((col, dtype, merged_dtypes[col]))

        if mismatch_report:
            comparison_results[name] = mismatch_report
        else:
            comparison_results[name] = "✅ All data types match"

    # Display results
    print("\n🔍 Column Data Type Comparison Results:\n")
    for name, result in comparison_results.items():
        if result == "✅ All data types match":
            print(f"{name}: ✅ All data types match")
        else:
            print(f"\n{name}: ❌ Mismatched Columns Detected")
            for col, dtype_unmerged, dtype_merged in result:
                print(f"  - Column: {col} | Unmerged Type: {dtype_unmerged} | Merged Type: {dtype_merged}")

# Run the comparison
compare_column_dtypes(dataframes)


🔍 Column Data Type Comparison Results:

Advanced Stats DataFrame: ✅ All data types match
FPTS & ROST DataFrame: ✅ All data types match
Redzone Stats DataFrame: ✅ All data types match


In [52]:
# data normalization - verification of column order
# output: display of the first three columns of all dataframes

# Define the correct order of columns
expected_first_columns = ['Season', 'Week', 'FantasyPros_ID']

# Function to check column order
def check_column_order(dataframes, expected_first_columns):
    """
    Check if the first columns of each dataframe match the expected columns.

    Parameters:
    - dataframes: dict of dataframes to check.
    - expected_first_columns: list of column names expected to appear first.

    Returns:
    - Prints the order check results for each dataframe.
    """
    print("\n🔍 Column Order Verification:\n")
    for name, df in dataframes.items():
        first_columns = df.columns[:len(expected_first_columns)].tolist()
        
        if first_columns == expected_first_columns:
            print(f"{name}: ✅ Columns are in the correct order: {first_columns}")
        else:
            print(f"{name}: ❌ Incorrect column order. Found: {first_columns} | Expected: {expected_first_columns}")

# Run the column order check
check_column_order(dataframes, expected_first_columns)


🔍 Column Order Verification:

Merged DataFrame: ✅ Columns are in the correct order: ['Season', 'Week', 'FantasyPros_ID']
Advanced Stats DataFrame: ✅ Columns are in the correct order: ['Season', 'Week', 'FantasyPros_ID']
FPTS & ROST DataFrame: ✅ Columns are in the correct order: ['Season', 'Week', 'FantasyPros_ID']
Redzone Stats DataFrame: ✅ Columns are in the correct order: ['Season', 'Week', 'FantasyPros_ID']


In [53]:
# data normalization - check to ensure there are no missing columns
# output: an empty dictionary {}

# Define expected columns for each dataframe
# Updated expected columns for each dataframe
expected_columns = {
    "Merged DataFrame": [
        'Season', 'Week', 'FantasyPros_ID', 'Player', 'PlayerTM', 'G', 'REC', 'YDS', 
        'YBCON', 'YACON', 'BRKTKL', 'TGT', 
        'RZ TGT', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 
        'FPTS', 'ROST', 'REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz', 
        'TK LOSS', 'TK LOSS YDS', 'ATT', 'Y/A', 'Y/A_rz'
    ],
    "Advanced Stats DataFrame": [
        'Season', 'Week', 'FantasyPros_ID', 'Player', 'PlayerTM', 'G', 'REC', 'YDS', 
        'YBCON', 'YACON', 'BRKTKL', 'TGT', 
        'RZ TGT', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 
        'TK LOSS', 'TK LOSS YDS', 'ATT'
    ],
    "FPTS & ROST DataFrame": [
        'Season', 'Week', 'FantasyPros_ID', 'FPTS', 'ROST', 'Y/A'
    ],
    "Redzone Stats DataFrame": [
        'Season', 'Week', 'FantasyPros_ID', 'REC_rz', 'TGT_rz', 
        'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz', 'Y/A_rz'
    ]
}

# Check for missing and unexpected columns in each dataframe
column_issues = {}

for df_name, df in dataframes.items():
    expected = set(expected_columns[df_name])
    actual = set(df.columns.tolist())
    
    # Identify missing and unexpected columns
    missing = expected - actual
    unexpected = actual - expected
    
    if missing or unexpected:
        column_issues[df_name] = {
            "Missing Columns": list(missing) if missing else None,
            "Unexpected Columns": list(unexpected) if unexpected else None
        }

# Display results
if column_issues:
    print("🔍 Column Presence Issues Detected:\n")
    for df_name, issues in column_issues.items():
        print(f"{df_name}:")
        if issues["Missing Columns"]:
            print(f"  ❌ Missing Columns: {issues['Missing Columns']}")
        if issues["Unexpected Columns"]:
            print(f"  ⚠️  Unexpected Columns: {issues['Unexpected Columns']}")
else:
    print("✅ All columns are present and consistent across all dataframes.")


✅ All columns are present and consistent across all dataframes.


In [54]:
# data normalization - remove leading and trailing whitespaces from all object (string) columns 
# no output

# def remove_whitespace(df, inplace=False):
#     if not inplace:
#         df = df.copy()  # Only create a copy if inplace is False

#     for col in df.select_dtypes(include=['object']).columns:
#         df[col] = df[col].str.strip()  # Automatically skips non-string values
    
#     if not inplace:
#         return df  # Return the modified DataFrame if not modifying in place


def remove_whitespace(df, inplace=False):
    """
    Safely strips leading/trailing whitespace from all string values in object-type columns.
    """
    if not inplace:
        df = df.copy()

    for col in df.select_dtypes(include='object').columns:
        # Only apply .str.strip() if the column contains string values
        if df[col].apply(lambda x: isinstance(x, str)).any():
            df[col] = df[col].astype(str).str.strip()

    if not inplace:
        return df

In [55]:
# data normalization - remove white spaces
# output: dataframes stripped of whitespaces

# Apply whitespace removal to each dataframe separately
dataframes["Merged DataFrame"] = remove_whitespace(dataframes["Merged DataFrame"])
dataframes["Advanced Stats DataFrame"] = remove_whitespace(dataframes["Advanced Stats DataFrame"])
dataframes["FPTS & ROST DataFrame"] = remove_whitespace(dataframes["FPTS & ROST DataFrame"])
dataframes["Redzone Stats DataFrame"] = remove_whitespace(dataframes["Redzone Stats DataFrame"])

# Confirm whitespace removal by displaying first few rows
for name, df in dataframes.items():
    print(f"\n🔍 {name} - Sample Data (Post Whitespace Removal):")
    print(df.head(5))


🔍 Merged DataFrame - Sample Data (Post Whitespace Removal):
  Season Week FantasyPros_ID            Player               PlayerTM  G ATT YDS YBCON YACON YACON BRKTKL TK LOSS TK LOSS YDS 10+ YDS 20+ YDS 30+ YDS 40+ YDS 50+ YDS REC TGT RZ TGT YACON YACON   Y/A  FPTS  ROST Y/A_rz REC_rz TGT_rz REC PCT_rz Y/R_rz TGT PCT_rz
3   2017    1          16450       Tarik Cohen       Tarik Cohen (FA)  1   5  66     6    60    20      2       1          -3       2       1       1       1       0   8  12      1    60    20  13.2  17.3  0.0%    5.0    1.0    1.0     100.0%   19.0      50.0%
7   2017    1           9317  Jonathan Stewart  Jonathan Stewart (FA)  1  18  65    18    47    11      1       0           0       1       0       0       0       0   2   2      1    47    11   3.6  14.2  0.0%    1.5    1.0    1.0     100.0%    9.0     100.0%
6   2017    1          13977  Ty Montgomery II  Ty Montgomery II (FA)  1  19  54    15    39    25      0       1          -1       0       0       0       

In [56]:
# data normalization  - standardize decimal places for numerical data and percentages
# no output
def format_precision(df, precision_map=None):
    if precision_map is None:
        precision_map = {
            'REC PCT_rz': '.2f',
            'TGT PCT_rz': '.2f',
            'ROST': '.3f',
            'Y/R_rz': '.1f'
        }

    df_copy = df.copy()
    
    for col, fmt in precision_map.items():
        if col in df_copy.columns:
            try:
                df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')  # Convert to numeric
                df_copy[col] = df_copy[col].map(lambda x: format(x, fmt) if pd.notna(x) else "nan")
            except Exception as e:
                print(f"⚠️ Error formatting column {col}: {e}")
    
    return df_copy

In [57]:
# data normalization - apply decimal place precision  
# output

# Define precision map (same for all relevant dataframes)
precision_map = {
    'REC PCT_rz': '.2f',
    'TGT PCT_rz': '.2f',
    'ROST': '.3f',
    'Y/R_rz': '.1f'
}

# Apply formatting function to each dataframe
dataframes["Merged DataFrame"] = format_precision(dataframes["Merged DataFrame"], precision_map)
dataframes["Advanced Stats DataFrame"] = format_precision(dataframes["Advanced Stats DataFrame"], precision_map)
dataframes["FPTS & ROST DataFrame"] = format_precision(dataframes["FPTS & ROST DataFrame"], precision_map)
dataframes["Redzone Stats DataFrame"] = format_precision(dataframes["Redzone Stats DataFrame"], precision_map)

# Confirm precision formatting by displaying relevant columns
for name, df in dataframes.items():
    print(f"\n🔍 {name} - Sample Data (Post Precision Formatting):")
    display(df.head(5))



🔍 Merged DataFrame - Sample Data (Post Precision Formatting):


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,ATT,YDS,YBCON,YACON,YACON,BRKTKL,TK LOSS,TK LOSS YDS,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,REC,TGT,RZ TGT,YACON,YACON,Y/A,FPTS,ROST,Y/A_rz,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz
3,2017,1,16450,Tarik Cohen,Tarik Cohen (FA),1,5,66,6,60,20,2,1,-3,2,1,1,1,0,8,12,1,60,20,13.2,17.3,nan,5.0,1.0,1.0,nan,19.0,nan
7,2017,1,9317,Jonathan Stewart,Jonathan Stewart (FA),1,18,65,18,47,11,1,0,0,1,0,0,0,0,2,2,1,47,11,3.6,14.2,nan,1.5,1.0,1.0,nan,9.0,nan
6,2017,1,13977,Ty Montgomery II,Ty Montgomery II (FA),1,19,54,15,39,25,0,1,-1,0,0,0,0,0,4,4,0,39,25,2.8,15.3,nan,6.0,0.0,0.0,nan,0.0,nan
1,2017,1,11657,Mike Gillislee,Mike Gillislee (FA),1,15,45,29,16,0,0,1,-4,1,0,0,0,0,0,0,0,16,0,3.0,22.5,nan,2.5,0.0,0.0,nan,0.0,nan
5,2017,1,13912,Todd Gurley II,Todd Gurley II (FA),1,19,40,7,33,41,2,3,-13,1,0,0,0,0,5,6,0,33,41,2.1,15.6,nan,3.0,0.0,0.0,nan,0.0,nan



🔍 Advanced Stats DataFrame - Sample Data (Post Precision Formatting):


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,ATT,YDS,YBCON,YACON,YACON,BRKTKL,TK LOSS,TK LOSS YDS,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,REC,TGT,RZ TGT,YACON,YACON
0,2017,1,16425,Kareem Hunt,Kareem Hunt (KC),1,17,148,104,44,36,6,0,0,4,1,1,1,1,5,5,0,44,36
1,2017,1,11657,Mike Gillislee,Mike Gillislee (FA),1,15,45,29,16,0,0,1,-4,1,0,0,0,0,0,0,0,16,0
2,2017,1,16378,Leonard Fournette,Leonard Fournette (FA),1,26,100,52,48,5,2,3,-3,2,0,0,0,0,3,3,0,48,5
3,2017,1,16450,Tarik Cohen,Tarik Cohen (FA),1,5,66,6,60,20,2,1,-3,2,1,1,1,0,8,12,1,60,20
4,2017,1,9398,LeSean McCoy,LeSean McCoy (FA),1,22,110,86,24,2,1,4,-6,4,3,0,0,0,5,6,0,24,2



🔍 FPTS & ROST DataFrame - Sample Data (Post Precision Formatting):


,Season,Week,FantasyPros_ID,Y/A,FPTS,ROST
0,2017,1,16425,8.7,40.6,nan
1,2017,1,11657,3.0,22.5,nan
2,2017,1,16378,3.8,18.4,nan
3,2017,1,16450,13.2,17.3,nan
4,2017,1,9398,5.0,15.9,nan



🔍 Redzone Stats DataFrame - Sample Data (Post Precision Formatting):


,Season,Week,FantasyPros_ID,Y/A_rz,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz
0,2017,1,11657,2.5,0,0,nan,0.0,nan
1,2017,1,16425,3.0,1,1,nan,3.0,nan
2,2017,1,16450,5.0,1,1,nan,19.0,nan
3,2017,1,13903,2.0,1,1,nan,11.0,nan
4,2017,1,9317,1.5,1,1,nan,9.0,nan


In [58]:
## End: data normalization for fantasypros dataframes ##

In [59]:
### Begin: Enhanced data integrity checks for the fantasypros dataframes ###

In [60]:
# Enhanced data integrity checks
# no output
def generate_md5_hash(row):
    """
    Generate a consistent MD5 hash string from a row's values.
    
    Parameters:
    - row: A pandas Series representing a row in a DataFrame.
    
    Returns:
    - Hash string (MD5)
    """
    row_str = "|".join(map(str, row))
    return hashlib.md5(row_str.encode()).hexdigest()

In [61]:
# Enhanced data integrity checks
# output: Merged DataFrame with a hash column

# Generate hashes for the Merged DataFrame
dataframes["Merged DataFrame"]["row_hash"] = dataframes["Merged DataFrame"].apply(generate_md5_hash, axis=1)

# Display sample hashes
print("\n✅ Merged DataFrame - Hash Generation Complete")
display(dataframes["Merged DataFrame"].head(5))


✅ Merged DataFrame - Hash Generation Complete


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,ATT,YDS,YBCON,YACON,YACON,BRKTKL,TK LOSS,TK LOSS YDS,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,REC,TGT,RZ TGT,YACON,YACON,Y/A,FPTS,ROST,Y/A_rz,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz,row_hash
3,2017,1,16450,Tarik Cohen,Tarik Cohen (FA),1,5,66,6,60,20,2,1,-3,2,1,1,1,0,8,12,1,60,20,13.2,17.3,nan,5.0,1.0,1.0,nan,19.0,nan,466140d08ca781cdcbfa5de13a1d3899
7,2017,1,9317,Jonathan Stewart,Jonathan Stewart (FA),1,18,65,18,47,11,1,0,0,1,0,0,0,0,2,2,1,47,11,3.6,14.2,nan,1.5,1.0,1.0,nan,9.0,nan,8da58948d83fb752104c2e1d1eddf862
6,2017,1,13977,Ty Montgomery II,Ty Montgomery II (FA),1,19,54,15,39,25,0,1,-1,0,0,0,0,0,4,4,0,39,25,2.8,15.3,nan,6.0,0.0,0.0,nan,0.0,nan,d624afa39452680f7bb78740e5c900bb
1,2017,1,11657,Mike Gillislee,Mike Gillislee (FA),1,15,45,29,16,0,0,1,-4,1,0,0,0,0,0,0,0,16,0,3.0,22.5,nan,2.5,0.0,0.0,nan,0.0,nan,71d99a51e700a043a8f4636db4394ad6
5,2017,1,13912,Todd Gurley II,Todd Gurley II (FA),1,19,40,7,33,41,2,3,-13,1,0,0,0,0,5,6,0,33,41,2.1,15.6,nan,3.0,0.0,0.0,nan,0.0,nan,8e468ddbccb28eab0baedec9d2b4ecc7


In [62]:
# Enhanced data integrity checks
# output: Advanced Stat DataFrame with a hash column

# Generate hashes for the Advanced Stats DataFrame
dataframes["Advanced Stats DataFrame"]["row_hash"] = dataframes["Advanced Stats DataFrame"].apply(generate_md5_hash, axis=1)

# Display sample hashes
print("\n✅ Advanced Stats DataFrame - Hash Generation Complete")
display(dataframes["Advanced Stats DataFrame"].head(5))



✅ Advanced Stats DataFrame - Hash Generation Complete


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,ATT,YDS,YBCON,YACON,YACON,BRKTKL,TK LOSS,TK LOSS YDS,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,REC,TGT,RZ TGT,YACON,YACON,row_hash
0,2017,1,16425,Kareem Hunt,Kareem Hunt (KC),1,17,148,104,44,36,6,0,0,4,1,1,1,1,5,5,0,44,36,5def653d1ef58b7764c7abdd4429cde9
1,2017,1,11657,Mike Gillislee,Mike Gillislee (FA),1,15,45,29,16,0,0,1,-4,1,0,0,0,0,0,0,0,16,0,515a3b618f0059ec8522763696fbf5c8
2,2017,1,16378,Leonard Fournette,Leonard Fournette (FA),1,26,100,52,48,5,2,3,-3,2,0,0,0,0,3,3,0,48,5,45c3d0b9ddf6cb851e52e84b3bd410d9
3,2017,1,16450,Tarik Cohen,Tarik Cohen (FA),1,5,66,6,60,20,2,1,-3,2,1,1,1,0,8,12,1,60,20,c6375b97d2656d6fd2c39e59c585b85b
4,2017,1,9398,LeSean McCoy,LeSean McCoy (FA),1,22,110,86,24,2,1,4,-6,4,3,0,0,0,5,6,0,24,2,7f199a87de10b2eb4bba0c94f3d047f5


In [63]:
# Enhanced data integrity checks
# output: FPTS & ROST DataFrame with a hash column

# Generate hashes for the FPTS & ROST DataFrame
dataframes["FPTS & ROST DataFrame"]["row_hash"] = dataframes["FPTS & ROST DataFrame"].apply(generate_md5_hash, axis=1)

# Display sample hashes
print("\n✅ FPTS & ROST DataFrame - Hash Generation Complete")
display(dataframes["FPTS & ROST DataFrame"].head(5))


✅ FPTS & ROST DataFrame - Hash Generation Complete


,Season,Week,FantasyPros_ID,Y/A,FPTS,ROST,row_hash
0,2017,1,16425,8.7,40.6,nan,c0699bddb70c8886d93364169ee483e8
1,2017,1,11657,3.0,22.5,nan,a08875975a9984efb46a70b3c74b6c42
2,2017,1,16378,3.8,18.4,nan,f3e36eedca523cd7baea9795a762f09e
3,2017,1,16450,13.2,17.3,nan,9f9f010e4e7d1b1d5f1ee541b8bd8253
4,2017,1,9398,5.0,15.9,nan,0b36ae2a3ffacc0eb700712f60ceca9c


In [64]:
# Enhanced data integrity checks
# output: Redzone Stats DataFrame with a hash column

# Generate hashes for the Redzone Stats DataFrame
dataframes["Redzone Stats DataFrame"]["row_hash"] = dataframes["Redzone Stats DataFrame"].apply(generate_md5_hash, axis=1)

# Display sample hashes
print("\n✅ Redzone Stats DataFrame - Hash Generation Complete")
display(dataframes["Redzone Stats DataFrame"].head(5))


✅ Redzone Stats DataFrame - Hash Generation Complete


,Season,Week,FantasyPros_ID,Y/A_rz,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz,row_hash
0,2017,1,11657,2.5,0,0,nan,0.0,nan,b246359aa520e88f1f3715813153b51f
1,2017,1,16425,3.0,1,1,nan,3.0,nan,00910e1f0cec301c240cc195157afb97
2,2017,1,16450,5.0,1,1,nan,19.0,nan,fe62692eeef2c1cf5b7708b7dadb8f63
3,2017,1,13903,2.0,1,1,nan,11.0,nan,3bcf4e0cb9e64a05c76c1c86027bf08f
4,2017,1,9317,1.5,1,1,nan,9.0,nan,d8b673e50883aacebb11e27d4a06b0d9


In [65]:
# Enhanced data integrity checks
# output: renamed the merged dataframe 

# Create a copy of the merged dataframe with "_hash" suffix for modification
rb_adv_fpts_rost_rz_merged_df_sorted_hash = dataframes["Merged DataFrame"].copy()

# Verify the creation of the new dataframe
print("\n✅ New Merged DataFrame Created: 'rb_adv_fpts_rost_rz_merged_df_sorted_hash'")


✅ New Merged DataFrame Created: 'rb_adv_fpts_rost_rz_merged_df_sorted_hash'


In [66]:
# Enhanced data integrity checks
# output: columns of merged dataframe with new hash columns

# Define the new columns to be added for hashing
new_hash_columns = ['adv_stats_hash', 'fpts_rost_hash', 'redzone_hash']

# Initialize the new columns with empty strings (or NaN if preferred)
for col in new_hash_columns:
    rb_adv_fpts_rost_rz_merged_df_sorted_hash[col] = ""  # Empty strings as placeholders

# Verify creation of columns
print("\n✅ Hash columns added to the new merged dataframe:")
print(rb_adv_fpts_rost_rz_merged_df_sorted_hash.columns.tolist())


✅ Hash columns added to the new merged dataframe:
['Season', 'Week', 'FantasyPros_ID', 'Player', 'PlayerTM', 'G', 'ATT', 'YDS', 'YBCON', 'YACON', 'YACON', 'BRKTKL', 'TK LOSS', 'TK LOSS YDS', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'REC', 'TGT', 'RZ TGT', 'YACON', 'YACON', 'Y/A', 'FPTS', 'ROST', 'Y/A_rz', 'REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz', 'row_hash', 'adv_stats_hash', 'fpts_rost_hash', 'redzone_hash']


In [67]:
# Enhanced data integrity checks
# output: last 5 columns of the new merged dataframe with row hashes

# Display the last 5 columns to confirm the new hash columns are present
print("\n🔍 Last 5 Columns in the New Merged DataFrame:")
print(rb_adv_fpts_rost_rz_merged_df_sorted_hash.columns[-5:])


🔍 Last 5 Columns in the New Merged DataFrame:
Index(['TGT PCT_rz', 'row_hash', 'adv_stats_hash', 'fpts_rost_hash', 'redzone_hash'], dtype='object')


In [68]:
# Enhanced data integrity checks - define Relevant Columns for Each Hash Type
# output: columns to be hashed

# Define the columns to be used for each hash type
hash_column_groups = {
    "adv_stats_hash": [
        'Season', 'Week', 'FantasyPros_ID', 'REC', 'YDS', 'YBCON', 'YACON',
        'TGT', 'BRKTKL', 'RZ TGT', '10+ YDS', '20+ YDS',
        '30+ YDS', '40+ YDS', '50+ YDS'
    ],
    "fpts_rost_hash": [
        'Season', 'Week', 'FantasyPros_ID', 'FPTS', 'ROST'
    ],
    "redzone_hash": [
        'Season', 'Week', 'FantasyPros_ID', 'REC_rz', 'TGT_rz',
        'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz'
    ]
}



# Confirm the column groups are set correctly
print("\n✅ Hash Column Groups Defined:")
for hash_name, columns in hash_column_groups.items():
    print(f"  - {hash_name}: {columns}")



✅ Hash Column Groups Defined:
  - adv_stats_hash: ['Season', 'Week', 'FantasyPros_ID', 'REC', 'YDS', 'YBCON', 'YACON', 'TGT', 'BRKTKL', 'RZ TGT', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS']
  - fpts_rost_hash: ['Season', 'Week', 'FantasyPros_ID', 'FPTS', 'ROST']
  - redzone_hash: ['Season', 'Week', 'FantasyPros_ID', 'REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz']


In [69]:
# Enhanced data integrity checks - view a sample of the hashes
# output: preview of row hashes in the merged dataframe

# Apply hashing for each hash type using the existing generate_md5_hash function
for hash_name, columns in hash_column_groups.items():
    rb_adv_fpts_rost_rz_merged_df_sorted_hash[hash_name] = rb_adv_fpts_rost_rz_merged_df_sorted_hash[columns].apply(generate_md5_hash, axis=1)

# Confirm hashes were generated
print("\n✅ Hashes Generated for All Hash Types.")

# Display the first 5 rows of the hash columns for verification
display(rb_adv_fpts_rost_rz_merged_df_sorted_hash[['adv_stats_hash', 'fpts_rost_hash', 'redzone_hash']].head())



✅ Hashes Generated for All Hash Types.


,adv_stats_hash,fpts_rost_hash,redzone_hash
3,9f24799c1ceeb9884030eee8fd5ed209,e8e23d41df64472e7a4701605318f03e,ec3d7cf6f6deb6445caf4dddbb575d33
7,5735adda7d8ea82464bfd85d614ec8bb,bd8b80677154b7623e50d9edcada4852,fc6b7edb4a996eddf28d0331caa6564a
6,4a03cd5f89587bd6fa556b8c944adc28,b470779404744c23b12e7151af3f181e,c12203d97bf0efe2917173cb5d8eede6
1,36fa0d642121fbcfd5793e50f7f09364,6ffd6f273c0b21f301f02ea2c02c0855,71b8b9b55c4026ce0af1329c3071c506
5,df07f5564400a2648ef166f2684ddfbb,6046fa15635393f6613629d085ad28a2,b8c3498c4fafacb6eb4617bb18fc19fe


In [70]:
# ** Final fantasypros merged dataframe **

# Enhanced data integrity checks - view a sample of the hashes
# output: preview of row hashes in the merged dataframe

# Display the first 5 rows of the merged dataframe for verification
print("\n🔍 First 5 Rows of the Merged DataFrame:")
display(rb_adv_fpts_rost_rz_merged_df_sorted_hash.head(5))


🔍 First 5 Rows of the Merged DataFrame:


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,ATT,YDS,YBCON,YACON,YACON,BRKTKL,TK LOSS,TK LOSS YDS,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,REC,TGT,RZ TGT,YACON,YACON,Y/A,FPTS,ROST,Y/A_rz,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz,row_hash,adv_stats_hash,fpts_rost_hash,redzone_hash
3,2017,1,16450,Tarik Cohen,Tarik Cohen (FA),1,5,66,6,60,20,2,1,-3,2,1,1,1,0,8,12,1,60,20,13.2,17.3,nan,5.0,1.0,1.0,nan,19.0,nan,466140d08ca781cdcbfa5de13a1d3899,9f24799c1ceeb9884030eee8fd5ed209,e8e23d41df64472e7a4701605318f03e,ec3d7cf6f6deb6445caf4dddbb575d33
7,2017,1,9317,Jonathan Stewart,Jonathan Stewart (FA),1,18,65,18,47,11,1,0,0,1,0,0,0,0,2,2,1,47,11,3.6,14.2,nan,1.5,1.0,1.0,nan,9.0,nan,8da58948d83fb752104c2e1d1eddf862,5735adda7d8ea82464bfd85d614ec8bb,bd8b80677154b7623e50d9edcada4852,fc6b7edb4a996eddf28d0331caa6564a
6,2017,1,13977,Ty Montgomery II,Ty Montgomery II (FA),1,19,54,15,39,25,0,1,-1,0,0,0,0,0,4,4,0,39,25,2.8,15.3,nan,6.0,0.0,0.0,nan,0.0,nan,d624afa39452680f7bb78740e5c900bb,4a03cd5f89587bd6fa556b8c944adc28,b470779404744c23b12e7151af3f181e,c12203d97bf0efe2917173cb5d8eede6
1,2017,1,11657,Mike Gillislee,Mike Gillislee (FA),1,15,45,29,16,0,0,1,-4,1,0,0,0,0,0,0,0,16,0,3.0,22.5,nan,2.5,0.0,0.0,nan,0.0,nan,71d99a51e700a043a8f4636db4394ad6,36fa0d642121fbcfd5793e50f7f09364,6ffd6f273c0b21f301f02ea2c02c0855,71b8b9b55c4026ce0af1329c3071c506
5,2017,1,13912,Todd Gurley II,Todd Gurley II (FA),1,19,40,7,33,41,2,3,-13,1,0,0,0,0,5,6,0,33,41,2.1,15.6,nan,3.0,0.0,0.0,nan,0.0,nan,8e468ddbccb28eab0baedec9d2b4ecc7,df07f5564400a2648ef166f2684ddfbb,6046fa15635393f6613629d085ad28a2,b8c3498c4fafacb6eb4617bb18fc19fe


In [71]:
# Enhanced data integrity checks - Generate Row Hashes for Advanced Stats DataFrame
# output: row hashe for the Advanced Stats DataFrame
# Define columns used for generating 'adv_stats_hash'
adv_stats_columns = [
    'Season', 'Week', 'FantasyPros_ID', 'REC', 'YDS', 'YBCON', 'YACON',
    'TGT', 'BRKTKL', 'RZ TGT', '10+ YDS', '20+ YDS', 
    '30+ YDS', '40+ YDS', '50+ YDS'
]


# Ensure all columns are present in the dataframe
adv_stats_columns = [col for col in adv_stats_columns if col in dataframes["Advanced Stats DataFrame"].columns]

# Generate hashes for the Advanced Stats DataFrame
print("\n📊 Generating Hashes for: Advanced Stats DataFrame")
dataframes["Advanced Stats DataFrame"]["row_hash"] = dataframes["Advanced Stats DataFrame"][adv_stats_columns].apply(generate_md5_hash, axis=1)

# Display a preview to confirm hashes were generated
print("Preview of Generated Hashes in Advanced Stats DataFrame:")
display(dataframes["Advanced Stats DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'row_hash']].head())



📊 Generating Hashes for: Advanced Stats DataFrame
Preview of Generated Hashes in Advanced Stats DataFrame:


,Season,Week,FantasyPros_ID,row_hash
0,2017,1,16425,c257188e32720154ae5d19dcbea49ff8
1,2017,1,11657,36fa0d642121fbcfd5793e50f7f09364
2,2017,1,16378,ac0ba8b03209b5d2d707e848779229f0
3,2017,1,16450,9f24799c1ceeb9884030eee8fd5ed209
4,2017,1,9398,26036b15a3e58d2ff1ae7dd009c8d509


In [72]:
# Enhanced data integrity checks - Prepare DataFrames for Comparison
# output:  no output

# Select relevant columns from the Advanced Stats DataFrame
adv_stats_subset = dataframes["Advanced Stats DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'row_hash']]

# Prepare the Merged DataFrame subset for comparison
merged_subset = rb_adv_fpts_rost_rz_merged_df_sorted_hash[['Season', 'Week', 'FantasyPros_ID', 'adv_stats_hash']]

In [73]:
# Enhanced data integrity checks - Create a merge dataFrame for hash comparison
# output: a merged dataframe
# output: df contains the advanced stats column (merged df) and the row hash (advanced stats df) 

# Merge the dataframes on the keys - JOIN ON FantasyPros_ID
comparison_df = pd.merge(
    merged_subset,
    adv_stats_subset,
    on=['Season', 'Week', 'FantasyPros_ID'],
    how='inner',
    suffixes=('_merged', '_original')
)

# Display the merged comparison dataframe
print("Comparison: adv_stats_hash column (rb_adv_fpts_rost_rz_merged_df_sorted_hash)")
print("Comparison: row_hash column (Advanced Stats DataFrame)\n")
display(comparison_df.head())

Comparison: adv_stats_hash column (rb_adv_fpts_rost_rz_merged_df_sorted_hash)
Comparison: row_hash column (Advanced Stats DataFrame)



,Season,Week,FantasyPros_ID,adv_stats_hash,row_hash
0,2017,1,16450,9f24799c1ceeb9884030eee8fd5ed209,9f24799c1ceeb9884030eee8fd5ed209
1,2017,1,9317,5735adda7d8ea82464bfd85d614ec8bb,5735adda7d8ea82464bfd85d614ec8bb
2,2017,1,13977,4a03cd5f89587bd6fa556b8c944adc28,4a03cd5f89587bd6fa556b8c944adc28
3,2017,1,11657,36fa0d642121fbcfd5793e50f7f09364,36fa0d642121fbcfd5793e50f7f09364
4,2017,1,13912,df07f5564400a2648ef166f2684ddfbb,df07f5564400a2648ef166f2684ddfbb


In [74]:
# Enhanced data integrity checks - Identify Matches and Mismatches
# output: hopefully no mismatches

# Identify mismatches where hashes don't align
mismatches = comparison_df[comparison_df['adv_stats_hash'] != comparison_df['row_hash']]

# Display mismatch results
print(f"\n❌ Mismatches Found: {len(mismatches)}")
if len(mismatches) > 0:
    display(mismatches.head())
else:
    print("✅ All hashes match.")


❌ Mismatches Found: 0
✅ All hashes match.


In [75]:
# Enhanced data integrity checks - Generate Row Hashes for FPTS & ROST DataFrame
# output: Row hashes for FPTS & ROST DataFrame

# Display current dataframe being processed
print("\n📊 Processing: FPTS & ROST DataFrame")

# Generate row hashes for FPTS & ROST DataFrame using the existing helper function
dataframes["FPTS & ROST DataFrame"]["row_hash"] = dataframes["FPTS & ROST DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'FPTS', 'ROST']].apply(generate_md5_hash, axis=1)

# Display a preview of the generated hashes
display(dataframes["FPTS & ROST DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'row_hash']].head())


📊 Processing: FPTS & ROST DataFrame


,Season,Week,FantasyPros_ID,row_hash
0,2017,1,16425,5d56b4c5bb987b4ffdd9c458dfb0f238
1,2017,1,11657,6ffd6f273c0b21f301f02ea2c02c0855
2,2017,1,16378,f5232461599d9ac5996eb38dba02a0d1
3,2017,1,16450,e8e23d41df64472e7a4701605318f03e
4,2017,1,9398,4352055ac98863ef51ecd9151597f357


In [76]:
# Enhanced data integrity checks - Check Matching IDs Before Comparison
# output: 

# Checking Matching IDs Between the DataFrames
print("\n📊 Checking Matching IDs Before Comparison:")

# Extract IDs from both dataframes
ids_unmerged = dataframes["FPTS & ROST DataFrame"]["FantasyPros_ID"].unique()
ids_merged = rb_adv_fpts_rost_rz_merged_df_sorted_hash["FantasyPros_ID"].unique()

# Find intersection of IDs
matching_ids = set(ids_unmerged).intersection(set(ids_merged))
print(f"🔍 Total Matching IDs Found: {len(matching_ids)}")


📊 Checking Matching IDs Before Comparison:
🔍 Total Matching IDs Found: 158


In [77]:
# Enhanced data integrity checks - Compare Hashes 
# output: Hopefully no mismatches
# Perform comparison by joining dataframes on keys: Season, Week, FantasyPros_ID
comparison_df = pd.merge(
    rb_adv_fpts_rost_rz_merged_df_sorted_hash[['Season', 'Week', 'FantasyPros_ID', 'fpts_rost_hash']],
    dataframes["FPTS & ROST DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'row_hash']],
    on=['Season', 'Week', 'FantasyPros_ID'],
    how='inner',
    suffixes=('_merged', '_unmerged')
)

# Identify mismatches
mismatches = comparison_df[comparison_df['fpts_rost_hash'] != comparison_df['row_hash']]

# Display comparison results
if mismatches.empty:
    print("\n✅ All hashes match for FPTS & ROST DataFrame.")
else:
    print(f"\n❌ Mismatches Found: {len(mismatches)}")
    display(mismatches.head())
display(comparison_df.head())


✅ All hashes match for FPTS & ROST DataFrame.


,Season,Week,FantasyPros_ID,fpts_rost_hash,row_hash
0,2017,1,16450,e8e23d41df64472e7a4701605318f03e,e8e23d41df64472e7a4701605318f03e
1,2017,1,9317,bd8b80677154b7623e50d9edcada4852,bd8b80677154b7623e50d9edcada4852
2,2017,1,13977,b470779404744c23b12e7151af3f181e,b470779404744c23b12e7151af3f181e
3,2017,1,11657,6ffd6f273c0b21f301f02ea2c02c0855,6ffd6f273c0b21f301f02ea2c02c0855
4,2017,1,13912,6046fa15635393f6613629d085ad28a2,6046fa15635393f6613629d085ad28a2


In [78]:
## Fixme: **No row hash validation for the redzone data**  
# Enhanced data integrity checks - Compare Hashes for Redzone Data
# output: Display mismatches if found

# Perform comparison by joining dataframes on keys: Season, Week, FantasyPros_ID
comparison_df_redzone = pd.merge(
    rb_adv_fpts_rost_rz_merged_df_sorted_hash[['Season', 'Week', 'FantasyPros_ID', 'redzone_hash']],
    dataframes["Redzone Stats DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'row_hash']],
    on=['Season', 'Week', 'FantasyPros_ID'],
    how='inner',
    suffixes=('_merged', '_unmerged')
)

# Identify mismatches
mismatches_redzone = comparison_df_redzone[comparison_df_redzone['redzone_hash'] != comparison_df_redzone['row_hash']]

# Display comparison results
if mismatches_redzone.empty:
    print("\n✅ All hashes match for Redzone Stats DataFrame.")
else:
    print(f"\n❌ Mismatches Found: {len(mismatches_redzone)}")
    display(mismatches_redzone.head())

# Display the first few rows of the comparison dataframe for reference
display(comparison_df_redzone.head())


❌ Mismatches Found: 1083


,Season,Week,FantasyPros_ID,redzone_hash,row_hash
0,2017,1,16450,ec3d7cf6f6deb6445caf4dddbb575d33,fe62692eeef2c1cf5b7708b7dadb8f63
1,2017,1,9317,fc6b7edb4a996eddf28d0331caa6564a,d8b673e50883aacebb11e27d4a06b0d9
2,2017,1,13977,c12203d97bf0efe2917173cb5d8eede6,7544496bd99ecda8fc250408083f08b7
3,2017,1,11657,71b8b9b55c4026ce0af1329c3071c506,b246359aa520e88f1f3715813153b51f
4,2017,1,13912,b8c3498c4fafacb6eb4617bb18fc19fe,88a3ab623a6c4fc6cf33af0e2f2838d9


,Season,Week,FantasyPros_ID,redzone_hash,row_hash
0,2017,1,16450,ec3d7cf6f6deb6445caf4dddbb575d33,fe62692eeef2c1cf5b7708b7dadb8f63
1,2017,1,9317,fc6b7edb4a996eddf28d0331caa6564a,d8b673e50883aacebb11e27d4a06b0d9
2,2017,1,13977,c12203d97bf0efe2917173cb5d8eede6,7544496bd99ecda8fc250408083f08b7
3,2017,1,11657,71b8b9b55c4026ce0af1329c3071c506,b246359aa520e88f1f3715813153b51f
4,2017,1,13912,b8c3498c4fafacb6eb4617bb18fc19fe,88a3ab623a6c4fc6cf33af0e2f2838d9


In [79]:
### End: Enhanced data integrity checks for the fantasypros dataframes ##

In [80]:
## Begin: webscraping sportsoddshistory betting lines data ###

In [81]:
# Generate Dynamic URLs for the regular season 
# output: urls with each year to scrape

# Helper: Define year range
def get_year_range(current_year, current_week):
    return list(range(2017, current_year + 1)) if current_week >= 1 else list(range(2017, current_year))

# Current time context
current_year = datetime.now().year
current_week = datetime.now().isocalendar()[1]

# Get the range and build the season URLs
season_list = get_year_range(current_year, current_week)
season_urls = [f"https://www.sportsoddshistory.com/nfl-game-season/?y={year}" for year in season_list]

# Preview
for season, url in zip(season_list, season_urls):
    print(f"{season}: {url}")

2017: https://www.sportsoddshistory.com/nfl-game-season/?y=2017
2018: https://www.sportsoddshistory.com/nfl-game-season/?y=2018
2019: https://www.sportsoddshistory.com/nfl-game-season/?y=2019
2020: https://www.sportsoddshistory.com/nfl-game-season/?y=2020
2021: https://www.sportsoddshistory.com/nfl-game-season/?y=2021
2022: https://www.sportsoddshistory.com/nfl-game-season/?y=2022
2023: https://www.sportsoddshistory.com/nfl-game-season/?y=2023
2024: https://www.sportsoddshistory.com/nfl-game-season/?y=2024
2025: https://www.sportsoddshistory.com/nfl-game-season/?y=2025


In [82]:
# Load Soup and Extract Regular Season Week Headers
# output: regular season years

# Container for (season, soup, regular season week headers)
tm_game_odds_season_soup_headers = []

for season, url in zip(season_list, season_urls):
    print(f"🔄 Fetching {season}...")
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract all <h3> tags
    all_headers = soup.find_all('h3')

    # Only keep regular season week headers (e.g., "2023 Regular Season - Week 1")
    reg_week_headers = [tag for tag in all_headers if f"{season} Regular Season - Week" in tag.text]

    # Store tuple for next stage
    tm_game_odds_season_soup_headers.append((season, soup, reg_week_headers))

🔄 Fetching 2017...
🔄 Fetching 2018...
🔄 Fetching 2019...
🔄 Fetching 2020...
🔄 Fetching 2021...
🔄 Fetching 2022...
🔄 Fetching 2023...
🔄 Fetching 2024...
🔄 Fetching 2025...


In [83]:
# Scrape All Weeks for Each Year
# output: none

# Container to collect team-level weekly DataFrames
tm_game_odds_regular_team_level_list = []

for season, soup, week_headers in tm_game_odds_season_soup_headers:
    for tag in week_headers:
        week = int(tag.text.strip().split("Week")[1])

        table = tag.find_next('table')
        if not table:
            continue
        rows = table.find('tbody').find_all('tr')

        base_data = []
        abbr_data = []

        for row in rows:
            cells = row.find_all('td')
            if len(cells) != 11:
                continue

            base_data.append([cell.get_text(strip=True) for cell in cells])

            fav_link = cells[4].find('a')
            und_link = cells[8].find('a')

            fav_abbr = fav_link['href'].split('=')[1].split('&')[0] if fav_link else None
            und_abbr = und_link['href'].split('=')[1].split('&')[0] if und_link else None

            abbr_data.append([fav_abbr, und_abbr])

        if not base_data:
            continue

        df = pd.DataFrame(base_data, columns=[
            'Day', 'Date', 'Time (ET)', 'Home_Away', 'Favorite', 'Score', 'Spread', '',
            'Underdog', 'Over/Under', 'Notes'
        ])
        df.drop(columns=[''], inplace=True)

        abbr_df = pd.DataFrame(abbr_data, columns=['Favorite_Abbr', 'Underdog_Abbr'])
        df = pd.concat([df, abbr_df], axis=1)
        df['season'] = season
        df['week'] = week

        # Expand to team-level format
        team_rows = []
        for _, row in df.iterrows():
            home_away = row['Home_Away']
            score = row['Score']

            team_rows.extend([
                {
                    'season': row['season'], 'week': row['week'],
                    'team_abbr': row['Favorite_Abbr'],
                    'opponent_abbr': row['Underdog_Abbr'],
                    'home': home_away != '@', 'role': 'Favorite',
                    'result': score.split()[0] if score else None,
                    'score': score.split()[1] if score else None,
                    'spread': row['Spread'], 'over_under': row['Over/Under']
                },
                {
                    'season': row['season'], 'week': row['week'],
                    'team_abbr': row['Underdog_Abbr'],
                    'opponent_abbr': row['Favorite_Abbr'],
                    'home': home_away == '@', 'role': 'Underdog',
                    'result': None, 'score': None,
                    'spread': row['Spread'], 'over_under': row['Over/Under']
                }
            ])

        weekly_df = pd.DataFrame(team_rows)

        # Clean the spread column but DO NOT normalize abbreviations yet
        weekly_df['spread'] = weekly_df['spread'].str.replace(r'^P.*', '0', regex=True)
        weekly_df['spread'] = weekly_df['spread'].str.replace(r'^[WL]\s*', '', regex=True)
        weekly_df['spread'] = pd.to_numeric(weekly_df['spread'].str.strip(), errors='coerce')

        tm_game_odds_regular_team_level_list.append(weekly_df)

In [84]:
# Combine and Export the Regular Season Data
# output: csv file of regular season odds lines

# Combine all regular season data
tm_game_odds_regular_team_level = pd.concat(
    tm_game_odds_regular_team_level_list,
    ignore_index=True
)

# Preview dimensions
print(f"📊 Total rows collected: {tm_game_odds_regular_team_level.shape[0]}")
print(f"✅ Columns: {list(tm_game_odds_regular_team_level.columns)}")

# Export for inspection
# tm_game_odds_regular_team_level.to_csv("tm_game_odds_regular_season.csv", index=False)
print("📝 Exported to tm_game_odds_regular_season.csv")

📊 Total rows collected: 4222
✅ Columns: ['season', 'week', 'team_abbr', 'opponent_abbr', 'home', 'role', 'result', 'score', 'spread', 'over_under']
📝 Exported to tm_game_odds_regular_season.csv


In [85]:
# extract playoff <h3> tags from each season and pair them with the corresponding soup object
# output: a list of tuples

# Identify playoff <h3> tags for each season 
playoff_h3_tags = []

for season, url in zip(season_list, season_urls):
    print(f"🔄 Checking {season} playoffs...")

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    tag = soup.find('h3', string=f"{season} Playoffs")
    
    if tag:
        playoff_h3_tags.append((season, soup, tag))
    else:
        print(f"⚠️ No playoff tag found for {season}")

🔄 Checking 2017 playoffs...
🔄 Checking 2018 playoffs...
🔄 Checking 2019 playoffs...
🔄 Checking 2020 playoffs...
🔄 Checking 2021 playoffs...
🔄 Checking 2022 playoffs...
🔄 Checking 2023 playoffs...
🔄 Checking 2024 playoffs...
🔄 Checking 2025 playoffs...
⚠️ No playoff tag found for 2025


In [86]:
# extract all <tr> rows from the playoff table corresponding to each <h3> tag
# output: a list of tuples: (season, [row1, row2, ..., rowN])

# Extract playoff table rows from each <h3> tag
playoff_table_rows = []

for season, soup, tag in playoff_h3_tags:
    print(f"📅 Extracting table rows for {season} playoffs...")
    
    try:
        table = tag.find_next('table')
        rows = table.find('tbody').find_all('tr')
        playoff_table_rows.append((season, rows))
    except Exception as e:
        print(f"⚠️ {season} - Failed to extract table rows: {e}")

📅 Extracting table rows for 2017 playoffs...
📅 Extracting table rows for 2018 playoffs...
📅 Extracting table rows for 2019 playoffs...
📅 Extracting table rows for 2020 playoffs...
📅 Extracting table rows for 2021 playoffs...
📅 Extracting table rows for 2022 playoffs...
📅 Extracting table rows for 2023 playoffs...
📅 Extracting table rows for 2024 playoffs...


In [87]:
# Parse each playoff row into game-level dictionaries
# output: dataframe of the playoff games

# Parse <tr> rows into dictionaries with playoff data
def parse_playoff_row(row):
    try:
        cells = row.find_all('td')
        if len(cells) < 11:
            return None  # Skip malformed rows

        # Remove seed info from names (e.g., "BUF (3)" → "BUF")
        favorite_raw = cells[5].get_text(strip=True).rsplit('(', 1)[0].strip()
        underdog_raw = cells[9].get_text(strip=True).rsplit('(', 1)[0].strip()

        return {
            'Round': cells[0].get_text(strip=True),
            'Day': cells[1].get_text(strip=True),
            'Date': cells[2].get_text(strip=True),
            'Time (ET)': cells[3].get_text(strip=True),
            'Home_Away': cells[4].get_text(strip=True),
            'Favorite': favorite_raw,
            'Score': cells[6].get_text(strip=True),
            'Spread': cells[7].get_text(strip=True),
            'Underdog': underdog_raw,
            'Over/Under': cells[10].get_text(strip=True)
        }

    except Exception as e:
        print(f"⚠️ Error parsing row: {e}")
        return None

# Apply the parser
parsed_playoff_games = []

for season, rows in playoff_table_rows:
    parsed = [parse_playoff_row(row) for row in rows]
    parsed = [game for game in parsed if game is not None]
    for game in parsed:
        game['season'] = season  # Tag season
    parsed_playoff_games.extend(parsed)

# Preview the first few
pd.DataFrame(parsed_playoff_games).head()

,Round,Day,Date,Time (ET),Home_Away,Favorite,Score,Spread,Underdog,Over/Under,season
0,AFC Wild Card,Sat,"Jan 6, 2018",4:35,@,Kansas City Chiefs,L 21-22,L -8.5,Tennessee Titans,U 44,2017
1,NFC Wild Card,Sat,"Jan 6, 2018",8:15,@,Los Angeles Rams,L 13-26,L -6.5,Atlanta Falcons,U 48,2017
2,AFC Wild Card,Sun,"Jan 7, 2018",1:05,@,Jacksonville Jaguars,W 10-3,L -8,Buffalo Bills,U 39.5,2017
3,NFC Wild Card,Sun,"Jan 7, 2018",4:40,@,New Orleans Saints,W 31-26,L -6.5,Carolina Panthers,O 47.5,2017
4,NFC Divisional,Sat,"Jan 13, 2018",4:35,,Atlanta Falcons,L 10-15,L -2.5,Philadelphia Eagles,U 40,2017


In [88]:
# Convert parsed playoff games into team-level rows
# output: dataframe of the playoff games with numbered weeks

# Helper to map playoff round name → week offset
def get_playoff_week(round_name, season):
    base_week = 19 if season >= 2021 else 18
    round_name = round_name.lower()
    if "wild card" in round_name:
        return base_week
    elif "divisional" in round_name:
        return base_week + 1
    elif "championship" in round_name:
        return base_week + 2
    elif "super bowl" in round_name:
        return base_week + 3
    return None  # fallback

# Convert each game into two team rows
team_level_rows = []

for game in parsed_playoff_games:
    week = get_playoff_week(game['Round'], game['season'])

    team_level_rows.append({
        'season': game['season'],
        'week': week,
        'team_abbr': game['Favorite'],
        'opponent_abbr': game['Underdog'],
        'home': game['Home_Away'] != '@',
        'role': 'Favorite',
        'result': game['Score'].split()[0] if game['Score'] else None,
        'score': game['Score'].split()[1] if game['Score'] else None,
        'spread': game['Spread'],
        'over_under': game['Over/Under']
    })

    team_level_rows.append({
        'season': game['season'],
        'week': week,
        'team_abbr': game['Underdog'],
        'opponent_abbr': game['Favorite'],
        'home': game['Home_Away'] == '@',
        'role': 'Underdog',
        'result': None,
        'score': None,
        'spread': game['Spread'],
        'over_under': game['Over/Under']
    })

# Build team-level playoff DataFrame
tm_game_odds_playoffs_team_level = pd.DataFrame(team_level_rows)

# Clean 'spread' values
tm_game_odds_playoffs_team_level['spread'] = (
    tm_game_odds_playoffs_team_level['spread']
    .str.replace(r'^P.*', '0', regex=True)
    .str.replace(r'^[WL]\s*', '', regex=True)
    .str.strip()
)
tm_game_odds_playoffs_team_level['spread'] = pd.to_numeric(
    tm_game_odds_playoffs_team_level['spread'], errors='coerce'
)

# Preview
display(tm_game_odds_playoffs_team_level.head())


,season,week,team_abbr,opponent_abbr,home,role,result,score,spread,over_under
0,2017,18,Kansas City Chiefs,Tennessee Titans,False,Favorite,L,21-22,-8.5,U 44
1,2017,18,Tennessee Titans,Kansas City Chiefs,True,Underdog,None,None,-8.5,U 44
2,2017,18,Los Angeles Rams,Atlanta Falcons,False,Favorite,L,13-26,-6.5,U 48
3,2017,18,Atlanta Falcons,Los Angeles Rams,True,Underdog,None,None,-6.5,U 48
4,2017,18,Jacksonville Jaguars,Buffalo Bills,False,Favorite,W,10-3,-8.0,U 39.5


In [89]:
# Merge Regular Season + Playoffs into One DataFrame
# output: combined dataframe of regular season and playoff odds and lines *(not final)
# output: csv file for inspection only *not final

# Combine regular season and playoff team-level rows
tm_game_odds_all_games = pd.concat(
    [tm_game_odds_regular_team_level, tm_game_odds_playoffs_team_level],
    ignore_index=True
)

# Optional: sort for easier inspection
tm_game_odds_all_games.sort_values(by=['season', 'week'], inplace=True)

# Export final combined dataset
# tm_game_odds_all_games.to_csv("tm_game_odds_all_games.csv", index=False)

# Preview
print("✅ Combined all regular season and playoff games.")
print(f"📊 Final shape: {tm_game_odds_all_games.shape}")
display(tm_game_odds_all_games.head())

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.
✅ Combined all regular season and playoff games.
📊 Final shape: (4418, 10)


,season,week,team_abbr,opponent_abbr,home,role,result,score,spread,over_under
0,2017,1,NEP,KCC,False,Favorite,L,27-42,-8.0,O 47.5
1,2017,1,KCC,NEP,True,Underdog,None,None,-8.0,O 47.5
2,2017,1,BUF,NYJ,False,Favorite,W,21-12,-7.0,U 42
3,2017,1,NYJ,BUF,True,Underdog,None,None,-7.0,U 42
4,2017,1,ATL,CHI,True,Favorite,W,23-17,-6.5,U 48


In [90]:
# standardize team abbreviations
# output: none

# Create One Unified Map
# Canonical full team name → abbreviation
full_name_to_abbr = {
    'Arizona Cardinals': 'ARI', 'Atlanta Falcons': 'ATL', 'Baltimore Ravens': 'BAL',
    'Buffalo Bills': 'BUF', 'Carolina Panthers': 'CAR', 'Chicago Bears': 'CHI',
    'Cincinnati Bengals': 'CIN', 'Cleveland Browns': 'CLE', 'Dallas Cowboys': 'DAL',
    'Denver Broncos': 'DEN', 'Detroit Lions': 'DET', 'Green Bay Packers': 'GB',
    'Houston Texans': 'HOU', 'Indianapolis Colts': 'IND', 'Jacksonville Jaguars': 'JAX',
    'Kansas City Chiefs': 'KC', 'Las Vegas Raiders': 'LV', 'Los Angeles Chargers': 'LAC',
    'Los Angeles Rams': 'LA', 'Miami Dolphins': 'MIA', 'Minnesota Vikings': 'MIN',
    'New England Patriots': 'NE', 'New Orleans Saints': 'NO', 'New York Giants': 'NYG',
    'New York Jets': 'NYJ', 'Philadelphia Eagles': 'PHI', 'Pittsburgh Steelers': 'PIT',
    'San Francisco 49ers': 'SF', 'Seattle Seahawks': 'SEA', 'Tampa Bay Buccaneers': 'TB',
    'Tennessee Titans': 'TEN', 'Washington Commanders': 'WAS', 'Washington Football Team': 'WAS'

}

# Abbreviation fix-ups
abbr_fix_map = {
    'ARZ': 'ARI', 'TBB': 'TB', 'NEP': 'NE', 'GBP': 'GB', 'KCC': 'KC',
    'SFF': 'SF', 'LAR': 'LA', 'NOS': 'NO', 'JAC': 'JAX', 'LVR': 'LV'
}


In [91]:
# apply the correct team abbreviations and rename the dataframe
# output: no output

# Rename to final dataframe name for clarity
tm_reg_playoffs_game_lines_odds = tm_game_odds_all_games.copy()

# Apply full name and abbreviation corrections
tm_reg_playoffs_game_lines_odds['team_abbr'] = (
    tm_reg_playoffs_game_lines_odds['team_abbr']
    .replace(full_name_to_abbr)
    .replace(abbr_fix_map)
)

tm_reg_playoffs_game_lines_odds['opponent_abbr'] = (
    tm_reg_playoffs_game_lines_odds['opponent_abbr']
    .replace(full_name_to_abbr)
    .replace(abbr_fix_map)
)

In [92]:
# Compare the team abbreviations: python nfl df to the tm odds df - the sets should match!
# output: list of team abbreviaion  
# Canonical set from the base dataset
baseline_nfl_tm_abbr = set(rb_ids_ngs_pfr_stats_sorted['recent_team'].dropna().unique())

# Abbreviation sets from the cleaned odds dataframe
home_odds_lines_abbr = set(tm_reg_playoffs_game_lines_odds['team_abbr'].dropna().unique())
away_odds_lines_abbr = set(tm_reg_playoffs_game_lines_odds['opponent_abbr'].dropna().unique())

# Combined odds abbreviations
odds_abbrs = home_odds_lines_abbr.union(away_odds_lines_abbr)

# Comparison
only_in_rb_ids = baseline_nfl_tm_abbr - odds_abbrs
only_in_odds = odds_abbrs - baseline_nfl_tm_abbr

# Output
print("✅ Unique team_abbr values (home):", sorted(home_odds_lines_abbr))
print()

print("✅ Unique opponent_abbr values (away):", sorted(away_odds_lines_abbr))
print()

print("❌ Abbreviations only in rb_ids_ngs_pfr_stats_sorted:", only_in_rb_ids)
print("❌ Abbreviations only in tm_reg_playoffs_game_lines_odds:", only_in_odds)

✅ Unique team_abbr values (home): ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS']

✅ Unique opponent_abbr values (away): ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS']

❌ Abbreviations only in rb_ids_ngs_pfr_stats_sorted: set()
❌ Abbreviations only in tm_reg_playoffs_game_lines_odds: set()


In [93]:
# **** Final Dataframe of team odds ***

# output: final dataframe of team odds and lines with the csv file
# Save the file
# tm_reg_playoffs_game_lines_odds.to_csv("tm_reg_playoffs_game_lines_odds.csv", index=False)

# Display the first 10 rows
display(tm_reg_playoffs_game_lines_odds.head(10))

,season,week,team_abbr,opponent_abbr,home,role,result,score,spread,over_under
0,2017,1,NE,KC,False,Favorite,L,27-42,-8.0,O 47.5
1,2017,1,KC,NE,True,Underdog,None,None,-8.0,O 47.5
2,2017,1,BUF,NYJ,False,Favorite,W,21-12,-7.0,U 42
3,2017,1,NYJ,BUF,True,Underdog,None,None,-7.0,U 42
4,2017,1,ATL,CHI,True,Favorite,W,23-17,-6.5,U 48
5,2017,1,CHI,ATL,False,Underdog,None,None,-6.5,U 48
6,2017,1,CIN,BAL,False,Favorite,L,0-20,-2.5,U 41.5
7,2017,1,BAL,CIN,True,Underdog,None,None,-2.5,U 41.5
8,2017,1,PIT,CLE,True,Favorite,W,21-18,-10.0,U 47
9,2017,1,CLE,PIT,False,Underdog,None,None,-10.0,U 47


In [94]:
## End: webscraping sportsoddshistory betting lines data ###

In [95]:
## Begin: Build the dataframe for the DFS Fanduel and Draft Kings salary data from BigDataBall ##
# ** Files must be in the local directory ** NFL-20xx-DFS-Dataset.xlsx

In [96]:
# create a helper function to clean the the dfs salary data
# no output

def clean_column_dfs(col):
    """
    Cleans and flattens multi-index column names for DFS salary Excel files:
    - Joins tuples if multi-index
    - Removes special characters
    - Normalizes spaces
    - Converts to lowercase for matching
    """
    if isinstance(col, tuple):
        col = ' '.join(str(x) for x in col if x)

    return (
        str(col)
        .replace('\n', ' ')
        .replace('(', '')
        .replace(')', '')
        .replace('"', '')
        .replace('#', '')
        .replace('$', '')
        .replace('/', '')
        .replace('-', ' ')
        .strip()
        .lower()
        .replace('  ', ' ')
        .replace('   ', ' ')
    )

In [97]:
# Read the excel files
filepath = 'NFL-2024-DFS-Dataset.xlsx'
dfs_raw = pd.read_excel(filepath, header=[0, 1])
original_row_count = len(dfs_raw)

dfs_raw.columns = [clean_column_dfs(col) for col in dfs_raw.columns]
dfs_raw.head()  # Optional preview

,game information bigdataball dataset,game information game id,game information date,game information week,game information start time et,game information player id,game information player dst,game information team,game information opponent,game information venue rh,position draftkings,position fanduel,salary for draftkings classic contests,salary for fanduel full roster contests,fantasy points scored draftkings,fantasy points scored fanduel
0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#N/A MEANING: The game was not included in any...,NaN,NaN,NaN,NaN,NaN
1,NFL 2024 Regular Season,45540-BAL@KAN,2024-09-05,1.0,8:20 PM,BAL,Baltimore Ravens,Baltimore Ravens,Kansas City Chiefs,Road,DST,D,2700,3900.0,4.0,4.0
2,NFL 2024 Regular Season,45540-BAL@KAN,2024-09-05,1.0,8:20 PM,nelson-agholor,Nelson Agholor,Baltimore Ravens,Kansas City Chiefs,Road,WR,WR,3800,4200.0,1.6,1.1
3,NFL 2024 Regular Season,45540-BAL@KAN,2024-09-05,1.0,8:20 PM,derrick-henry,Derrick Henry,Baltimore Ravens,Kansas City Chiefs,Road,RB,RB,6900,7900.0,10.6,10.6
4,NFL 2024 Regular Season,45540-BAL@KAN,2024-09-05,1.0,8:20 PM,patrick-ricard,Patrick Ricard,Baltimore Ravens,Kansas City Chiefs,Road,RB,RB,4000,4100.0,0.0,0.0


In [98]:
# helper function does the following:
# Fanduel and Draft Kings player salary data for all positions (QB, RB, TE, WR, DST)
# creates and combines the dataframes for years 2017 - present 
# performs data validation checks
# output: none

def process_single_year(filepath, year):
    
    # Step 1: Read and clean the headers
    dfs_raw = pd.read_excel(filepath, header=[0, 1])
    original_row_count = len(dfs_raw)
    dfs_raw.columns = [clean_column_dfs(col) for col in dfs_raw.columns]

    # ✅ Step 2: Extract only relevant columns using cleaned names
    expected_cols = {
        'player': 'game information player dst',
        'week': 'game information week',
        'date': 'game information date',
        'player_id': 'game information player id',
        'dk_position': 'position draftkings',
        'fd_position': 'position fanduel',
        'dk_salary': 'salary for draftkings classic contests',
        'fd_salary': 'salary for fanduel full roster contests'
    }

    # Subset the dataframe using cleaned column names
    dfs_subset = dfs_raw[list(expected_cols.values())].copy()

    # Rename them to simple identifiers for internal use
    dfs_subset.columns = list(expected_cols.keys())

    
    dfs_subset['date'] = pd.to_datetime(dfs_subset['date'])

    team_abbreviation_mapping = {
        'NWE': 'NE',
        'SFO': 'SF',
        'OAK': 'LV',
        'KAN': 'KC',
        'TAM': 'TB',
        'NOR': 'NO',
        'LAR': 'LA',
        'GNB': 'GB'
    }
    mask_dst = dfs_subset['dk_position'] == 'DST'
    dfs_subset.loc[mask_dst, 'player_id'] = dfs_subset.loc[mask_dst, 'player_id'].replace(team_abbreviation_mapping)

    def fix_season(row):
        game_year = row['date'].year
        game_month = row['date'].month
        game_week = row['week']
        
        if game_month in [1, 2]:
            if (game_year <= 2020 and game_week >= 18):
                return game_year - 1
            elif (game_year >= 2021 and game_week >= 19):
                return game_year - 1
            elif (game_year >= 2021 and game_week == 18):
                return game_year - 1
        return game_year

    dfs_subset['season'] = dfs_subset.apply(fix_season, axis=1)

    # 🔥 Track NaNs before dropping
    season_nulls_before = dfs_subset['season'].isna().sum()

    dfs_subset = dfs_subset.dropna(subset=['season'])
    dfs_subset['season'] = dfs_subset['season'].astype(int)

    season_nulls_after = dfs_subset['season'].isna().sum()

    print(f"🔎 Season NaN rows dropped: {season_nulls_before}")
    print(f"Remaining NaN rows (should be 0): {season_nulls_after}")

    dfs_subset = dfs_subset.drop(columns=['date'])

    dfs_subset['dk_salary'] = pd.to_numeric(dfs_subset['dk_salary'], errors='coerce')
    dfs_subset['fd_salary'] = pd.to_numeric(dfs_subset['fd_salary'], errors='coerce')
    dfs_subset = dfs_subset.dropna(subset=['dk_salary', 'fd_salary'])
    dfs_subset['dk_salary'] = dfs_subset['dk_salary'].astype(int)
    dfs_subset['fd_salary'] = dfs_subset['fd_salary'].astype(int)
    dfs_subset['week'] = dfs_subset['week'].astype(int)
    
    dfs_subset = dfs_subset[['season', 'week', 'player_id', 'player', 'dk_position', 'fd_position', 'dk_salary', 'fd_salary']]
    
    unique_weeks = dfs_subset['week'].nunique()
    min_week = dfs_subset['week'].min()
    max_week = dfs_subset['week'].max()
    expected_weeks = 21 if int(year) <= 2020 else 22

    print(f"\nProcessing file: {filepath}")
    print(f"Original rows in xlsx file: {original_row_count}")
    print(f"Number of players with no salary data found in xlsx: {original_row_count - len(dfs_subset)}")
    print(f"Rows in csv file after dropping NaNs: {len(dfs_subset)}")

    if original_row_count - (original_row_count - len(dfs_subset)) == len(dfs_subset):
        print("✅ Salary Validation passed: Counts match after dropping NaNs.")
        salary_validation = 'Passed'
    else:
        print("❌ Salary Validation failed: Counts mismatch!")
        salary_validation = 'Failed'

    print(f"Weeks detected: {min_week} to {max_week}")
    print(f"Total unique weeks found: {unique_weeks}")
    print("🔔 Reminder: Missing final playoff week (e.g., Super Bowl) is normal if no salary data exists.")

    if unique_weeks == expected_weeks or unique_weeks == expected_weeks - 1:
        print(f"✅ Week Validation passed: {unique_weeks} weeks found (expected {expected_weeks}).\n")
        week_validation = 'Passed'
    else:
        print(f"❌ Week Validation failed: {unique_weeks} weeks found, expected {expected_weeks}.\n")
        week_validation = 'Failed'
    
    return dfs_subset, {
        'year': int(year),
        'original_rows': original_row_count,
        'nan_rows': original_row_count - len(dfs_subset),
        'rows_after_drop': len(dfs_subset),
        'min_week': min_week,
        'max_week': max_week,
        'unique_weeks': unique_weeks,
        'expected_weeks': expected_weeks,
        'salary_validation': salary_validation,
        'week_validation': week_validation
    }

In [99]:
# ** dataframe of Fanduel and Draft Kings Salaries FOR all positions ** 

# main control flow implements the helper function 
# output: combined dataframe and csv files of all seasons fanduel draft kings player salary data 
# output: data validation checks

# Find all matching files
file_list = sorted(glob.glob('NFL-*-DFS-Dataset.xlsx'))

# Handle if no files found
if not file_list:
    print("❌ No xlsx files detected.\nPlease download and place the BigDataBall NFL DFS Excel files into the same directory as this Jupyter Notebook file.")
else:
    # Process each file
    all_years_dfs = []
    validation_records = []
    file_years = []

    for file in file_list:
        year = file.split('-')[1]  # Extract year from filename
        file_years.append(int(year))
        
        year_df, validation_info = process_single_year(file, year)

        # ** csv file ***
        # Save per-year CSV
        # year_df.to_csv(f'nfl_fd_dk_salary_{year}.csv', index=False)
        
        # Append to master list
        all_years_dfs.append(year_df)
        validation_records.append(validation_info)

    # Create validation summary DataFrame
    validation_summary_df = pd.DataFrame(validation_records)
    print("\n📋 Validation Summary:")
    display(validation_summary_df)

    # Combine all years into one big dataframe
    nfl_fd_dk_salary_combined = pd.concat(all_years_dfs, ignore_index=True)

    # Determine latest season dynamically
    current_season = max(file_years)

    # Export final combined CSV
    final_filename = f'nfl_fd_dk_salary_2017_{current_season}.csv'

    # If the file already exists, create a backup
    if os.path.exists(final_filename):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = f'nfl_fd_dk_salary_2017_{current_season}_backup_{timestamp}.csv'
        shutil.copy(final_filename, backup_filename)
        print(f"🛡️ Backup created: {backup_filename}")


    # *** csv file ***
    # nfl_fd_dk_salary_combined.to_csv(final_filename, index=False)

    print(f"\n✅ Final combined CSV saved as: {final_filename}")

    # Display a quick preview
    display(nfl_fd_dk_salary_combined.head())

🔎 Season NaN rows dropped: 1
Remaining NaN rows (should be 0): 0

Processing file: NFL-2017-DFS-Dataset.xlsx
Original rows in xlsx file: 7664
Number of players with no salary data found in xlsx: 799
Rows in csv file after dropping NaNs: 6865
✅ Salary Validation passed: Counts match after dropping NaNs.
Weeks detected: 1 to 20
Total unique weeks found: 20
🔔 Reminder: Missing final playoff week (e.g., Super Bowl) is normal if no salary data exists.
✅ Week Validation passed: 20 weeks found (expected 21).

🔎 Season NaN rows dropped: 1
Remaining NaN rows (should be 0): 0

Processing file: NFL-2018-DFS-Dataset.xlsx
Original rows in xlsx file: 7216
Number of players with no salary data found in xlsx: 197
Rows in csv file after dropping NaNs: 7019
✅ Salary Validation passed: Counts match after dropping NaNs.
Weeks detected: 1 to 20
Total unique weeks found: 20
🔔 Reminder: Missing final playoff week (e.g., Super Bowl) is normal if no salary data exists.
✅ Week Validation passed: 20 weeks found 

,year,original_rows,nan_rows,rows_after_drop,min_week,max_week,unique_weeks,expected_weeks,salary_validation,week_validation
0,2017,7664,799,6865,1,20,20,21,Passed,Passed
1,2018,7216,197,7019,1,20,20,21,Passed,Passed
2,2019,7252,268,6984,1,20,20,21,Passed,Passed
3,2020,7572,385,7187,1,20,20,21,Passed,Passed
4,2021,7994,1650,6344,1,21,21,22,Passed,Passed
5,2022,7872,138,7734,1,21,21,22,Passed,Passed
6,2023,7897,65,7832,1,21,21,22,Passed,Passed
7,2024,7912,136,7776,1,21,21,22,Passed,Passed



✅ Final combined CSV saved as: nfl_fd_dk_salary_2017_2024.csv


,season,week,player_id,player,dk_position,fd_position,dk_salary,fd_salary
0,2017,1,KC,Kansas City Chiefs,DST,DST,2300,4300
1,2017,1,NE,New England Patriots,DST,DST,3400,4500
2,2017,1,NaN,Alex Smith,QB,QB,5400,6900
3,2017,1,NaN,Tom Brady,QB,QB,7600,8900
4,2017,1,NaN,Anthony Sherman,RB,RB,3000,4500


In [100]:
# team abbreviations validations check
def validate_team_defenses(dfs_df, year_label, baseline_set):
    dfs_team_defense = dfs_df.loc[dfs_df['dk_position'] == 'DST', 'player_id']
    dfs_nfl_tm_abbr = set(dfs_team_defense.dropna().unique())
    
    difference_dfs = dfs_nfl_tm_abbr - baseline_set
    difference_baseline = baseline_set - dfs_nfl_tm_abbr
    
    print(f"\nValidating Team Defenses for {year_label}:")
    if not difference_dfs and not difference_baseline:
        print(f"✅ Team defenses match for {year_label}")
    else:
        print(f"❌ Team mismatch detected for {year_label}")
        print(f"Teams only in DFS: {difference_dfs}")
        print(f"Teams only in Baseline: {difference_baseline}")

# Validate each yearly dataframe
for df, info in zip(all_years_dfs, validation_records):
    validate_team_defenses(df, year_label=info['year'], baseline_set=baseline_nfl_tm_abbr)

# Validate the full combined dataframe
validate_team_defenses(nfl_fd_dk_salary_combined, year_label='Combined', baseline_set=baseline_nfl_tm_abbr)


Validating Team Defenses for 2017:
✅ Team defenses match for 2017

Validating Team Defenses for 2018:
✅ Team defenses match for 2018

Validating Team Defenses for 2019:
✅ Team defenses match for 2019

Validating Team Defenses for 2020:
✅ Team defenses match for 2020

Validating Team Defenses for 2021:
✅ Team defenses match for 2021

Validating Team Defenses for 2022:
✅ Team defenses match for 2022

Validating Team Defenses for 2023:
✅ Team defenses match for 2023

Validating Team Defenses for 2024:
✅ Team defenses match for 2024

Validating Team Defenses for Combined:
✅ Team defenses match for Combined


In [101]:
# ** Final RB dataframe of Fanduel and Draft Kings player Salaries ** 

# output: 

# Determine current season based on available data
current_season = nfl_fd_dk_salary_combined['season'].max()

# Extract RB players where DraftKings position is RB
rb_fd_dk_salary_2017_current_df = nfl_fd_dk_salary_combined.loc[
    nfl_fd_dk_salary_combined['dk_position'] == 'RB'
]

# *** csv file ***
rb_csv_filename = f'rb_fd_dk_salary_2017_{current_season}.csv'
# rb_fd_dk_salary_2017_current_df.to_csv(rb_csv_filename, index=False)

print(f"✅ RB DFS dataframe created and saved as {rb_csv_filename}")

# Optional: Display a quick preview
display(rb_fd_dk_salary_2017_current_df.head())

✅ RB DFS dataframe created and saved as rb_fd_dk_salary_2017_2024.csv


,season,week,player_id,player,dk_position,fd_position,dk_salary,fd_salary
4,2017,1,NaN,Anthony Sherman,RB,RB,3000,4500
5,2017,1,NaN,Charcandrick West,RB,RB,4000,5000
6,2017,1,NaN,Kareem Hunt,RB,RB,5800,6400
7,2017,1,NaN,Brandon Bolden,RB,RB,3000,4500
8,2017,1,NaN,Dion Lewis,RB,RB,3800,5400


In [102]:
## End: Build the dataframe for the DFS Fanduel and Draft Kings salary data from BigDataBall ##

In [103]:
### Begin: Data Normalization and Merge Process ###

In [104]:
## Dataframes ##
# rb_ids_ngs_pfr_stats_sorted
# rb_adv_fpts_rost_rz_merged_df_sorted_hash (not going to use this one)
# tm_reg_playoffs_game_lines_odds
# rb_fd_dk_salary_2017_current_df


## Apply Alternative for merging fantasypros ##
# rb_fp_advanced_stats_df
# rb_fpts_perct_rost_df
# rb_redzone_stats_df

In [105]:
# view columns of each fantasypros dataframe prior to normalization
# output: columns in each fp dataframe

print('rb_fp_advanced_stats_df')
print(rb_fp_advanced_stats_df.columns.tolist())
print()

print('rb_fpts_perct_rost_df')
print(rb_fpts_perct_rost_df.columns.tolist())
print()

print('rb_redzone_stats_df')
print(rb_redzone_stats_df.columns.tolist())

rb_fp_advanced_stats_df
['Year', 'Week', 'FantasyPros_ID', 'Player', 'Player', 'G', 'ATT', 'YDS', 'YBCON', 'YACON', 'BRKTKL', 'TK LOSS', 'TK LOSS YDS', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'REC', 'TGT', 'RZ TGT', 'YACON']

rb_fpts_perct_rost_df
['FantasyPros_ID', 'Y/A', 'FPTS', 'ROST', 'Year', 'Week']

rb_redzone_stats_df
['FantasyPros_ID', 'Y/A_rz', 'REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz', 'Year', 'Week']


In [106]:
# helper function for normalization methods
def normalize_dataframe(df: pd.DataFrame, type_map: dict = None) -> pd.DataFrame:
    """
    Clean and normalize dataframe:
    - Strip whitespace from object columns
    - Lowercase common ID/name fields
    - Replace special NA tokens with np.nan
    - Cast to types from provided type_map
    """
    df = df.copy()

    # Replace common string-based missing values with np.nan
    df.replace(["N/A", "NA", "-", ""], np.nan, inplace=True)

    # Strip whitespace from string/object columns
    for col in df.select_dtypes(include='object').columns:
        try:
            df[col] = df[col].astype(str).str.strip()
        except Exception as e:
            print(f"⚠️ Could not strip column '{col}': {e}")

    # Lowercase likely ID/name fields (if present)
    for key in ['player', 'player_name', 'player_id', 'fantasypros_id', 'FantasyPros_ID', 'merge_name']:
        if key in df.columns:
            df[key] = df[key].astype(str).str.lower()

    # Apply type conversions as defined in type_map
    if type_map:
        for col, dtype in type_map.items():
            if col in df.columns:
                try:
                    df[col] = df[col].astype(dtype)
                except Exception as e:
                    print(f"⚠️ Warning: could not convert column '{col}' to {dtype}. Reason: {e}")

    return df

In [107]:
def clean_percentage_columns(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    df = df.copy()
    for col in columns:
        if col in df.columns:
            try:
                # Convert to string first (safe for NaNs)
                df[col] = df[col].astype(str).str.replace('%', '', regex=False)
                df[col] = df[col].astype(float)
            except Exception as e:
                print(f"⚠️ Could not clean and convert '{col}': {e}")
    return df

In [108]:
def inspect_dataframe_types(df: pd.DataFrame, name: str = "DataFrame") -> None:
    print(f"📋 Inspecting: {name}")
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    summary = pd.DataFrame({
        'dtype': df.dtypes,
        'na_count': df.isna().sum()
    }).sort_index()
    display(summary)

In [109]:
# normalize data types for merging 
# RB Baseline type map
type_map_rb = {
    'season': 'int32',
    'week': 'int32',
    'player_id': 'string',
    'fantasypros_id': 'string',
}

# FP Advanced Stats type map
type_map_fp_adv = {
    'fantasypros_id': 'string',
    'season': 'int32',
    'week': 'int32',
    'Player': 'string',
    'G': 'int32',
    'ATT': 'int32',
    'YDS': 'float32',
    'YBCON': 'float32',
    'YACON': 'float32',
    'BRKTKL': 'float32',
    'TK LOSS': 'float32',
    'TK LOSS YDS': 'float32',
    '10+ YDS': 'float32',
    '20+ YDS': 'float32',
    '30+ YDS': 'float32',
    '40+ YDS': 'float32',
    '50+ YDS': 'float32',
    'REC': 'float32',
    'TGT': 'float32',
    'RZ TGT': 'float32',
}


# FP Fantasy Points + Rostered type map
type_map_fp_fpts = {
    'fantasypros_id': 'string',
    'season': 'int32',
    'week': 'int32',
    'FPTS': 'float32',
    'ROST': 'float32',
    'Y/A': 'float32',
}

# FP Redzone type map
type_map_fp_rz = {
    'fantasypros_id': 'string',
    'season': 'int32',
    'week': 'int32',
    'REC PCT_rz': 'float32',
    'REC_rz': 'int32',
    'TGT PCT_rz': 'float32',
    'TGT_rz': 'int32',
    'Y/R_rz': 'float32',
    'Y/A_rz': 'float32',
}

In [110]:
# apply the normalization function to the baseline NFL Python stats dataframe
# no output
rb_ids_ngs_pfr_stats_sorted_normalized = normalize_dataframe(
    rb_ids_ngs_pfr_stats_sorted,
    type_map_rb
)

# Fix FP ID formatting
rb_ids_ngs_pfr_stats_sorted_normalized['fantasypros_id'] = (
    rb_ids_ngs_pfr_stats_sorted_normalized['fantasypros_id']
    .astype(str).str.replace(r'\.0$', '', regex=True)
)

In [111]:
# Rename merge keys consistently BEFORE normalization to the fantasypros dataframes
# no output
rb_fp_advanced_stats_df = rb_fp_advanced_stats_df.rename(columns={
    'Year': 'season',
    'week': 'week',
    'FantasyPros_ID': 'fantasypros_id'
})

rb_fpts_perct_rost_df = rb_fpts_perct_rost_df.rename(columns={
    'Year': 'season',
    'week': 'week',
    'FantasyPros_ID': 'fantasypros_id'
})

rb_redzone_stats_df = rb_redzone_stats_df.rename(columns={
    'Year': 'season',
    'week': 'week',
    'FantasyPros_ID': 'fantasypros_id'
})

# remove duplicate 'Player' column
rb_fp_advanced_stats_df = rb_fp_advanced_stats_df.loc[:, ~rb_fp_advanced_stats_df.columns.duplicated()]

# Clean percentages BEFORE normalization 
rb_redzone_stats_df = clean_percentage_columns(
    rb_redzone_stats_df,
    ['REC PCT_rz', 'TGT PCT_rz']
)

rb_fpts_perct_rost_df = clean_percentage_columns(
    rb_fpts_perct_rost_df,
    ['ROST']
)


In [112]:
# Apply the type_map data normzliaztions to the fantasypros dataframes
# no output

rb_fp_advanced_stats_df = normalize_dataframe(
    rb_fp_advanced_stats_df,
    type_map_fp_adv
)

rb_fpts_perct_rost_df = normalize_dataframe(
    rb_fpts_perct_rost_df,
    type_map_fp_fpts
)

rb_redzone_stats_df = normalize_dataframe(
    rb_redzone_stats_df,
    type_map_fp_rz
)

In [113]:
# Standardize all column names to lowercase after normalization
rb_fp_advanced_stats_df.columns = rb_fp_advanced_stats_df.columns.str.lower()
rb_fpts_perct_rost_df.columns = rb_fpts_perct_rost_df.columns.str.lower()
rb_redzone_stats_df.columns = rb_redzone_stats_df.columns.str.lower()

In [114]:
# view columns of each fantasypros dataframes after normalization
# output: columns in each fp dataframe

print('rb_fp_advanced_stats_df')
print(rb_fp_advanced_stats_df.columns.tolist())
print()

print('rb_fpts_perct_rost_df')
print(rb_fpts_perct_rost_df.columns.tolist())
print()

print('rb_redzone_stats_df')
print(rb_redzone_stats_df.columns.tolist())

rb_fp_advanced_stats_df
['season', 'week', 'fantasypros_id', 'player', 'g', 'att', 'yds', 'ybcon', 'yacon', 'brktkl', 'tk loss', 'tk loss yds', '10+ yds', '20+ yds', '30+ yds', '40+ yds', '50+ yds', 'rec', 'tgt', 'rz tgt']

rb_fpts_perct_rost_df
['fantasypros_id', 'y/a', 'fpts', 'rost', 'season', 'week']

rb_redzone_stats_df
['fantasypros_id', 'y/a_rz', 'rec_rz', 'tgt_rz', 'rec pct_rz', 'y/r_rz', 'tgt pct_rz', 'season', 'week']


In [115]:
# Merge the NFL Python Stats and FantasyPros dataframes
# no output

# Start from the normalized baseline
rb_nfl_py_fp_merged = rb_ids_ngs_pfr_stats_sorted_normalized.copy()

# Merge: Advanced stats
rb_nfl_py_fp_merged = rb_nfl_py_fp_merged.merge(
    rb_fp_advanced_stats_df,
    on=['fantasypros_id', 'season', 'week'],
    how='left',
    suffixes=('', '_fp_adv')
)

# Merge: FPTS + ROST
rb_nfl_py_fp_merged = rb_nfl_py_fp_merged.merge(
    rb_fpts_perct_rost_df,
    on=['fantasypros_id', 'season', 'week'],
    how='left',
    suffixes=('', '_fp_fpts')
)

# Merge: Redzone stats
rb_nfl_py_fp_merged = rb_nfl_py_fp_merged.merge(
    rb_redzone_stats_df,
    on=['fantasypros_id', 'season', 'week'],
    how='left',
    suffixes=('', '_fp_rz')
)

In [116]:
# ** Merged Dataframe: NFL Python Stats and FantasyPros

# output: merged nfl py and fp validation stats
# output: merged nfl py and fp dataframe and csv file
print(f"📊 Final merged shape: {rb_nfl_py_fp_merged.shape}")

print(f"📦 FP Advanced Stats Columns Merged: {rb_fp_advanced_stats_df.shape[1] - 3} / {rb_fp_advanced_stats_df.shape[1] - 3}")
print(f"📦 FP Fantasy Points & ROST Columns Merged: 2 / 2")
print(f"📦 FP Redzone Columns Merged: {rb_redzone_stats_df.shape[1] - 3} / {rb_redzone_stats_df.shape[1] - 3}")

# Calculate merge coverage
adv_merged = rb_nfl_py_fp_merged[rb_fp_advanced_stats_df.columns.difference(['fantasypros_id', 'season', 'week']).tolist()].dropna(how='all').shape[0]
fpts_merged = rb_nfl_py_fp_merged[rb_fpts_perct_rost_df.columns.difference(['fantasypros_id', 'season', 'week']).tolist()].dropna(how='all').shape[0]
rz_merged = rb_nfl_py_fp_merged[rb_redzone_stats_df.columns.difference(['fantasypros_id', 'season', 'week']).tolist()].dropna(how='all').shape[0]

print(f"📈 FP Advanced Stats coverage: {adv_merged} rows ({adv_merged / len(rb_fp_advanced_stats_df) * 100:.2f}%)")
print(f"📈 FP FPTS/ROST coverage: {fpts_merged} rows ({fpts_merged / len(rb_fpts_perct_rost_df) * 100:.2f}%)")
print(f"📈 FP Redzone Stats coverage: {rz_merged} rows ({rz_merged / len(rb_redzone_stats_df) * 100:.2f}%)")

# Display first 5 rows
display(rb_nfl_py_fp_merged.head(10))

# *** csv file ***
# rb_nfl_py_fp_merged.to_csv("rb_nfl_py_fp_merged.csv", index=False)
# print("📁 Exported: rb_nfl_py_fp_merged.csv")

📊 Final merged shape: (11082, 99)
📦 FP Advanced Stats Columns Merged: 17 / 17
📦 FP Fantasy Points & ROST Columns Merged: 2 / 2
📦 FP Redzone Columns Merged: 6 / 6
📈 FP Advanced Stats coverage: 1118 rows (99.82%)
📈 FP FPTS/ROST coverage: 10610 rows (61.20%)
📈 FP Redzone Stats coverage: 6727 rows (95.85%)


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,carries_x,attempts,targets,target_share,receptions,rushing_yards,receiving_yards,receiving_yards_after_catch,rushing_first_downs,receiving_first_downs,rushing_tds,receiving_tds,rushing_epa,receiving_epa,rushing_2pt_conversions,receiving_2pt_conversions,player_display_name,team_abbr,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,avg_rush_yards,rush_touchdowns,player_gsis_id,player_first_name,player_last_name,player_short_name,expected_rush_yards,rush_yards_over_expected,rush_yards_over_expected_per_att,rush_pct_over_expected,game_type,team,opponent,pfr_player_name,pfr_player_id,carries_y,rushing_yards_before_contact,rushing_yards_before_contact_avg,rushing_yards_after_contact,rushing_yards_after_contact_avg,receiving_broken_tackles,player,g,att,yds,ybcon,yacon,brktkl,tk loss,tk loss yds,10+ yds,20+ yds,30+ yds,40+ yds,50+ yds,rec,tgt,rz tgt,y/a,fpts,rost,y/a_rz,rec_rz,tgt_rz,rec pct_rz,y/r_rz,tgt pct_rz
0,2017,REG,1,00-0033923,k.hunt,RB,RB,KC,13138,0ef0d0ca-2d2d-455b-ab63-a20c01303e37,16425,11841.0,4098.0,nan,3059915.0,30199.0,NaN,2079567.0,HuntKa00,kareem-hunt-1,11739.0,12295.0,214.0,30199.0,746613.0,18944.0,746613.0,Kareem Hunt,kareem hunt,17,0,5,0.147059,5,148.0,98.0,76.0,6.0,3.0,1,2,3.455881,9.309378,0,0,Kareem Hunt,KC,2.821014,35.294118,2.920235,17.0,148.0,8.705882,1.0,00-0033923,Kareem,Hunt,K.Hunt,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,Kareem Hunt,1.0,17.0,148.0,104.0,44.0,6.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0,5.0,5.0,0.0,8.7,40.599998,37.299999,3.0,1.0,1.0,100.0,3.0,100.0
1,2017,REG,1,00-0033893,d.cook,RB,RB,MIN,13128,8960d61e-433b-41ea-a7ad-4e76be87b582,16374,11796.0,4029.0,nan,3116593.0,30154.0,NaN,2130893.0,CookDa01,dalvin-cook-1,11700.0,12138.0,NaN,30154.0,824080.0,18872.0,824080.0,Dalvin Cook,dalvin cook,22,0,5,0.156250,3,127.0,10.0,12.0,4.0,0.0,0,0,-0.011037,-3.058858,0,0,Dalvin Cook,MIN,3.939449,40.909091,2.865950,22.0,127.0,5.772727,0.0,00-0033893,Dalvin,Cook,D.Cook,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.8,13.700000,0.700000,3.0,0.0,2.0,0.0,0.0,100.0
2,2017,REG,1,00-0027029,l.mccoy,RB,RB,BUF,9448,166292fc-629e-4c7b-b7bf-f572ca9eeb43,9398,4976.0,676.0,leseanmccoy/79607,12514.0,9317.0,NaN,1243187.0,McCoLe01,lesean-mccoy-1,5970.0,5168.0,NaN,9317.0,397945.0,11932.0,NaN,LeSean McCoy,lesean mccoy,22,0,6,0.222222,5,110.0,49.0,39.0,4.0,2.0,0,0,-0.711850,1.988743,0,0,LeSean McCoy,BUF,3.686273,4.545455,2.676778,22.0,110.0,5.000000,0.0,00-0027029,LeSean,McCoy,L.McCoy,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,LeSean McCoy,1.0,22.0,110.0,86.0,24.0,1.0,4.0,-6.0,4.0,3.0,0.0,0.0,0.0,5.0,6.0,0.0,5.0,15.900000,2.000000,3.2,0.0,0.0,0.0,0.0,0.0
3,2017,REG,1,00-0033045,e.elliott,RB,RB,DAL,12625,bef8b2b4-78bd-4a4d-bb5d-6b55ada9ef6a,15498,10638.0,3164.0,nan,3051392.0,29238.0,NaN,2060769.0,ElliEz00,ezekiel-elliott-1,10736.0,11265.0,NaN,29238.0,728338.0,17923.0,728338.0,Ezekiel Elliott,ezekiel elliott,24,0,5,0.131579,5,104.0,36.0,42.0,6.0,1.0,0,0,0.570525,-0.114128,0,0,Ezekiel Elliott,DAL,4.143077,20.833333,2.896261,24.0,104.0,4.333333,0.0,00-0033045,Ezekiel,Elliott,E.Elliott,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3,14.000000,3.700000,-3.5,1.0,1.0,100.0,3.0,100.0
4,2017,REG,1,00-0033856,l.fournette,RB,RB,JAX,13129,7f46a7be-286e-4bfe-8778-d03dbe600ce9,16378,11759.0,3969.0,nan,3115364.0,30117.0,NaN,2131693.0,FourLe00,leonard-fournette-1,11687.0,12132.0,NaN,30117.0,822013.0,18803.0,822013.0,Leonard Fournette,leonard fournette,26,0,3,0.142857,3,100.0,24.0,25.0,5.0,1.0,1,0,3.412744,1.308894,0,0,L

In [117]:
# compare match rates using different columns
# output: match rates show you should join the 'recent_team' column in the baseline df

# Option A: merge on 'team_abbr'
merged_team_abbr = rb_nfl_py_fp_merged.merge(
    tm_reg_playoffs_game_lines_odds,
    how='left',
    left_on=['season', 'week', 'team_abbr'],
    right_on=['season', 'week', 'team_abbr']
)

# Option B: merge on 'recent_team'
merged_recent_team = rb_nfl_py_fp_merged.merge(
    tm_reg_playoffs_game_lines_odds,
    how='left',
    left_on=['season', 'week', 'recent_team'],
    right_on=['season', 'week', 'team_abbr']  # same column name on right
)

# Count matches for each
matched_abbr = merged_team_abbr['spread'].notna().sum()
matched_recent = merged_recent_team['spread'].notna().sum()
total_rows = len(rb_nfl_py_fp_merged)

# Calculate match rates
rate_abbr = round(100 * matched_abbr / total_rows, 2)
rate_recent = round(100 * matched_recent / total_rows, 2)

print(f"Match rate using team_abbr:     {matched_abbr} / {total_rows} rows matched ({rate_abbr}%)")
print(f"Match rate using recent_team:   {matched_recent} / {total_rows} rows matched ({rate_recent}%)")

Match rate using team_abbr:     4198 / 11082 rows matched (37.88%)
Match rate using recent_team:   11056 / 11082 rows matched (99.77%)


In [118]:
# *** Dataframe of nfl python stats, fantasypros, and sportsOddsHistory ***
# Merge the nfl python stats and fantasypros dataframe with the odds / lines dataframe
# output: merged dataframe and csv file
rb_nfl_py_fp_odds_merged = rb_nfl_py_fp_merged.merge(
    tm_reg_playoffs_game_lines_odds,
    how='left',
    left_on=['season', 'week', 'recent_team'],
    right_on=['season', 'week', 'team_abbr'],
    validate='many_to_one'
)

# ✅ Row count check
print("Row check:")
print("Before merge:", len(rb_nfl_py_fp_merged))
print("After merge: ", len(rb_nfl_py_fp_odds_merged))

# ✅ Column count check
print("\nColumn check:")
print("Before merge:", len(rb_nfl_py_fp_merged.columns))
print("After merge: ", len(rb_nfl_py_fp_odds_merged.columns))

# ✅ Display sample
print("\nSample rows:")
display(rb_nfl_py_fp_odds_merged.head())

# ✅ ** csv file ***
# rb_nfl_py_fp_odds_merged.to_csv('rb_nfl_py_fp_odds_merged.csv', index=False)

Row check:
Before merge: 11082
After merge:  11082

Column check:
Before merge: 99
After merge:  107

Sample rows:


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,carries_x,attempts,targets,target_share,receptions,rushing_yards,receiving_yards,receiving_yards_after_catch,rushing_first_downs,receiving_first_downs,rushing_tds,receiving_tds,rushing_epa,receiving_epa,rushing_2pt_conversions,receiving_2pt_conversions,player_display_name,team_abbr_x,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,avg_rush_yards,rush_touchdowns,player_gsis_id,player_first_name,player_last_name,player_short_name,expected_rush_yards,rush_yards_over_expected,rush_yards_over_expected_per_att,rush_pct_over_expected,game_type,team,opponent,pfr_player_name,pfr_player_id,carries_y,rushing_yards_before_contact,rushing_yards_before_contact_avg,rushing_yards_after_contact,rushing_yards_after_contact_avg,receiving_broken_tackles,player,g,att,yds,ybcon,yacon,brktkl,tk loss,tk loss yds,10+ yds,20+ yds,30+ yds,40+ yds,50+ yds,rec,tgt,rz tgt,y/a,fpts,rost,y/a_rz,rec_rz,tgt_rz,rec pct_rz,y/r_rz,tgt pct_rz,team_abbr_y,opponent_abbr,home,role,result,score,spread,over_under
0,2017,REG,1,00-0033923,k.hunt,RB,RB,KC,13138,0ef0d0ca-2d2d-455b-ab63-a20c01303e37,16425,11841.0,4098.0,nan,3059915.0,30199.0,NaN,2079567.0,HuntKa00,kareem-hunt-1,11739.0,12295.0,214.0,30199.0,746613.0,18944.0,746613.0,Kareem Hunt,kareem hunt,17,0,5,0.147059,5,148.0,98.0,76.0,6.0,3.0,1,2,3.455881,9.309378,0,0,Kareem Hunt,KC,2.821014,35.294118,2.920235,17.0,148.0,8.705882,1.0,00-0033923,Kareem,Hunt,K.Hunt,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,Kareem Hunt,1.0,17.0,148.0,104.0,44.0,6.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0,5.0,5.0,0.0,8.7,40.599998,37.299999,3.0,1.0,1.0,100.0,3.0,100.0,KC,NE,True,Underdog,None,None,-8.0,O 47.5
1,2017,REG,1,00-0033893,d.cook,RB,RB,MIN,13128,8960d61e-433b-41ea-a7ad-4e76be87b582,16374,11796.0,4029.0,nan,3116593.0,30154.0,NaN,2130893.0,CookDa01,dalvin-cook-1,11700.0,12138.0,NaN,30154.0,824080.0,18872.0,824080.0,Dalvin Cook,dalvin cook,22,0,5,0.156250,3,127.0,10.0,12.0,4.0,0.0,0,0,-0.011037,-3.058858,0,0,Dalvin Cook,MIN,3.939449,40.909091,2.865950,22.0,127.0,5.772727,0.0,00-0033893,Dalvin,Cook,D.Cook,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.8,13.700000,0.700000,3.0,0.0,2.0,0.0,0.0,100.0,MIN,NO,False,Favorite,W,29-19,-3.0,O 47.5
2,2017,REG,1,00-0027029,l.mccoy,RB,RB,BUF,9448,166292fc-629e-4c7b-b7bf-f572ca9eeb43,9398,4976.0,676.0,leseanmccoy/79607,12514.0,9317.0,NaN,1243187.0,McCoLe01,lesean-mccoy-1,5970.0,5168.0,NaN,9317.0,397945.0,11932.0,NaN,LeSean McCoy,lesean mccoy,22,0,6,0.222222,5,110.0,49.0,39.0,4.0,2.0,0,0,-0.711850,1.988743,0,0,LeSean McCoy,BUF,3.686273,4.545455,2.676778,22.0,110.0,5.000000,0.0,00-0027029,LeSean,McCoy,L.McCoy,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,LeSean McCoy,1.0,22.0,110.0,86.0,24.0,1.0,4.0,-6.0,4.0,3.0,0.0,0.0,0.0,5.0,6.0,0.0,5.0,15.900000,2.000000,3.2,0.0,0.0,0.0,0.0,0.0,BUF,NYJ,False,Favorite,W,21-12,-7.0,U 42
3,2017,REG,1,00-0033045,e.elliott,RB,RB,DAL,12625,bef8b2b4-78bd-4a4d-bb5d-6b55ada9ef6a,15498,10638.0,3164.0,nan,3051392.0,29238.0,NaN,2060769.0,ElliEz00,ezekiel-elliott-1,10736.0,11265.0,NaN,29238.0,728338.0,17923.0,728338.0,Ezekiel Elliott,ezekiel elliott,24,0,5,0.131579,5,104.0,36.0,42.0,6.0,1.0,0,0,0.570525,-0.114128,0,0,Ezekiel Elliott,DAL,4.143077,20.833333,2.896261,24.0,104.0,4.333333,0.0,00-0033045,Ezekiel,Elliott,E.Elliott,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3,14.000000,3.700000,-3.5,1.0,1.0,100.0,3.0,100.0,DAL,NYG,False,Favorite,W,19-3,-6.0,U 46.5
4,2017,REG,1,00-0033856,l.fournette,RB,RB,JAX,13129,7f46a7be-286e-4bfe-8778-d03dbe600ce9,16378,117

In [119]:
# create key values for merging and match tests
# output: sample dataframe of matched rows and validation tests

# Standardize and construct merge keys safely ---
rb_fd_dk_salary_2017_current_df = rb_fd_dk_salary_2017_current_df.copy()
rb_fd_dk_salary_2017_current_df.loc[:, 'merge_name'] = (
    rb_fd_dk_salary_2017_current_df['player']
    .str.lower()
    .str.replace(r'[^a-z ]', '', regex=True)
)

rb_fd_dk_salary_2017_current_df.loc[:, 'merge_key'] = (
    rb_fd_dk_salary_2017_current_df['merge_name'] + '_' +
    rb_fd_dk_salary_2017_current_df['season'].astype(str) + '_' +
    rb_fd_dk_salary_2017_current_df['week'].astype(str)
)

rb_nfl_py_fp_odds_merged = rb_nfl_py_fp_odds_merged.copy()
rb_nfl_py_fp_odds_merged.loc[:, 'merge_key'] = (
    rb_nfl_py_fp_odds_merged['merge_name'] + '_' +
    rb_nfl_py_fp_odds_merged['season'].astype(str) + '_' +
    rb_nfl_py_fp_odds_merged['week'].astype(str)
)

# Inner join for validation ---
cols_salary = ['season', 'week', 'merge_key', 'merge_name', 'dk_salary', 'fd_salary']
test_matches = rb_nfl_py_fp_odds_merged.merge(
    rb_fd_dk_salary_2017_current_df[cols_salary],
    on=['season', 'week', 'merge_key'],
    how='inner',
    suffixes=('_main', '_salary')
)

# Match statistics
print(f"✅ Inner join matches: {len(test_matches):,}")

num_matches = test_matches['merge_key'].nunique()
total_salary_rows = rb_fd_dk_salary_2017_current_df['merge_key'].nunique()
match_percent = (num_matches / total_salary_rows) * 100

print(f"✅ Match rate: {num_matches:,} / {total_salary_rows:,} salary rows matched ({match_percent:.2f}%)")

# --- STEP 4: Display random sample for visual verification ---
sample_matches = test_matches.sample(10, random_state=42)[
    ['season', 'week', 'merge_key', 'merge_name_main', 'merge_name_salary', 'dk_salary', 'fd_salary']
]

display(sample_matches)

✅ Inner join matches: 10,216
✅ Match rate: 10,173 / 14,577 salary rows matched (69.79%)


,season,week,merge_key,merge_name_main,merge_name_salary,dk_salary,fd_salary
2045,2018,11,joe mixon_2018_11,joe mixon,joe mixon,7100,7300
4816,2020,14,raheem mostert_2020_14,raheem mostert,raheem mostert,6200,7100
5012,2020,16,artavis pierce_2020_16,artavis pierce,artavis pierce,4000,4500
33,2017,1,bilal powell_2017_1,bilal powell,bilal powell,4900,6300
2926,2019,5,bilal powell_2019_5,bilal powell,bilal powell,3000,4500
8236,2023,11,ty chandler_2023_11,ty chandler,ty chandler,4800,5900
3317,2019,11,tevin coleman_2019_11,tevin coleman,tevin coleman,6100,6700
4241,2020,5,keshawn vaughn_2020_5,keshawn vaughn,keshawn vaughn,4000,5000
5202,2021,2,najee harris_2021_2,najee harris,najee harris,7900,6100
304,2017,4,peyton barber_2017_4,peyton barber,peyton barber,3000,4500


In [120]:
# Compare fuzzy similarity between merge_name_main and merge_name_salary
# Note: key_match should = rapidfuzz so rapidfuzz not necessary but good for verification
# output: comparison of the two methods
test_matches['fuzzy_ratio'] = test_matches.apply(
    lambda row: fuzz.token_sort_ratio(row['merge_name_main'], row['merge_name_salary']),
    axis=1
)

# Check exact vs fuzzy
exact_matches = (test_matches['merge_name_main'] == test_matches['merge_name_salary']).sum()
fuzzy_matches_90plus = (test_matches['fuzzy_ratio'] >= 90).sum()

print(f"🎯 Exact name matches: {exact_matches:,} / {len(test_matches):,}")
print(f"👌 Fuzzy matches ≥ 90: {fuzzy_matches_90plus:,} / {len(test_matches):,}")

🎯 Exact name matches: 10,216 / 10,216
👌 Fuzzy matches ≥ 90: 10,216 / 10,216


In [121]:
# *** Final Merge***
# output: complete merge of nfly python stats, fantasypros, sportsoddshistory, and bigdataball salary data
# Merge salary data into baseline
rb_nfl_py_fp_odds_salary_merged = rb_nfl_py_fp_odds_merged.merge(
    rb_fd_dk_salary_2017_current_df[['season', 'week', 'merge_key', 'dk_salary', 'fd_salary']],
    on=['season', 'week', 'merge_key'],
    how='left'
)

# Summary match stats
num_salary_matches = rb_nfl_py_fp_odds_salary_merged['dk_salary'].notna().sum()
total_rows = len(rb_nfl_py_fp_odds_salary_merged)
print(f"💰 Salaries merged into: {num_salary_matches:,} / {total_rows:,} rows ({(num_salary_matches / total_rows) * 100:.2f}%)")

# Optional: remove merge_key if you're done with it
# rb_nfl_py_fp_odds_salary_merged.drop(columns=['merge_key'], inplace=True)

# ** csv file **
# rb_nfl_py_fp_odds_salary_merged.to_csv("rb_nfl_py_fp_odds_salary_merged.csv", index=False)
# print("📁 Saved as: rb_nfl_py_fp_odds_salary_merged.csv")

💰 Salaries merged into: 10,216 / 11,125 rows (91.83%)


In [122]:
### End: Data Normalization and Merge Process ###

In [123]:
### Begin: Feature Engineering ###

In [124]:
# drop unnecessary columns
# output: updated dataframe and csv file with modified columns
# List of columns to drop
cols_to_drop = [
    'player_id', 'player_name', 'position_group', 'mfl_id', 'sportradar_id',
    'fantasypros_id', 'pff_id', 'sleeper_id', 'nfl_id', 'espn_id', 'yahoo_id',
    'fleaflicker_id', 'cbs_id', 'pfr_id', 'cfbref_id', 'rotowire_id',
    'rotoworld_id', 'ktc_id', 'stats_id', 'stats_global_id', 'fantasy_data_id',
    'swish_id', 'merge_name', 'player_gsis_id', 'player_first_name',
    'player_last_name', 'player_short_name', 'game_type', 'team', 'opponent',
    'pfr_player_name', 'pfr_player_id', 'player', 'team_abbr_y', 'merge_key'
]

# Initial state
initial_shape = rb_nfl_py_fp_odds_salary_merged.shape
initial_col_count = initial_shape[1]

print("Before modification:")
print(f"Shape: {initial_shape}")
print(f"Total Columns: {initial_col_count}")

# Drop only existing columns
cols_existing = [col for col in cols_to_drop if col in rb_nfl_py_fp_odds_salary_merged.columns]
rb_nfl_py_fp_odds_salary_merged_mod_cols = rb_nfl_py_fp_odds_salary_merged.drop(columns=cols_existing)

# Post-drop validation
final_shape = rb_nfl_py_fp_odds_salary_merged_mod_cols.shape
final_col_count = final_shape[1]
dropped_count = len(cols_existing)
expected_final_col_count = initial_col_count - dropped_count

print("\nAfter modification:")
print(f"Shape: {final_shape}")
print(f"Total Columns After Drop: {final_col_count}")
print(f"Number of Columns Dropped: {dropped_count}")
print(f"Expected Final Column Count: {expected_final_col_count}")
print(f"Column Count Validation Passed: {final_col_count == expected_final_col_count}")

# Confirm none of the dropped columns remain
all_dropped = all(col not in rb_nfl_py_fp_odds_salary_merged_mod_cols.columns for col in cols_to_drop)
print(f"All Specified Columns Successfully Dropped: {all_dropped}")

# ** csv file **
output_csv_path = "rb_nfl_py_fp_odds_salary_merged_mod_cols.csv"
# rb_nfl_py_fp_odds_salary_merged_mod_cols.to_csv(output_csv_path, index=False)
# print(f"\nModified dataframe saved to {output_csv_path}")

Before modification:
Shape: (11125, 110)
Total Columns: 110

After modification:
Shape: (11125, 75)
Total Columns After Drop: 75
Number of Columns Dropped: 35
Expected Final Column Count: 75
Column Count Validation Passed: True
All Specified Columns Successfully Dropped: True


In [125]:
# reorder columns
# Define new column order
priority_cols = [
    'season', 'season_type', 'week', 'name', 'position', 'recent_team',
    'fpts', 'dk_salary', 'fd_salary', 'rost',
    'opponent_abbr', 'home', 'role', 'result', 'score', 'spread', 'over_under'
]

# Capture original state
original_shape = rb_nfl_py_fp_odds_salary_merged_mod_cols.shape
original_columns = rb_nfl_py_fp_odds_salary_merged_mod_cols.columns.tolist()

print("Original Shape:", original_shape)
print("Original First 20 Columns:", original_columns[:20])

# Build final column order
remaining_cols = [col for col in original_columns if col not in priority_cols]
final_col_order = priority_cols + remaining_cols

# Reorder the columns
rb_nfl_py_fp_odds_salary_merged_mod_cols = rb_nfl_py_fp_odds_salary_merged_mod_cols[final_col_order]

# Post-reorder validation
new_shape = rb_nfl_py_fp_odds_salary_merged_mod_cols.shape
new_columns = rb_nfl_py_fp_odds_salary_merged_mod_cols.columns.tolist()

print("\nNew Shape:", new_shape)
print("New First 20 Columns:", new_columns[:20])

# Validation Checks
print("\n✅ Shape Integrity:", original_shape == new_shape)
print("✅ Column Count Matches:", len(original_columns) == len(new_columns))
print("✅ All Columns Preserved:", set(original_columns) == set(new_columns))

# ** csv file **
output_reordered_csv = "rb_nfl_py_fp_odds_salary_merged_mod_cols_reordered.csv"
# rb_nfl_py_fp_odds_salary_merged_mod_cols.to_csv(output_reordered_csv, index=False)
# print(f"\nReordered dataframe saved to {output_reordered_csv}")

Original Shape: (11125, 75)
Original First 20 Columns: ['season', 'season_type', 'week', 'position', 'recent_team', 'name', 'carries_x', 'attempts', 'targets', 'target_share', 'receptions', 'rushing_yards', 'receiving_yards', 'receiving_yards_after_catch', 'rushing_first_downs', 'receiving_first_downs', 'rushing_tds', 'receiving_tds', 'rushing_epa', 'receiving_epa']

New Shape: (11125, 75)
New First 20 Columns: ['season', 'season_type', 'week', 'name', 'position', 'recent_team', 'fpts', 'dk_salary', 'fd_salary', 'rost', 'opponent_abbr', 'home', 'role', 'result', 'score', 'spread', 'over_under', 'carries_x', 'attempts', 'targets']

✅ Shape Integrity: True
✅ Column Count Matches: True
✅ All Columns Preserved: True


In [126]:
# Apply Rolling Averages and Aggregates - 3,5,and 7 week averages
# Ouput: updated dataframe with aggregates (optional csv file)

# Start from sorted copy of the main DF
rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg = (
    rb_nfl_py_fp_odds_salary_merged_mod_cols
    .sort_values(by=['name', 'season', 'week'])
    .reset_index(drop=True)
)

# Feature map: full column -> short prefix
feature_map = {
    'targets': 'tgt',
    'receptions': 'rec',
    'rushing_yards': 'rush_yds',
    'fpts': 'fpts'
}

windows = [3, 5, 7]

# Apply rolling averages and lag features
for full_col, short in feature_map.items():
    # Group once
    grouped = rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.groupby(['name', 'season'])

    # Rolling averages using apply (preserves group boundaries)
    for window in windows:
        col_name = f"{short}_{window}wk_avg"
        rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg[col_name] = grouped[full_col].apply(
            lambda x: x.rolling(window=window, min_periods=window).mean().shift(1)
        ).reset_index(drop=True)

    # Lag feature (1-game lookback)
    lag_col = f"{short}_lag_1"
    rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg[lag_col] = grouped[full_col].shift(1).reset_index(drop=True)

# Final integrity check
print("✅ Final shape:", rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.shape)

# ** csv file **
# rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.to_csv(
#     "rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.csv",
#     index=False,
#     float_format="%.2f"
# )
# print("📤 Exported to: rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.csv")


✅ Final shape: (11125, 91)


In [127]:
# Validation tests of aggregates
# output: there should be no aggregates prior to week 4

# Structural Check — No aggregates in first 3 weeks of a season
def check_early_aggregates(df, cols, earliest_week=4):
    early = df[df['week'] < earliest_week]
    violations = early[cols].notna().sum()
    print("🚨 Aggregates present before week", earliest_week)
    print(violations[violations > 0])

# Boundary Check — Rolling aggregates must reset per season
def check_season_boundaries(df, col_prefix):
    errors = []
    for short in col_prefix:
        col_name = f'{short}_3wk_avg'
        season_transitions = df.groupby(['name'])['season'].diff().fillna(0)
        cross_season_rows = df[season_transitions != 0]
        if cross_season_rows[col_name].notna().any():
            errors.append(col_name)
    if errors:
        print("❌ Rolling values leaked across seasons:", errors)
    else:
        print("✅ No cross-season leakage detected.")

# Shape check
def check_shape(df, expected_cols_added):
    print("✅ Final shape:", df.shape)
    print("✅ Final columns:", df.columns[-expected_cols_added:])

# === Apply Checks ===
rolling_cols = [f"{short}_{w}wk_avg" for short in ['tgt', 'rec', 'rush_yds', 'fpts'] for w in [3, 5, 7]]
lag_cols = [f"{short}_lag_1" for short in ['tgt', 'rec', 'rush_yds', 'fpts']]
check_early_aggregates(rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg, rolling_cols)
check_season_boundaries(rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg, ['tgt', 'rec', 'rush_yds', 'fpts'])
check_shape(rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg, expected_cols_added=len(rolling_cols + lag_cols))

🚨 Aggregates present before week 4
Series([], dtype: int64)
✅ No cross-season leakage detected.
✅ Final shape: (11125, 91)
✅ Final columns: Index(['tgt_3wk_avg', 'tgt_5wk_avg', 'tgt_7wk_avg', 'tgt_lag_1', 'rec_3wk_avg', 'rec_5wk_avg', 'rec_7wk_avg', 'rec_lag_1', 'rush_yds_3wk_avg', 'rush_yds_5wk_avg', 'rush_yds_7wk_avg', 'rush_yds_lag_1', 'fpts_3wk_avg', 'fpts_5wk_avg', 'fpts_7wk_avg', 'fpts_lag_1'], dtype='object')


In [128]:
# Add Trend Features (deltas) - recent performance over / under (3wk, 5wk, 7wk) averages
# output: updated dataframe with deltas (optional csv file)

# new dataframe
rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend = rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.copy()

# Column map
feature_map = {
    'targets': 'tgt',
    'receptions': 'rec',
    'rushing_yards': 'rush_yds',
    'fpts': 'fpts'
}

windows = [3, 5, 7]

# Create delta (deviation from trend) features
for full_col, short in feature_map.items():
    for window in windows:
        avg_col = f"{short}_{window}wk_avg"
        delta_col = f"{short}_{window}wk_delta"
        rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend[delta_col] = (
            rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend[full_col] -
            rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend[avg_col]
        )

# Summary and export
delta_cols = [f"{short}_{w}wk_delta" for short in feature_map.values() for w in windows]
print("✅ Added delta columns:", delta_cols)
print("✅ Final shape:", rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend.shape)

# ** csv file **
# rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend.to_csv(
#     "rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend.csv",
#     index=False,
#     float_format="%.2f"
# )
# print("📤 Exported to: rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend.csv")


✅ Added delta columns: ['tgt_3wk_delta', 'tgt_5wk_delta', 'tgt_7wk_delta', 'rec_3wk_delta', 'rec_5wk_delta', 'rec_7wk_delta', 'rush_yds_3wk_delta', 'rush_yds_5wk_delta', 'rush_yds_7wk_delta', 'fpts_3wk_delta', 'fpts_5wk_delta', 'fpts_7wk_delta']
✅ Final shape: (11125, 103)


In [129]:
# Create boolean columns
# output: updated dataframe with booleans (optional csv file)

# Start from the previous trend-enhanced dataframe
rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool = rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend.copy()

# Define boolean columns as 0/1 integers
rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['tgt_ge_5'] = (rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['targets'] >= 5).astype(int)
rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['tgt_ge_7'] = (rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['targets'] >= 7).astype(int)
rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['rush_ge_5'] = (rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['receptions'] >= 5).astype(int)
rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['rush_ge_7'] = (rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['receptions'] >= 7).astype(int)

rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['target_share_ge_20'] = (rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['target_share'] >= 0.2).astype(int)
rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['target_share_ge_30'] = (rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['target_share'] >= 0.3).astype(int)

rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['over_100_yds'] = (rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['rushing_yards'] >= 100).astype(int)
rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['double_digit_targets'] = (rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['targets'] >= 10).astype(int)
rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['boom_week'] = (rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['fpts'] >= 20).astype(int)
rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['bust_week'] = (rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['fpts'] < 5).astype(int)

# If 'home' is already boolean, convert to int
rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['is_home_game'] = rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['home'].astype(int)

# Final shape and column check
print("✅ Final shape:", rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool.shape)
print("✅ New boolean columns added.")

# ** csv file **
# rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool.to_csv(
#     "rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool.csv",
#     index=False,
#     float_format="%.2f"
# )
# print("📤 Exported to: rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool.csv")

✅ Final shape: (11125, 114)
✅ New boolean columns added.


In [130]:
# *** Dataframe: this can be used as the final dataframe but the filename is long ***

# split the over / under column into two columns: o_u and total
# output: updated dataframe with o_u and total columns (optional csv file) 

# Copy from final boolean-enriched dataframe
rb_nfl_py_fp_odds_salary_features = rb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool.copy()

# Extract 'O' or 'U' and map to "over"/"under"
rb_nfl_py_fp_odds_salary_features['O_U'] = (
    rb_nfl_py_fp_odds_salary_features['over_under']
    .str[0]
    .map({'O': 'over', 'U': 'under'})
)

# Extract the numeric total (handles int or float)
rb_nfl_py_fp_odds_salary_features['Total'] = (
    rb_nfl_py_fp_odds_salary_features['over_under']
    .str.extract(r'(\d+\.?\d*)')[0]
    .astype(float)
)

# Validation
print("✅ Final shape:", rb_nfl_py_fp_odds_salary_features.shape)
print("✅ Sample 'O_U' values:", rb_nfl_py_fp_odds_salary_features['O_U'].unique())
print("✅ Sample 'Total' values:", rb_nfl_py_fp_odds_salary_features['Total'].dropna().unique()[:5])

# ** csv file **
# rb_nfl_py_fp_odds_salary_features.to_csv(
#     "rb_nfl_py_fp_odds_salary_features.csv",
#     index=False,
#     float_format="%.2f"
# )
# print("📤 Exported to: rb_nfl_py_fp_odds_salary_features.csv")

✅ Final shape: (11125, 116)
✅ Sample 'O_U' values: ['over' 'under' nan]
✅ Sample 'Total' values: [44.  51.  56.5 55.  55.5]


In [131]:
### End: Feature Engineering ###

In [132]:
### Begin: Final Dataframe ###

In [133]:
### *** Final Dataframe *** ###

# Rename final dataframe
rb_nfl_df_sorted_final = rb_nfl_py_fp_odds_salary_features.copy()

# **csv file **
rb_nfl_df_sorted_final.to_csv(
    "rb_nfl_df_sorted_final.csv",
    index=False,
    float_format="%.2f"
)

print("✅ Final dataframe saved as 'rb_nfl_df_sorted_final.csv'")
print("✅ Final shape:", rb_nfl_df_sorted_final.shape)

✅ Final dataframe saved as 'rb_nfl_df_sorted_final.csv'
✅ Final shape: (11125, 116)


In [134]:
### End: Final Dataframe ###

In [135]:
### Begin: Final Dataframe Summary Statistics ###

In [136]:
# Final Dataframe Summary Statistics
# Output: csv output of the dataframe summary statistics on data types and missing values

# Total columns and datatypes
total_cols = rb_nfl_df_sorted_final.shape[1]
dtypes_summary = rb_nfl_df_sorted_final.dtypes.value_counts().sort_values(ascending=False)
dtypes_percent = (dtypes_summary / total_cols * 100).round(2)

print("🧠 Data Type Distribution (by count and %):")
for dtype, count in dtypes_summary.items():
    print(f"{str(dtype):15} {count:>3} columns  ({dtypes_percent[dtype]:>5.1f}%)")

# Object-type columns
object_cols = rb_nfl_df_sorted_final.select_dtypes(include='object').columns.tolist()
print(f"\n🚨 Object-type columns found ({len(object_cols)} total / {total_cols} columns):")
print(object_cols)

# Missing value summary by count and % of rows
row_count = len(rb_nfl_df_sorted_final)
na_counts = rb_nfl_df_sorted_final.isna().sum()
na_percent = (na_counts / row_count * 100).round(2)
na_summary = pd.DataFrame({'Missing': na_counts, 'Percent': na_percent})
na_summary = na_summary[na_summary['Missing'] > 0].sort_values(by='Percent', ascending=False)

print(f"\n⚠️ Missing Value Summary (non-zero only) — Top {len(na_summary)} columns:")
display(na_summary)

# ** csv file **
na_summary.to_csv("rb_df_final_summary_stats.csv")

🧠 Data Type Distribution (by count and %):
float64          51 columns  ( 44.0%)
float32          31 columns  ( 26.7%)
object           12 columns  ( 10.3%)
int64            11 columns  (  9.5%)
int32            10 columns  (  8.6%)
bool              1 columns  (  0.9%)

🚨 Object-type columns found (12 total / 116 columns):
['season_type', 'name', 'position', 'recent_team', 'opponent_abbr', 'role', 'result', 'score', 'over_under', 'player_display_name', 'team_abbr_x', 'O_U']

⚠️ Missing Value Summary (non-zero only) — Top 79 columns:


,Missing,Percent
receiving_broken_tackles,11125,100.00
50+ yds,10000,89.89
rz tgt,10000,89.89
tgt,10000,89.89
rec,10000,89.89
...,...,...
fpts,472,4.24
y/a,472,4.24
rost,472,4.24
O_U,109,0.98


In [137]:
### End: Final Dataframe Summary Statistics ###